# Custom NER algorithm for product extraction - Spacy transformerd.

## Introduction

In this Jupyter Notebook, I developed a custom NER (Name Entity Recognition) model using the latest version of Spacy. As an input to the model, I used the previously created annotations (in the [NER-Annotator](https://tecoholic.github.io/ner-annotator/)) and downloaded the configuration files from the Spacy website ([base_config.cfg](https://spacy.io/usage/training#config-overrides) - which contains the configurations of the Spacy model). The model used is Roberta - a transformer based architecture optimized for accuracy.

In [4]:
#Import all required libraries
import spacy
from spacy.tokens import DocBin
import random
import time
import numpy as np
import sys
from spacy import displacy
from itertools import chain
import matplotlib.pyplot as plt 
from matplotlib.ticker import MaxNLocator
import pandas as pd
import json
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from spacy.scorer import Scorer
!pip install spacy-transformers
import spacy_transformers

In [2]:
data = [('Factory Buys Euro Top Mattress King Description Pamper yourself with Giselle Bedding Como Series Euro Top Mattress that will give you a soothing sleep night after night. Incorporating the cutting-edge technology, the lavish construction addresses all elements that will offer you a comfortable and luxurious experience Euro top padding with high-density foam and independent coil system made to provide you with an undisturbed sleep all night long. Features Plush Euro Top padding 5 zoned pocket springs Different level of support Independent coil system Excellent support and sublime comfort 6 working turn coil system High-density foam Medium firmness Hypo allergic material Luxury knitted jacquard fabric Side handles for positioning Vacuum packaging: The item will expand to its normal shape within 1 day after unpacking Specifications Size: King Number of coils: 928 Firmness: 5.0 / Medium Thickness: 32cm Dimensions: 203cm x 183cm x 32cm Package Contents 1 x Euro Top Mattress Care Instructions Do not machine wash Do not bleach Do not iron Do not dry clean Do not saturate with water or other cleaning liquids Treat spills with a cloth and dry with blow dryer May be spot cleaned with water or soap Rotate mattress regularly Vacuum using low suction',
  {'entities': [(0, 35, 'PRODUCT'),
    (97, 114, 'PRODUCT'),
    (965, 982, 'PRODUCT')]}),
 ('Handmade by Beadlight in the UK, the Cirrus collection was originally designed for use in private jet aircraft, and first and business class cabins. The range has proved incredibly successful in hotels, private residences and mega yachts. Built to the exacting standards demanded by the aeropsace industry, each Beadlight reading light is made by hand and wrapped in English leather with hand stitching. Ideal as a bedhead light, the luxurious fitting provides two brightness settings for reading. Can be mounted to existing walls and bed heads. Simple installation is required by an electrician. Energy Efficient. These LED lights become the perfect solution to make substantial energy savings. The wall lights use 1.4 watts of energy and require a 12v transformer, not included. Previously available only to architects and interior designers, the Beadlight collection is now available to the public exclusively through Dunlin. Overall Length from wall 45cm Technical PDF',
  {'entities': [(312, 335, 'PRODUCT')]}),
 ('The Hamar Plant Stand is a wonderful accessory that holds at least three pot plants or other objects such as vases, sculptural objects or lamps. Use it to fill a corner or that tricky spot, bring height and greenery to your room. Dimensions: 65 W at widest point x 92 H Materials: Solid Ash, Clear Lacquer',
  {'entities': [(4, 21, 'PRODUCT')]}),
 ('Danai Dining Chair $345.00 The Danai Dining Chair is upholstered in fabric with steel legs which have also been upholstered in the same fabric. Available in 8 colours. 54cm Wide X 57cm Deep X 80cm High',
  {'entities': [(0, 18, 'PRODUCT'), (31, 49, 'PRODUCT')]}),
 ("Gift Card Shopping for someone else but not sure what to give them Give them the gift of choice with a Donna's Home Furnishings gift card. Gift cards are delivered to you, the purchaser, by email and contain instructions to redeem them at checkout. To send a gift card to someone, just forward them the email. Our gift cards have no additional processing fees.",
  {'entities': [(103, 138, 'PRODUCT')]}),
 ('Carlis a classic, Victorian-inspired accent stool, that can also be used as an small ottoman, vanity seat, planter table or footrest. With its beautifully detailed wooden legs, and a tufted like center, Carlis upholstered in a soft, carefully-selected velvet, and will bring a truly sophisticated gust of fresh air into your home, office, or studio. Shades range from classic black to fiery red to best match your desired look. DIMENSIONS: OVERALL DIMENSIONS (H x W):15 x 23 MORE INFO: COMPOSITION: Velvet LEG COUNT: 3 ASSEMBLY: Light assembly required. MODEL: OT02-VV PLEASE NOTE: True color may vary based on your monitor settings.',
  {'entities': [(0, 6, 'PRODUCT'), (203, 209, 'PRODUCT')]}),
 ("The beautiful Mason Chair can work in any setting; from contemporary to casual and even formal. Made in a beautiful black leather and natural wood leather. These chairs are perfect for surrounding your dining table. Details Dimensions Shipping Black Leather Wood P-23032 22'' Wide x 24'' Deep x 34'' High",
  {'entities': [(14, 25, 'PRODUCT')]}),
 ('The coordinating Azara bedroom chests each feature an accent drawer faced in solid exotic Tiger bamboo, angled tapered legs, hidden under mount drawer glides and soft closing drawers. Variation in grain and color enrich the natural beauty of bamboo furniture and make each piece unique. Slight differences in bamboo shade colors are signs that the furniture has been crafted from solid bamboo and is considered a benchmark of quality. Expertly crafted in earth friendly solid bamboo, Greenington combines beautiful design, rich colors and a winning sustainability story. Available in Queen, Eastern King, and California King sizes, in both caramelized and rich Sable finish.',
  {'entities': [(17, 37, 'PRODUCT')]}),
 ('Aaron Retro Sofa Further your sensibilities with Aaron. Elegantly inset with a centered line of tufted buttons and fine polyester upholstery, Aaron enhances an ambiance of exceptionalism with a refined design that spans the ages. Aaron features dense foam cushioning, walnut stained wood legs with non-marking foot caps, and a supportive seating experience. This modern take on mid-century design is perfect for living rooms, lounge spaces, and reception areas.',
  {'entities': [(0, 16, 'PRODUCT'),
    (49, 55, 'PRODUCT'),
    (142, 147, 'PRODUCT'),
    (230, 235, 'PRODUCT')]}),
 ('The Enso platform bed invites peaceful slumber with a sophisticated, Zen-inspired aesthetic. Smooth, clean lines create a platform bed that exudes quiet serenity. The higher headboard enhances back support for reading and laptop-surfing, or working in bed.',
  {'entities': [(4, 21, 'PRODUCT')]}),
 ('Product Details Designed by Chicago architect Stephen Katz in 2013, the Eliza blends a beautiful technically derived form with superior function and movement. The unique blade shape is designed to maximize air movement at the outer edge of the blade. The result is more efficient air velocity rings, less blade drag and greater motor optimization. Equally important is the Eliza fanï¿½s stationary visual statement which combines modern utility with minimalist geometry. Dimensions 56 D Additional Details ï¿½ï¿½Energy efficient, ultra-quiet, six-speed and reversible DC motor with new reliable technology. ï¿½ï¿½Damp location finishes: brushed nickel and gloss white. ï¿½ï¿½Blades available in Gloss White, Brushed Nickel, Walnut, Barnwood or Gray Ash. ï¿½ï¿½Constructed of cast aluminum, polycarbonate and heavy stamped steel.',
  {'entities': [(72, 77, 'PRODUCT')]}),
 ('Enveloping your space with an airy feel, our Ariane 6 drawer chest is all about the details from the fluted drawer fronts to brushed gold handles. Accentuating your bedroom or living room space with its fresh colour and textural finish, scaled for both maximum storage and design presence. The Ariane chest of drawers is the epitome of function and beauty that will instantly elevate your colour scheme and design.',
  {'entities': [(45, 66, 'PRODUCT'), (294, 300, 'PRODUCT')]}),
 ('Give a Gift Card: Any Size! Shopping for someone else but not sure what to give them Give them the gift of choice with a Yellow Leaf Hammocks Gift Card! You pick a size + they pick the color they love most! Gift cards are delivered by email and contain instructions to redeem them at checkout. Our gift cards have no additional processing fees and include free shipping for your recipient.',
  {'entities': [(121, 151, 'PRODUCT')]}),
 ('In collaboration with Project Nursery, we introduce the beautifully modern Wooster 3-in-1 Convertible Crib. The Woosterï¿½s clean lines and bold angles give it a sleek, minimalist design while itï¿½s detailed multi-level paneling make for a distinctive centerpiece that fits effortlessly into any space. A removable front panel allows the Wooster Crib to seamlessly transform into a stylish daybed andwith optional conversion railstoddler bed, letting your crib adapt to your familyï¿½s changing needs. The Wooster comes in five great colorways, including our brand new Blanc/Noir, a striking black and white dual tone for a truly iconic look. In collaboration with Project Nursery, we introduce the beautifully modern Wooster 3-in-1 Convertible Crib. The Woosterï¿½s clean lines and bold angles give it a sleek, minimalist design while itï¿½s detailed multi-level paneling make for a distinctive centerpiece that fits effortlessly into any space. A removable front panel allows the Wooster Crib to seamlessly transform into a stylish daybed andwith optional conversion railstoddler bed, letting your crib adapt to your familyï¿½s changing needs.',
  {'entities': [(75, 107, 'PRODUCT'),
    (112, 121, 'PRODUCT'),
    (335, 347, 'PRODUCT'),
    (713, 745, 'PRODUCT'),
    (973, 985, 'PRODUCT')]}),
 ('Esme Natural Rattan Day Bed $1,400.00 Esme rattan day bed APRIL DELIVERY Esme measures approximately 198cm long x 76cm high x 96cm deep. Mattress measures 92cm wide x 188cm long x 10cm thick and has a removable cover. This is a natural rattan product and colour variation will occur. Available for pickup from out Adelaide warehouse or please contact us for a shipping quote.',
  {'entities': [(0, 27, 'PRODUCT'),
    (38, 57, 'PRODUCT'),
    (73, 77, 'PRODUCT')]}),
 ('Handmade solid Oak bench. Beautifully simple with a gently curved seat, softened edges and tapered legs.',
  {'entities': [(15, 25, 'PRODUCT')]}),
 ("Guardsman's comprehensive Furniture Protection Plan protects your new fabric, leather, and wood furniture investment for a period of 5 years. Guardsman Gold Complete Plus Furniture Protection Plans cover the accidental stains and damage of a life well-lived. When you need us, weï¿½ll send an expert technician to restore your fabric, leather, wood, and other hard-surface furniture to a look youï¿½ll love a long, long time *Guardsman plan must be purchased prior to furniture delivery. Guardsman Gold Complete Plus Plan Brochure Guardsman Gold Complete Plus Plan Contract",
  {'entities': [(0, 51, 'PRODUCT'),
    (142, 197, 'PRODUCT'),
    (484, 517, 'PRODUCT'),
    (527, 560, 'PRODUCT')]}),
 ('Product Description Transform your bedroom into a forest retreat with this Tacoma chest. Five deep drawers with simple brass hardware offer plenty of room to organize your clothing and linens. From the Kenlin drawer glides and English dovetail joints, this solid pine chest offers the sturdy construction you want for such an important bedroom staple. Most importantly, the look is simple yet supremely stylish ï¿½ pine veneers which have been hand distressed give this wood frame the dark and rustic look of the west coast woods. Dimensions Width: 33 (83.8 cm)Height: 50.25 (127.6 cm)Depth: 16.63 (42.2 cm) Shipping Dimensions Width: 35.75 (90.8 cm)Height: 53 (134.6 cm)Depth: 19.25 (48.9 cm) Weight 121.25 lbs (55.0 kg) Unique Features Care Instructions Veneers General Details Finish Colour Antique Pine Construction Details Primary Construction Material Pine Furniture made with pine has a natural, creamy yellow colouring that looks elegant and adds warmth to the home. Pine has a tight grain pattern with a smooth surface to hold finishes well. Finish Method Veneers By gluing a thin layer of beautiful wood or stone onto quality construction materials, veneers provide a striking texture and finish to furniture. Veneers wonï¿½t warp, crack or split over time, which keeps your products looking as good as the day you bought them. Hardware Brass Knobs Storage Details Number of Drawers 5 Drawer Glides Kenlin With over a century of experience, The Handy Kenlin Group (Kenlin) are the masters of creating high-quality drawer glides. Kenlin makes a wide variety of drawer hardware and accessories, perfect for making sure everything moves smoothly.',
  {'entities': [(75, 88, 'PRODUCT'),
    (202, 222, 'PRODUCT'),
    (227, 250, 'PRODUCT'),
    (1391, 1411, 'PRODUCT')]}),
 ('Shop Toli Lamp  20 cm EU Garden Green 405.00USD Specifications Maintenance Light source Materials: Powder-Coated Steel Size & Weight: H: 28 x : 20 cm 0,3 kg 0,3 kg Product Information: Made in powder-coated steel with a gloss finish. Including plastic canopy for the EU version; in white for the Pearl Grey lamp, in black for the Parterre Brown and Garden Green lamp. Maintenance: Clean with damp cloth. Light source: Standard E27 bulb socket. 4 m textile-covered cord. About Toli Lamp The name Toli is an abbreviation of Tivoli. Inspired by multiple exterior details of Tivoliï¿½s architecture, such as windows, facades and grids, the Toli pendants have an architectural look with a glossy finish. Sharp lines and an ultra-modern silhouette define the design. Fine, oblong holes at the bottom of the lampshade add a graphic expression and create playful patterns of light and shadow in the room. Bring the Tivoli feeling home with design objects by Normann Copenhagen. Bring the Tivoli feeling home with design objects by Normann Copenhagen.',
  {'entities': [(5, 14, 'PRODUCT'),
    (476, 485, 'PRODUCT'),
    (634, 647, 'PRODUCT')]}),
 ('Gift Cards Shopping for someone else but not sure what to give them The gift of choice with a DECORUM Gift Card of course. Gift Cards are delivered by email and contain instructions to redeem them at checkout. The perfect gift for someone with good taste.',
  {'entities': [(94, 111, 'PRODUCT')]}),
 ("Jack Leather and Wood Stool Tan This item is non-returnable. Excluded from shipping promotions. The Jack leather and wood stool is the perfect compliment to a beautiful kitchen island or bar. It is equally suited to a relaxed modern Los Angeles home as it is to a New York City apartment. The iron footrest is an added comfort for resting your heels. Thoughtful detailing enhances the simple design of this Lostine collection. Each thick leather seat has its own character and variations lending to the appeal of the material. The leather will vary in color and darken with age, like a great bag or a favorite pair of boots. The frame is made of a hard maple wood and sealed to protect the wood but keep the beauty of the wood's natural color. Made in the USA. Also available in black leather and as the Jack Chair. LEAD TIME + Made to order. Please allow 12-14 weeks for delivery. About our Leather Our Harness leather ages over time, in what we call a living finish. Scratches, dents, folds and marks occur naturally in the leather. We do not view these marks as defects. Please note that hue and color varies between hides. Since each side of leather is different, the shade and texture of the leather will vary with each chair. Please care for your Jack chairs with a soft cloth and leather cleaner. Please be advised that exposure to sunlight and water will darken the leather. Finish samples available upon request.",
  {'entities': [(0, 27, 'PRODUCT'),
    (100, 127, 'PRODUCT'),
    (804, 815, 'PRODUCT')]}),
 ("Need some incentive to start on that pile of paperwork Look no further than our Big Cay Desk. With plenty of surface space on which to spread out, ample storage and great looks, you'll be anxious to get started. Did we mention that we finish the back so you can float the Big Cay anywhere in the room! Bottom drawers equipped with hanging hardware for files. Note that this desk is also available in a smaller size we refer to as our Little Cay Desk if floor space is at a premium.",
  {'entities': [(80, 93, 'PRODUCT'),
    (272, 279, 'PRODUCT'),
    (434, 449, 'PRODUCT')]}),
 ('$1,149.00 $849.00 Features All-weather material, meticulously handwoven and handcrafted Slim low-profile cushion for an ultra modern look Constructed from a durable lightweight aluminum frame Built-in wheels for easy mobility.',
  {'entities': [(105, 112, 'PRODUCT')]}),
 ('Regular List: $0 Westlake is modern in design with American Shaker influence. This transitional bedroom collection is made out of sustainable Solid Mahogany and is designed especially for those with more compact bedrooms or simply those who have the need for more storage space. Case pieces feature felt lined top drawers, with English dovetailed solid albasia drawer boxeswith full extension 3 tier heavy-duty ball bearing drawer glides. Finished in a glowing cherry brown or dark mahogany with dark gun-metal color hardware, this clean line solid Mahogany bedroom group is a great value for any home. -Vendor: A-America -Collection: Westlake -Finish: Cherry Brown -Hardware: Dark Antique Bronze',
  {'entities': [(328, 367, 'PRODUCT')]}),
 ('Jordan Living Room, Black Get the alluring look of leather at a fabulous price! The Jordan living room will instantly update your space to a room that will make your friends jealous with envy. Upholstered in sleek and durable faux leather, this group features pillow armrests, contrast white stitching, box cushions, tapered square feet and overstuffed, attached pillow backs that will never be out of place. Add the matching oversized club chair to complete the room. Available in Gray, Red, Chocolate or Black upholstery. The sofa is also available as a queen sleeper sofa. Treat yourself at an amazing price!',
  {'entities': [(0, 18, 'PRODUCT'), (84, 102, 'PRODUCT')]}),
 ('Winndom Winndom Berkeley III Regular price $1,999.00 Sale price $0.00 Berkeley III mattress is a 1 sided mattress and made EXCLUSIVELY for Contempo Concepts. Available with just a Mattress or with a Mattress and Foundation Cover: Organic Cotton panel with Chalk border Quilt: 2 Organic latex foam rubber Build up: 6 organic foam rubber, 1 ContempoPedic Proprietary foam Warranty: 15 year',
  {'entities': [(8, 28, 'PRODUCT'), (70, 91, 'PRODUCT')]}),
 ('Muse Please check delivery times below. Substantial rounded proportions with a focus on comfort Muse is inspired by vintage designs from the 1970s. With his curvy modular form, Muse is a dynamic show stopper that will be the dramatic hero of any room. Muse is available as separate modules to enable you to customise to your needs.',
  {'entities': [(0, 4, 'PRODUCT'),
    (96, 100, 'PRODUCT'),
    (177, 181, 'PRODUCT'),
    (252, 256, 'PRODUCT')]}),
 ('Get organized quickly with this contemporary four-shelf bookcase made of black finished metal and solid pine wood. The four spacious shelves have a walnut finish for an industrial appearance. ',
  {'entities': [(45, 64, 'PRODUCT')]}),
 ('Contemporary Fabric Adjustable Barstool in 3 Color Options $179.00 This designer chair will make an attractive statement in the home. This stool stands out with stylish line stitching throughout the upholstery. The height adjustable swivel seat adjusts from counter to bar height with the handle located below the seat. The chrome footrest supports your feet while also providing a contemporary chic design.',
  {'entities': [(0, 39, 'PRODUCT')]}),
 ('usm haller side table O118 This Side Table O118 is part of the USM modular furniture collection created to combine elegance with architectural functionality. This system features a storage system with chrome-plated brass ball joints attached to steel tubes and metal panels. This rectangular storage media stand features one drop down door. Available in 6 different colors',
  {'entities': [(0, 26, 'PRODUCT'), (32, 47, 'PRODUCT')]}),
 ('Shopping for someone else but not sure what to give them Give them the gift of choice with a American Backyard gift card. Gift cards are delivered by email and contain instructions to redeem them at checkout. Our gift cards have no additional processing fees.',
  {'entities': [(93, 121, 'PRODUCT')]}),
 ("Craft Associates Wool Lounge Chairs 1519 Canadian Velvet Armchairs 1519 These Modern Armchairs 1519 These Modern Armchairs 1519 The Canadian Craft Associates Furniture are expertly crafted and upholstered. Each chair boasts hand cut foam and high grade commercial fabric. The chair's frame and base are constructed by hand from premium walnut and not machine. The legs are then hand turned by artisans and finished with a hand applied oil. Craft Associates Furniture is proud each piece is made in the US. We strive to craft a product that will stand the test of time. We use the best materials and have the best craftsman and upholstery in the business. We stand 100% behind our product.",
  {'entities': [(0, 40, 'PRODUCT'),
    (41, 71, 'PRODUCT'),
    (78, 99, 'PRODUCT'),
    (106, 127, 'PRODUCT')]}),
 ('Gift Card Shopping for someone else but not sure what to give them Give them the gift of choice with a Huset gift card. Details Gift cards are delivered by email and contain instructions to redeem them at checkout. Our gift cards have no additional processing fees.',
  {'entities': [(103, 119, 'PRODUCT')]}),
 ('Hodges Tweed 100% wool. milled in the UK. width 54 inches horizontal repeat 2 3/4 inches. vertical repeat 2 3/4 inches #YRK4507-01 memo samples available please contact us directly trade pricing available please contact us directly',
  {'entities': [(0, 12, 'PRODUCT')]}),
 ('Color Eave Desk Weï¿½re turning the tables on the desk. Work time or meal time, Eave is a dual-purpose ï¿½table-deskï¿½ ï¿½ making it the consummate multitasker. Spaciously appointed Generously sized desk at 55ï¿½ï¿½. Sturdy and handsome Side panels provide stability and enhance form. Frictionless experience Easily assembled and disassembled. The brainchild of Jonathan Sabine and Jessica Nakanishi, MSDS Studio is a Toronto-based design studio known for distinctive yet minimal products that synthesize the duoï¿½s Scandinavian and Japanese sensibilities. ï¿½Flip[s] the archetype... clean lines, soft curves, and striking details.ï¿½ ï¿½You owe it to yourself.ï¿½ï¿½ One of the ï¿½best home office desks to make your workspace actually workable.',
  {'entities': [(0, 15, 'PRODUCT'), (78, 82, 'PRODUCT')]}),
 ('Description Build your own collection with the CH24 Soft, a gentle variation on Hans J. Wegnerï¿½s Wishbone Chair from 1950. Just like the original, the special edition is timelessly elegant with its clean and characteristic expression. The CH24 Soft has an irresistible matte finish, available in five classic colours: black, grey, white, green and red. The new finish has an inviting quality that tempts the gaze to follow along the backrest, and your hand to do the same. Furthermore, the coating makes it easier to maintain the chair, keeping the timeless design looking even cleaner. Think of CH24 Soft as a gentle counterpart to a Wishbone Chair with a brighter finish. Or maybe as a subtle contrast to Hans J. Wegnerï¿½s CH327 dining table in soap-treated beech ',
  {'entities': [(47, 56, 'PRODUCT'),
    (97, 111, 'PRODUCT'),
    (239, 248, 'PRODUCT'),
    (596, 600, 'PRODUCT'),
    (635, 649, 'PRODUCT'),
    (724, 742, 'PRODUCT')]}),
 ('Gray Floating TV Stand Entertainment Center Item Number: 711849-711149-711149 Search no further, this is the cream of the crop. The ECO GEO is a unique three piece console meticulously handcrafted from REAL wood. The Solid Poplar framing and Genuine Maple Veneers over Engineered Plywood adorns the high-end nature of this entertainment center. It embellishes any space with a sleek, modern and clean design. The one-of-a-kind natural wood grain enhances the beauty and singularity of each and every console. The expertly crafted butcher block pattern solid wood roll-up doors are made with various hand selected repurposed wood slats. These doors take more time to make, but prevent filling landfills by using eco-friendly recycled end cuts. The tambour doors conveniently slide up and down for quick access to the hidden storage spaces. Once closed, the tambour doors conceal their contents displaying the beautiful wood doors. The varying nature of the character, grain, and tone are perfect for tying in other wood tones in your room. This entertainment center includes all of the necessary easy-to-use wall mounting hardware that features built in bubble levels for mounting this console level on your wall. The included mounting screws are designed for drywall with wood studs. If mounting this to block walls or metal studs, you would need to purchase the appropriate screws or anchors. ',
  {'entities': [(132, 139, 'PRODUCT')]}),
 ('Our Notch Desk is a sleek nod to mid-century design, with faceted tapered legs and elegant details. Perfect for working from home, reading, or catching up on your hand-written notes. Dimensions: 60w x 29h x 24d (Custom sizes available) Material: White Oak Finish: Polymerized Oil Production time: 1-2 weeks Crafted in Denver, Colorado This item is eligible for free contactless delivery in the Denver Metro area!',
  {'entities': [(4, 14, 'PRODUCT')]}),
 ('Little Partners Grow with Me Chair is the perfect supportive seating solution for your little one. We have thoughtfully designed this chair to provide all the correct basic seating principles. Not only beautiful in its design, the Grow with Me Chairprovides a high back, adjustable height, durable hardwood construction, and TWO CHAIRS for the price of one. And the best part No dangling feet! Through extensive research, we have learned that having your childï¿½s feet positioned firmly on the ground while sitting not only helps them to focus more clearly but also gives them a greater sense of confidence and independence. PRODUCT INFORMATION The Grow with Me Chair by Little Partners includes TWO adjustable toddler chairs. Hassle-free, easy assembly! Our EZ Assemblydesign takes less than 15 minutes to assemble with provided hardware. Perfect for busy parents; especially those who dislike spending a lot of time on assembly. Lightweight design, and convenient built-in carry handle makes moving chair from one place to another easy. 3-Level height adjustment helps children sit comfortably with both feet positioned on the ground. Comfortable high seat back provides added comfort and security. Supports up to 50lbs. DETAILS THAT MATTER GREENGUARD GOLD CERTIFIED*: This product has undergone rigorous scientific testing for over 10,000 chemical emissions and VOCs. It contributes to cleaner indoor air, creating a healthier environment for your baby to sleep, play, and grow. Constructed from the highest-quality layered Birch and Poplar that is EPA TSCA Title VI and FSC (Forest Stewardship Counsel) compliant. Finished in child-safe, non-toxic paints and materials. Safety Tested to ASTM F963-17 and 16 CFY Part 1307 for lead, heavy metals, and phthalates. Our products come with a LIFETIME WARRANTY*, guaranteeing long-lasting durability. Includes two chairs. Fits well with standard toddler play tables. No tools needed for height adjustment.',
  {'entities': [(0, 34, 'PRODUCT'),
    (231, 257, 'PRODUCT'),
    (648, 666, 'PRODUCT')]}),
 ('MOSS Sofa 205 1,790.00 ï¿½ï¿½ (including VAT) Delivery time: 8-10 weeks. FREE shipping in EU! Elena is a polyester fabric which has a high abrasion resistance of 29 000 Martindale. The multicoloured fabric is soft and nice to touch. The designer selected grey colours are perfect for scandinavian style homes. We suggest to use only special upholstery cleaning products to remove stains and recommend weekly regular cleaning with a vacuum cleaner soft nozzle or dry cleaning for general dirtiness. Lido is an upholstery fabric with excellent abrasion parameters ï¿½ 106 700 Martindale cycles, which is why it is a great choice for both household and public utility interiors. It is very popular fabric, partly because of the wide range of colors in the collection and mainly because of its quality. The fabric is covered with a special finish, which protects it against liquid stains. We suggest to use only special products to remove stains or dry cleaning for general dirtiness. Stella is a polyester fabric which has a very high abrasion resistance ï¿½ 180 500 Martindale. The fabric is soft and nice to touch, yet with a dense fibre which makes it suit well for homes with pets and small children. Stella is also very easily cleanable and can be washed at 60 degrees. With our stain tests we have conquered both coffee and red wine stains. Stella is a fabric with high bleach resistance and therefore suits well also to bright sunny rooms. Megan is a high-quality 100% polyester velvet fabric with a high resistance to abrasion ï¿½ 170 000 Martindale cycles. The fabric is soft, yet with a dense fibre which makes it suit well for homes with also pets. It is highly recommended for modern interiors. The fabric is also water repellent. We suggest to use only special upholstery cleaning products to remove stains and recommend weekly regular cleaning with a vacuum cleaner soft nozzle or dry cleaning for general dirtiness. Wooly is a stylish Italian refined wool that brings out furnitureï¿½s beautiful form. The fabric consists of 70% wool. Its abrasion resistance is high ï¿½ 150 000 Martinadale. Wool is the most easily cleanable natural fibre fabric. Nonetheless we suggest to use only special cleaning products to remove stains or dry cleaning for general dirtiness. With our stain tests we have conquered both coffee and red wine stains. Wooly is a fabric with high bleach resistance and therefore suits well also to bright sunny rooms.',
  {'entities': [(0, 9, 'PRODUCT')]}),
 ('The Bayswater Armchair The Bayswater Armchair design has been gracing drawing rooms for nearly 150 years. This elegantly proportioned design classic was made by Howard and Sons and named the ï¿½Bridgewaterï¿½. Built and finished just as the model was in the 19th Century with hand-tied coil springs and hand-sewn down-filled sac-a-duvet cushions. Compact yet supremely comfortable, the Bayswater is perfectly at home gracing a drawing room yet equally at home nestling in the corner of a cottage bedroom.',
  {'entities': [(4, 22, 'PRODUCT'),
    (27, 45, 'PRODUCT'),
    (382, 391, 'PRODUCT')]}),
 ("Ottoman LeoMaison Sarah Lavoine Ottoman Leo: Designed by Sarah Lavoine for her recent project the Roch Hotel & Spa in Paris, this ottoman will find the perfect spot in your lounge or bedroom. Its round shapes, luxurious velvet material and contrasting colours will bring character and cosiness to your space. French interior designer Sarah Lavoine embodies the Parisian spirit: smart, chic and timeless. Her exquisite range of furniture, soft furnishing, tableware and lighting is classic and elegant with a vibrant colour palette dominated by Sarah's signature blue and luxurious textures like velvet, wood and brass. Every piece is crafted by the best artisans. A new French art de vivre, contemporary and sophisticated, but always welcoming.",
  {'entities': [(18, 39, 'PRODUCT')]}),
 ('A romantic love seat for two, with intricate heart shaped hand forged scrollwork on the seat back and arms and perforated decorative seat. Perfect for sharing a conversation with a friend or alongside an outdoor table, the 1900 bench is a delightful pretty addition to your outdoor space. Inspired by the famous romantic gardens in France, and made using hand-forged traditional techniques, this bench is both elegant and ultra-comfortable.',
  {'entities': [(223, 233, 'PRODUCT')]}),
 ('KOBO ST-DESK/TABLE The KOBO ST-DESK and TABLE series is a multi-purpose desk or dining table and ideal for work or studying. The laminated plywood desktop is finished with linoleum made from all-natural ingredients, creating a soft texture that is pleasant to touch. The desk is supported by L-shaped steel legs that are both slender and sturdy. The sleek look of the desk is synonymous with contemporary living, drawing out the innate creativity of its users. *Optional drawer upon request. BRAND : Ishinomaki Laboratory DESIGN BY : Keiji Ashizawa',
  {'entities': [(0, 18, 'PRODUCT'), (23, 45, 'PRODUCT')]}),
 ('All our Framed Prints are custom made in Australia to suit your specifications, and as such, have a production timeframe of 7 business days. Giclee printed using archival inks and paper which means no fading! Available in multiple sizes, in either black, white or oak finish. Made of quality timber molding and mounted behind museum grade Plexiglas. Comes ready to hang.',
  {'entities': [(8, 21, 'PRODUCT')]}),
 ('This product is Made to Order just for you and will take approximately 2 weeks to complete. Transform your deck or patio into an outdoor living space with the addition of this Baywood sofa. Incorporate it with chairs, and a coffee table for a complete unified look Rust resistant UV/Moisture Resistant Cushions Durable construction',
  {'entities': [(176, 189, 'PRODUCT')]}),
 ('Large white top works perfect for makeup and study. Three drawers without pulls to give a urban look Solid wood legs Large mirror gives fresh look to closed spaces Made with the highest environmental standards Weights & Dimensions ',
  {'entities': [(0, 15, 'PRODUCT')]}),
 ('American Oak Side Table $749.00', {'entities': [(0, 23, 'PRODUCT')]}),
 ('Anderson Chair', {'entities': [(0, 14, 'PRODUCT')]}),
 ('Sale! Angular Mango Wood Buffet (FLOOR STOCK CLEARANCE) $1,599.00 Out of stock Description Angular Mango Wood Buffet (FLOOR STOCK CLEARANCE)',
  {'entities': [(6, 31, 'PRODUCT'), (91, 116, 'PRODUCT')]}),
 ('Sale! Antique Wood Lantern $179.00 Out of stock Description Antique Wood Lantern',
  {'entities': [(6, 26, 'PRODUCT'), (60, 80, 'PRODUCT')]}),
 ('Sale! Antoine de Praiteau Wall Clock 60 CM $189.00 ',
  {'entities': [(6, 36, 'PRODUCT')]}),
 ('Sale! Anton Throw with Pom Poms ï¿½ Teal $229.00 Additional information |Colour| As Shown',
  {'entities': [(6, 17, 'PRODUCT')]}),
 ('Aqua and White Cotton Throw $39.00', {'entities': [(0, 27, 'PRODUCT')]}),
 ('Sale! Aqua Indian Style Cabinet 3 Drawers and 2 Doors SOLD we can source a similar piece. $1,499.00.',
  {'entities': [(6, 31, 'PRODUCT')]}),
 ('Sale! Aqua Storage/Blanket Box $369.00 Out of stock Additional information ',
  {'entities': [(6, 30, 'PRODUCT')]}),
 ('Arab Candlestick $139.00', {'entities': [(0, 16, 'PRODUCT')]}),
 ('Sale! Arctic Console / Dining Table $899.00 Out of stock Description Arctic Console / Dining Table',
  {'entities': [(6, 35, 'PRODUCT'), (69, 98, 'PRODUCT')]}),
 ('Areca Palm Tree 1.8 M $339.00', {'entities': [(0, 15, 'PRODUCT')]}),
 ('MST-004 Grand Opening Banner Product Description: Grand Opening Banner >19H x 50W >Glossy Paper Material >Indoor Use Only Sale Grand Opening Banner',
  {'entities': [(0, 28, 'PRODUCT'),
    (50, 70, 'PRODUCT'),
    (127, 147, 'PRODUCT')]}),
 ('MST-003 Clearance SALE Banner Sign Product Description: Clearance Sale Banner >19H x 50W >Glossy Paper Material >Indoor Use Only Sale Clearance Sale Banner',
  {'entities': [(0, 29, 'PRODUCT'),
    (56, 77, 'PRODUCT'),
    (134, 155, 'PRODUCT')]}),
 ('MST-700 Up To 70% Off Banner Product Description: >Up to 70% Off Banner >19H x 50W >Glossy Paper Material >Indoor Use Only Sale >Up to 70% Off Banner',
  {'entities': [(0, 28, 'PRODUCT'),
    (51, 71, 'PRODUCT'),
    (129, 149, 'PRODUCT')]}),
 ('MST-500 Up To 50% Off Banner Product Description: >Up to 50% Off Banner >19H x 50W >Glossy Paper Material >Indoor Use Only Sale >Up to 50% Off Banner',
  {'entities': [(0, 28, 'PRODUCT'),
    (51, 71, 'PRODUCT'),
    (129, 149, 'PRODUCT')]}),
 ('MST-001 Blow Out Sale Banner Product Description: Blow Out Sale Banner >19H x 50W >Glossy Paper Material >Indoor Use Only Sale Blow Out Sale Banner',
  {'entities': [(0, 28, 'PRODUCT'),
    (50, 70, 'PRODUCT'),
    (127, 147, 'PRODUCT')]}),
 ('MST-002 Opening Soon Banner Product Description: >Opening Soon Banner >19H x 50W >Glossy Paper Material >Indoor Use Only Sale >Opening Soon Banner',
  {'entities': [(0, 27, 'PRODUCT'),
    (50, 69, 'PRODUCT'),
    (127, 146, 'PRODUCT')]}),
 ('MST-801-V Verticle SALE Banner Product Description: >Vertical Sale Banner >19W x 50H >Glossy Paper Material >Indoor Use Only Sale >Vertical Sale Banner',
  {'entities': [(0, 30, 'PRODUCT'),
    (53, 73, 'PRODUCT'),
    (131, 151, 'PRODUCT')]}),
 ('Warning Premises Monitored Product Description: >Warning Premises Monitored By CCTV 24 Hours Sign >Available in 2 Sizes 12W x 17H or 5W x 7H (smaller size comes with adhesive tape) >Plastic Material >Indoor Use Only',
  {'entities': [(0, 26, 'PRODUCT'), (49, 75, 'PRODUCT')]}),
 ('Though compact, our Expand Dove Cushioned Single Swing packs plenty of other craftsmanship perks as well a 40-inch polished oak spreader bar, hammock-style clew knots securing the spray of hanging ropes to zinc-plated hanging rings, two rows of brass-colored rolled-rim grommets and a double lockstitched border of the same solid-color fabric as the seatback. Durable Fabrics Not only is the craftsmanship of our swings the highest level of quality, but we also choose the highest quality outdoor fabrics as well. We use a combination of Sunbrella Fabric and our in-house brand DuraCord Fabric. Sunbrella branded fabric is the foremost fabric in the outdoor industry made from 100% solution-dyed acrylic. Solution-dyed acrylic, which means the fibers of the fabric are dyed in color before they are woven into the fabric. Sunbrella is not printed or dyed after the fabric is made. Sunbrella is water-resistant and will not fade. You can even soak it in bleach and the color will remain the same. Which is why to use Sunbrella fabric on the tops of our swings. DuraCord is not only fast-drying but is also resistant to rot, mold, mildew, and staining, as well as Sunbrella. We use the DuraCord Fabric on the bottom of our Fabric swings because has better abrasion resistance than Sunbrella. DuraCord Rope Our own proprietary solution-dyed synthetic blend of rope, DuraCord, blends the softness of cotton with the durability of polyester. Made in the Carolinas, this combination yields a rope that has excellent color steadfastness and superior feel. DuraCord is dyed when spun, producing a rope that is colored all the way through. Like a carrot when sliced open, DuraCord has the same color on both the inside and the outside. Oak Spreader Bar A wood that is naturally resistant to both rot and insect infestation, and has oils that repel water, Oak blends both exceptional strength and a handsome appearance. Our Oak spreader bars are repeatedly hand-dipped in honey-gold marine-grade varnish allowing a longer outdoor life and exceptional good looks. Our Oak spreader bars are a full 1 inch thick, compared to most other competitors.  A thicker spreader bar allows for more stability, allowing the Swing Cushion to sit firmly between the Ring knots. Our spreader bars also feature countersink drilled holes. Countersink drilling is a technique that uses a trumpet drill bit that produces a hole with no hard angles, allowing the rope to pass through without any sharp edges to wear the rope. Rolled Rim Grommets with Spur Washers We are very particular with every aspect of our swings even down to the grommets on all our fabric swings. Every grommet we use has tiny spurs on the back of the connecting washer forcing part of the fabric into the lip of the grommet itself, creating a more secure bond. We have also experimented with making the fabric around the grommet stronger by sewing an ï¿½Xï¿½ï¿½ on the fabric before placing the grommet, by adding that extra stitching it gives those tiny spurs more fabric to grip onto. The industry standard, however, is the telescoping grommet, which our testing has shown rips lose from the fabric before the rolled-rim grommet every time. We plate all our hanging hardware with Zinc Plated Coating, including Hanging bracket that comes standard with our single swings. The zinc-plated coating prevents the accumulation of rust and ensures a longer outdoor life. Our very own One-piece ring-knot and chain, a product of years of development, permanently attaches the chain to the O-ring. This produces a swing that is more durable and most importantly makes it impossible to lose the chain. Non-absorbent Padding Each of our Single Cushioned Swings are filled with a non-absorbent 1.5ï¿½ï¿½ Urethane Foam that is sewn in between the two layers of fabric. The foam adds a pillow-like softness to the already comfortable fabric seat. The Foam Center is a nonabsorbent material allowing water to drain through and provide quick drying.',
  {'entities': [(20, 54, 'PRODUCT'), (3665, 3688, 'PRODUCT')]}),
 ("Picture yourself on a beach, listening to sound of ocean waves, seagulls chirping, and the feeling of a cool ocean breeze in your face, wasting the day away in pure relaxation bliss. The Deluxe Cushion Swing in Midori Indigo is the first step to creating this relaxation experience in the comfort of your own backyard! The top fabric layer of our Midori Indigo Deluxe Cushion Double Swing is made from solution-dyed synthetic Sunbrella fabric and a neutral-colored bottom layer made from our quick-drying Duracord fabric. These two synthetics sandwich a lavish center of resilient, nonabsorbent polyester hollow-fill fiber manufactured from recycled plastic drink bottles. For our Hatteras Hammock wood frame, we use a toasted almond-brown Brazilian Teak Cumaru that's been sanded to a soft finish. Cumaru is one of the strongest wooden materials around, and looks great as well! Our hardware is zinc plated steel, for quality rust protection. Even though Hatteras Hammocks are extremely strong and weather-resistant, we do always recommend storing it in an enclosed dry place away from any overly-harsh weather, hungry wildlife and other lingering periods when it's going unused, to ensure its longest possible life. Durable Fabrics Not only is the craftsmanship of our Deluxe Double Swings the highest level of quality, but we also choose the highest quality outdoor fabrics as well. We use a combination of Sunbrella Fabric and our in-house brand DuraCord Fabric. Sunbrella branded fabric is the foremost fabric in the outdoor industry made from 100% solution dyed acrylic. Solution-dyed acrylic, which means the fibers of the fabric are dyed in color before they are woven into the fabric. Sunbrella is not printed or dyed after the fabric is made. Sunbrella is water resistant and will not fade. You can even soak it in bleach and the color will remain the same. Which is why to use Sunbrella fabric on the tops of our deluxe double swings. DuraCord is not only fast-drying but is also resistant to rot, mold, mildew, and staining, as well as Sunbrella. We use the DuraCord Fabric on the bottom of our double swings because DuraCord has better abrasion resistance than Sunbrella. Cumaru Wood Frame A Brazilian hardwood similar to teak, Cumaru, is 5 times harder and denser than pine, cedar, and redwood. Our Cumaru hardwood has a history of outstanding outdoor performance. Even after years of use, our Cumaru hardwood ages naturally to a beautiful silver grey without losing any of its durability. Exclusive to Hatteras Hammocks and is by far the best natural wood used in a porch swing on the market. Eco-Friendly Made from recycled soda bottles, our polyester fiberfill is stuffed between two layers of durable fabric. The recycled fiberfill is non-absorbent, allowing water to drain easily and provide quick dying. Our goal is to repurpose recyclable materials as much as possible and leave a smaller carbon footprint. We plate all our hanging hardware with Zinc Plated Coating, including both hanging hooks that come standard with our deluxe double swings. The zinc plated coating prevents the accumulation of rust and ensures a longer outdoor life.",
  {'entities': [(187, 207, 'PRODUCT'),
    (361, 388, 'PRODUCT'),
    (1271, 1291, 'PRODUCT'),
    (2517, 2534, 'PRODUCT')]}),
 ('Though compact, our Cast Laurel Cushioned Single Swing packs plenty of other craftsmanship perks as well a 40-inch polished oak spreader bar, hammock-style clew knots securing the spray of hanging ropes to zinc-plated hanging rings, two rows of brass-colored rolled-rim grommets and a double lockstitched border of the same solid-color fabric as the seatback. Durable Fabrics Not only is the craftsmanship of our swings the highest level of quality, but we also choose the highest quality outdoor fabrics as well. We use a combination of Sunbrella Fabric and our in-house brand DuraCord Fabric. Sunbrella branded fabric is the foremost fabric in the outdoor industry made from 100% solution-dyed acrylic. Solution-dyed acrylic, which means the fibers of the fabric are dyed in color before they are woven into the fabric. Sunbrella is not printed or dyed after the fabric is made. Sunbrella is water-resistant and will not fade. You can even soak it in bleach and the color will remain the same. Which is why to use Sunbrella fabric on the tops of our swings. DuraCord is not only fast-drying but is also resistant to rot, mold, mildew, and staining, as well as Sunbrella. We use the DuraCord Fabric on the bottom of our Fabric swings because has better abrasion resistance than Sunbrella. DuraCord Rope Our own proprietary solution-dyed synthetic blend of rope, DuraCord, blends the softness of cotton with the durability of polyester. Made in the Carolinas, this combination yields a rope that has excellent color steadfastness and superior feel. DuraCord is dyed when spun, producing a rope that is colored all the way through. Like a carrot when sliced open, DuraCord has the same color on both the inside and the outside. Oak Spreader Bar A wood that is naturally resistant to both rot and insect infestation, and has oils that repel water, Oak blends both exceptional strength and a handsome appearance. Our Oak spreader bars are repeatedly hand-dipped in honey-gold marine-grade varnish allowing a longer outdoor life and exceptional good looks. A thicker spreader bar allows for more stability, allowing the Swing Cushion to sit firmly between the Ring knots. Our spreader bars also feature countersink drilled holes. Countersink drilling is a technique that uses a trumpet drill bit that produces a hole with no hard angles, allowing the rope to pass through without any sharp edges to wear the rope. Rolled Rim Grommets with Spur Washers We are very particular with every aspect of our swings even down to the grommets on all our fabric swings. Every grommet we use has tiny spurs on the back of the connecting washer forcing part of the fabric into the lip of the grommet itself, creating a more secure bond. We have also experimented with making the fabric around the grommet stronger by sewing an ï¿½Xï¿½ï¿½ on the fabric before placing the grommet, by adding that extra stitching it gives those tiny spurs more fabric to grip onto. The industry standard, however, is the telescoping grommet, which our testing has shown rips lose from the fabric before the rolled-rim grommet every time. Hardware We plate all our hanging hardware with Zinc Plated Coating, including Hanging bracket that comes standard with our single swings. The zinc-plated coating prevents the accumulation of rust and ensures a longer outdoor life. Our very own One-piece ring-knot and chain, a product of years of development, permanently attaches the chain to the O-ring. This produces a swing that is more durable and most importantly makes it impossible to lose the chain. Non-absorbent Padding Each of our Single Cushioned Swings are filled with a non-absorbent 1.5ï¿½ï¿½ Urethane Foam that is sewn in between the two layers of fabric. The foam adds a pillow-like softness to the already comfortable fabric seat. The Foam Center is a nonabsorbent material allowing water to drain through and provide quick drying.',
  {'entities': [(20, 54, 'PRODUCT'), (3590, 3613, 'PRODUCT')]}),
 ('3 Piece UV Outdoor Sylvia Setting The Sylvia setting is a great small compact setting for balconies and beside the pool. Our all weather wicker furniture is woven from a synthetic material that is ideal for Australian weather. A UV inhibitor is baked into the wicker fibre during the manufacturing process and we offer a 3 year warranty on the wicker against peeling, cracking and fading. The frame is aluminium so it will not rust or corrode. The UV fabric on the cushions is fade resistant, water resistant, chlorine resistant and specifically designed for outdoor use. The fabric comes with a 2 year warranty for colour fastness. Available in White & Black.',
  {'entities': [(8, 33, 'PRODUCT'), (38, 52, 'PRODUCT')]}),
 ('Adnaul Armchair $925.00 The comfy Adnaul armchair is made from brown natural wicker with a beige fabric seat and back cushion.',
  {'entities': [(0, 15, 'PRODUCT'), (34, 49, 'PRODUCT')]}),
 ('Akit Armchair $695.00 The gorgeous Akit armchair is made from tan rattan with timber legs.',
  {'entities': [(0, 13, 'PRODUCT'), (35, 48, 'PRODUCT')]}),
 ('Aminy Dining Chair $385.00 The Aminy Dining Chair is versatile and modern in appearance. Made from quilted PU fabric with powder coated metal legs and available in Taupe, Anthracite and Green. 60cm Wide X 56cm Deep X 87cm High',
  {'entities': [(0, 18, 'PRODUCT'), (31, 49, 'PRODUCT')]}),
 ('Andi Bar Stool $265.00 The Andi Bar stool is made with an upholstered seat with powder coated metal legs. 53cm Wide X 49cm Deep X 88cm High (seat height 65cm)',
  {'entities': [(0, 14, 'PRODUCT'), (27, 41, 'PRODUCT')]}),
 ('Andi Dining Chair $245.00 The Andi Dining Chair is made with an upholstered seat and powder coated metal legs. 46cm Wide X 60cm Deep X 83cm High',
  {'entities': [(0, 17, 'PRODUCT'), (30, 47, 'PRODUCT')]}),
 ('Arya Glass Top Dining Table w/ Black Legs $795.00 ï¿½ $895.00 The Arya Glass Top Dining Table w/ Black legs is available in 3 sizes: 150cm Long X 90cm Wide X 75cm High $795 180cm Long X 100cm Wide X 75cm High $845 200cm Long X 100cm Wide X 75cm HIgh $895 ',
  {'entities': [(0, 27, 'PRODUCT'), (64, 91, 'PRODUCT')]}),
 ('Arya Glass Top Dining Table w/ Timber Veneer look legs $995.00 ï¿½ $1,100.00 The Arya Glass Top Dining Table w/ powder coated Timber Veneer look legs is available in 3 sizes. 150cm Long X 90cm Wide X 75cm High $995 180cm Long X 100cm Wide X 75cm High $1050 200cm Long X 100cm Wide X 75cm High $1100 Only available on full priced items. Not available on sale items',
  {'entities': [(0, 27, 'PRODUCT'), (79, 106, 'PRODUCT')]}),
 ('Arya Glass Top Dining Table w/ White legs $795.00 ï¿½ $895.00 The Arya Glass Top Dining Table w/ White legs is available in 3 sizes: 150cm Long X 90cm Deep X 75cm High ï¿½ $795 180cm Long X 100cm Deep X 75cm High ï¿½ $845 180cm Long X 100cm Deep X 75cm High $895 Additional information |Dimensions||N/A| |Available sizes| Stainless Steel 180 x 100cm, Stainless Steel 200 x 100cm, Black or white Epoxy 180 x 100cm, Black or white Epoxy 200 x 100cm',
  {'entities': [(0, 27, 'PRODUCT'), (64, 91, 'PRODUCT')]}),
 ('Azzar Bar stool $325.00 These superb bar stools are made with 100% leather with a brushed stainless steel base. Available in Chocolate, Black & white. Height to seat cushion 66cm ï¿½ perfect for kitchen bench height.',
  {'entities': [(0, 15, 'PRODUCT')]}),
 ('Azzar Bookcase $1,795.00 Made from sustainable mango timber with a rustic steel framework. 150CM L X 40CM D 210CM H',
  {'entities': [(0, 14, 'PRODUCT')]}),
 ('Azzar Coffee Table $795.00 The Azzar Coffee Table is made from sustainable mango timber with steel legs and is available in various furniture stains. Complimentary pieces are also available within the Azzar Range. 135cm Wide X 75cm Deep X 40cm High',
  {'entities': [(0, 18, 'PRODUCT'), (31, 49, 'PRODUCT')]}),
 ('Measurements: Outside: 68ï¿½W 94ï¿½D 62.5ï¿½H Footboard Height: 18ï¿½H Tufted headboard bed in the Michael Weiss Collection. #9 Nail head trim only standard on headboard, footboard, rails, and wings. Finish shown on our showroom is Amarone. SKU: W521Q-HF Contact store for more details and pricing. More wood finish and fabric options are available. King and California King sizes are also available.',
  {'entities': [(63, 83, 'PRODUCT')]}),
 ('Choose Select Primary Color Fabric Robert Allen: +$ Adriatic Bed Item# MKS-BED007 3,500.00 $3,500.00 Its bold wings and high back give this upholstered bed a modern appeal. The seamless silhouette looks striking in this statement fabric and classic in its luxurious texture. This simple yet dynamic bed is an irresistible find as it is difficult to come across a piece as luxurious and thoughtfully designed as this. The ultra-high back becomes the focal point of the room, easily filling up large bedroom walls yet not overpowering a space or leaving little room for much else. With a bed like this as your starting point you can create an oasis from a variety of different design custom sizes available. ',
  {'entities': [(52, 64, 'PRODUCT')]}),
 ("Choose Select Primary Color Fabric Basha: +$0 Elmosoft +$0 One of the great things about a bed such as this is its timelessness. You could throw this into a time machine and transport it hundreds of years into the future and it would still look like it belonged. O.K. Maybe not hundreds of years. But the point is, this design isn't chasing the latest ravings of the fashionistas. with a sense of style it is confident enough to stand on its own. KING SIZE : L 90 X W87 X H91 KING PRICE:$3400 Item# KMP-S003 Construction: Solid, hard wood, Birch frame that is reinforced for durability and extra stability with interlocking joists. High density-resiliency polyurethane foam is used for the headboard and footboard padding. Wooden legs. Upholstered and precisely stitched in any fabric of your choice. Size: QUEEN :L88 x W70 x H91",
  {'entities': [(91, 94, 'PRODUCT')]}),
 ('Choose Select Primary Color Fabric Robert Allen: +$ Fabric +$ Castiglione high headboard bed Item# MKS-BED-010 7,495.00 $6,745.50 The Upholstered Castiglione high headboard bed delivers physical comfort as well as aesthetic pleasure. Standing at 72 high the padded headboard creates a visual harmony and sets the tone of the room. Like a picture on the wall, a thin 2.5 wood frame runs along the sides as well as the lower bed edges and is finished in chocolate espresso finish.The lower part of the bed is fully upholstered and includes a full base for the mattress to rest on. Manufactured in the USA; the Castiglione high headboard bed can be fully customized in height and materials. The construction of the bed is made of reinforced solid wood padded in foam front and back for a lifetime of use. Due to the size and weight of the bed a white glove delivery is recommended. MADE IN THE USA Item# MKS-BED-010 Construction: Solid hardwood frame High density/resiliency foam Customizable to the size of your choice Several materials available to choose from Custom orders allow 3-4 weeks manufacturing time Size: QUEEN :W 76 x L 80 X H 72 inches',
  {'entities': [(62, 92, 'PRODUCT'),
    (146, 176, 'PRODUCT'),
    (608, 638, 'PRODUCT')]}),
 ('Choose Select Primary Color Fabric Basha: +$ Tweed +$ Grand Luxe +$ Forma Bed Item# SLX--3018 2,420.00 $2,420.00 This simple , modern lines beautifully designed Forma bed features a very balanced proportions. Made with soft upholstery overlapping a foam-padded frame with hardwood, this bed sleeps like a dream. The full-sized platform requires only a mattress and comes with wooden slats that serve the same function as a box spring it is a spectacular piece of modern bedroom furniture. KING SIZE : L91 X W80 X H40x bed frame 14 high Item# SLX--3018 Construction: Size: QUEEN: $2320 / L 91 X W64 X H40',
  {'entities': [(68, 77, 'PRODUCT')]}),
 ('Choose Select Primary Color Fabric Basha: +$ Linen +$ Tweed +$ Grand Luxe +$ Geronimo Queen Bed Item# JWSXL-16057 1,895.00 $1,895.00 This modern, luxurious and beautifully designed Geronimo bed features a high headboard with an entirely removable cover. Made with soft white upholstery overlapping a foam-padded frame with hardwood, this bed sleeps like a dream. The full-sized platform requires only a mattress and comes with wooden slats that serve the same function as a box spring. Bright chromed nickel or bronzed metal legs finish off this spectacular piece of modern bedroom furniture. The Geronimo bed is also available solid wood, finished in brushed light or black oak, gray or brown oak. ',
  {'entities': [(77, 95, 'PRODUCT'),
    (181, 193, 'PRODUCT'),
    (597, 609, 'PRODUCT')]}),
 ('Choose Select Primary Color Fabric Basha: +$0 Elmosoft +$0 Novo +$0 Choose Your Beds Size... When customers ask for a bed recommendation with real pizzazz, I point them to the Grand Bed. Handmade with high-quality solid reinforced hardwood Birch frame with interlocking joists, it features a tall rectangular headboard, which gently wraps around, to create just the right amount of privacy between you and your peaceful spot and the rest of the world. The thickly padded frame is crafted with high density-resiliency foam for additional durability and comfort, while its unique and sculptural feet are finished in beautiful brushed chromed steel. Made in U.S.A. A bed luxurious enough for royalty, the Grand Beds frame can be upholstered in your choice of custom fabrics, which are removable for dry-cleaning. Extra covers are available for purchase, and semi-aniline leather upholstery is also available. Comes with back cushions filled with a soft blend of down and feathers. ',
  {'entities': [(176, 186, 'PRODUCT'), (702, 712, 'PRODUCT')]}),
 ('Choose Select Primary Color Fabric Brentwood Boucle: +$ Hollywood Velvet +$ Madrid King Bed Item# MKS-Bed 3,285.00 $3,285.00 Simple in design and light in color, this modern fabric covered headboard with distinct paneling complements every bedding style and stunningly fills bedrooms with even the tallest of ceilings. This piece was built with lasting quality in mind; crafted from top of the line materials. The highlights and shadows of the professionally upholstered headboard give character to what would otherwise be a seamless piece. With this bed youll have a piece that complements your style, no matter how it evolves. The versatility of this classic yet distinctive bed promises endless possibilities for your own personal touch Item# MKS-Bed Construction: Size: 78 x 89x',
  {'entities': [(76, 91, 'PRODUCT'),
    (98, 105, 'PRODUCT'),
    (746, 753, 'PRODUCT')]}),
 ('Choose Select Primary Color Fabric Basha: +$ Grand Luxe +$ Mateo Bed Item# KMP-MATEO 1,995.00 $1,995.00 This timeless design is custom tailored to your style, requirements and aesthetics! Made out of the finest materials; is carefully combined and coordinated to enhance, the colors, textures and shapes of your bedroom. This is definitely a piece that engages an elegantcontemporary look; taking into account the personality and aspects of your home. Made out of solid maple wood the backboard is lightly padded and tufted; for a comfortable resting experience, The frame like aesthetics are meant to center attention around the bed The Mateo bed can be fully customized in a variety of sizes, fabrics and leather. ',
  {'entities': [(59, 68, 'PRODUCT'), (638, 647, 'PRODUCT')]}),
 ('Choose Select Primary Color Fabric B: +$ Milano Bed Item# BM-BL-C2102 2,950.00 $2,950.00 You will love every detail on this stunning Bed. A straight headboard frames the wall with padded cushions . The right amount of tone and texture is styling the bed in contemporary colors of leather and fabric. Gloss, L-shaped, power-coated legs give the right height to the bed frame . Ample selection of leather-colors and fabrics can be chosen to be customized to your preference All the details in perfect harmony...A Chic cocoon perfect to sleep in! ',
  {'entities': [(41, 51, 'PRODUCT')]}),
 ("Choose Select Primary Color Fabric Linen: +$ Grand Luxe +$ Oxford Bed Item# JFXL-3083 1,950.00 $1,950.00 Tucks and buttons are a popular headboard feature today. This upholstered bed has a beautifully designed button pattern that evokes a contemporary style and resembles a quilted pattern. If you're looking for a tall upholstered headboard with a no-fuss design, this is it. Add it to your contemporary, modern or traditional house, and it will be at home. The wood frame can be disassembled for ease of use, plus the bed has an added feature. Hydraulics make the mattress pull up at the foot of the bed to reveal under bed storage. ",
  {'entities': [(59, 69, 'PRODUCT')]}),
 ('Choose Select Primary Color Fabric Basha: +$ Linen +$ Grand Luxe +$ Philippi Bed Item# SLX-3017 1,995.00 $1,995.00 The Philippi has a clean, geometric silhouette which makes it a dynamic addition to your bedroom....It is the kind of bed that manifests in most design settings with a powerful presence of elegance and inspiration ...The padded headboard is tall enough to provide a comfortable backrest while watching TV and yet discrete not to over-dominate your space... A sophisticated array of power coated steel bars create the beds base-support ; hence providing a strong foundation for a long lasting lifespan....Solid beech wood slats are reinforced on its steel frame to create a sturdy platform for your mattress...Removable sleepovers make the cleaning of the upholstery easier; while additional covers can be purchased separately...The Philippi can be ordered in an array of fabrics and colors ...Assembly is required . KING price: $2095.',
  {'entities': [(68, 80, 'PRODUCT'),
    (119, 127, 'PRODUCT'),
    (847, 855, 'PRODUCT')]}),
 ('Product Story Provide exquisite yet functional accenting for the room with this versatile Roosevelt Media Cabinet Sideboard. Featuring mahogany wood construction and designed with ample shelving space, cabinets, and dovetail drawers, this handsome sideboard is ideal for organizing the room and enhancing decor. This piece is offered with a vast array of finish and distress options to create a truly custom design.',
  {'entities': [(90, 124, 'PRODUCT')]}),
 ('Product Description PU grid-pattern swivel chrome bar stool Product Specifications: Sizes available: 42cm X 40cm X 95cm to 116cm Colours available: Black, beige, chocolate, white, red 0 Items  2018 Valentines Furniture. All rights reserved.',
  {'entities': [(20, 59, 'PRODUCT')]}),
 ('Product Description Vinyl swivel chrome gas lift bar stool Product Specifications: Sizes available: 46 X 41 X 85 to 106 Colours available: Black, white, chocolate 0 Items  2018 Valentines Furniture. All rights reserved.',
  {'entities': [(20, 58, 'PRODUCT')]}),
 ('Product Description PU swivel gas lift chrome bar stool Product Specifications: Sizes available: 40 X 40 X 82 to 104 Colours available: Black, beige, chocolate, white, red 0 Items  2018 Valentines Furniture. All rights reserved.',
  {'entities': [(20, 55, 'PRODUCT')]}),
 ('Product Description Colorado Power Sofa Sloped arms and sleek metal legs give the Colorado duo 3 Seater a Mid-century modern flair. But look closer, this timeless design has been given a modern makeover. Simply use the dual side-mounted 2-button control panels to recline and raise the leg rest, and, adjust the headrest. The button control also has a built-in USB port to charge your phones. Available in a choice of fabric and leathers. Brand : Lazboy',
  {'entities': [(20, 39, 'PRODUCT'), (82, 90, 'PRODUCT')]}),
 ('Product Description The Adele buffet is a stylish addition to any space. It is made from American Oak from sustainable timbers. 0 Items  2018 Valentines Furniture. All rights reserved.',
  {'entities': [(24, 36, 'PRODUCT')]}),
 ('Product Description This coffee table is beautifully crafted from American Oak. 0 Items  2018 Valentines Furniture. All rights reserved.',
  {'entities': [(25, 37, 'PRODUCT')]}),
 ('Product Description The Adele Lamp Table is a practical addition to any living space. Designed in Norway and made from American Oak. 0 Items  2018 Valentines Furniture. All rights reserved.',
  {'entities': [(24, 40, 'PRODUCT')]}),
 ('Product Description Beautifully crafted with a practical design the Adele TV lowline is a lovely addition to any living space. 0 Items  2018 Valentines Furniture. All rights reserved.',
  {'entities': [(68, 84, 'PRODUCT')]}),
 ('Product Description The Adele Dining Table comes in three sizes and is made from American Oak 0 Items  2018 Valentines Furniture. All rights reserved.',
  {'entities': [(24, 42, 'PRODUCT')]}),
 ('Product Description Beatifully crafted from American Oak the Adele dining chair is very comfortable and comes in either a natural linen or a black PU fabric. 0 Items  2018 Valentines Furniture. All rights reserved.',
  {'entities': [(61, 79, 'PRODUCT')]}),
 ("Voted: Best for work table with sleek, timeless design This Apex Study Table is perfect for any home or office use. It's a simple yet stylish design! This table is customisable with addon options like cable management flip cover. Table top size and colour can be customised too. Please enquire to find out more. *Listed price excludes the price of optional accessories like Cable Flip Cover and Cable Duct. OW Overall Width OD Overall Depth OH Overall Height SW Seat Width SD Seat Depth SH Seat Height AH Arm Height Thk -Thickness Dia Diameter IW Internal Width ID Internal Depth IH Internal Height LW Leg Width LD Leg Depth LH Leg Height WH Wood Plank Height FH Footboard Height For all our office chairs, Arm Height (AH) and Overall Chairï¿½s Height (OH) is taken when Seat Height (SH) is fixed at 450mm. Should an office chairï¿½s SH be higher than 450mm, then AH and OH will be taken at the lowest available SH of that chair. * All measurements given are in millimetres (mm).",
  {'entities': [(60, 76, 'PRODUCT')]}),
 ('A mesh mid back office chair that meets the basic requirements of an ergonomic chair. Comes with height adjustable lumbar support, back tilt and adjustable seat height. Available in 3 colours: Green, Grey, Black. Concerned about the wheels scratching your flooring Consider getting a PVC Floor Mat for added protection. Imagine It How It Can Accentuate Your Space Geek+ Geek+ Koufu Office Headquarters Koufu Office Headquarters Product Specifications Features Height adjustable seat Syncro-tilt Mechanism for an ergonomic experience Backrest tilts with lock/unlock mechanism Backrest tension can be tightened or loosened with a control knob located on the underside of the seat Height adjustable lumbar gives support to the lower back Tilt Mechanism: Knee-tilt mechanism provides a 1:1 tilt. Synchronised-tilt mechanism provides a 2:1 tilt. Synchronised-tilt provides a better ergonomic experience. It opens the angle of the torso for improved blood circulation to the legs and better spinal alignment. OW Overall Width OD Overall Depth OH Overall Height SW Seat Width SD Seat Depth SH Seat Height AH Arm Height Thk -Thickness Dia Diameter IW Internal Width ID Internal Depth IH Internal Height LW Leg Width LD Leg Depth LH Leg Height WH Wood Plank Height FH Footboard Height For all our office chairs, Arm Height (AH) and Overall Chairï¿½s Height (OH) is taken when Seat Height (SH) is fixed at 450mm. Should an office chairï¿½s SH be higher than 450mm, then AH and OH will be taken at the lowest available SH of that chair. * All measurements given are in millimetres (mm). ',
  {'entities': [(16, 28, 'PRODUCT'), (284, 297, 'PRODUCT')]}),
 ("Boost Tables are foldable and nestable, making it great for ad-hoc use. With an adjustable width, this table base fits a wide range of table top sizes. It also comes with cable storage area for tidiness and safety needs. Boost can be used as a Work From Home Table, Manager Table, Conference Table, Training Table as well as an ad-hoc Reception Table thatï¿½s often needed during events. Up Close & Personal Imagine It How It Can Accentuate Your Space Life Research Wellness Product Specifications Features Foldable & nestable table Comes with 2 telesopic beams that allow cables to run within leg structure; hiding unsightly cables and preventing tripping hazards Comes with removable covers for quick access to cables that's hidden within table leg 4 Lockable castors at Dia75mm OW Overall Width OD Overall Depth OH Overall Height SW Seat Width SD Seat Depth SH Seat Height AH Arm Height Thk -Thickness Dia Diameter IW Internal Width ID Internal Depth IH Internal Height LW Leg Width LD Leg Depth LH Leg Height WH Wood Plank Height FH Footboard Height For all our office chairs, Arm Height (AH) and Overall Chairï¿½s Height (OH) is taken when Seat Height (SH) is fixed at 450mm. Should an office chairï¿½s SH be higher than 450mm, then AH and OH will be taken at the lowest available SH of that chair. * All measurements given are in millimetres (mm). ",
  {'entities': [(0, 12, 'PRODUCT'), (221, 226, 'PRODUCT')]}),
 ("Our friendly team will help customers to load bulky furniture items into their cars. Boris Highback Office Chair Voted: Best value-for-money < $260 Boris is a high back office chair that comes with height adjustable lumbar support, adjustable backrest tension to fit different user's preference, and 3 levels of backrest tilt position-lock. We especially love the jacket hanger feature on this chair as it allows us to hang our sweaters or office jackets separately from the chair's backrest so our outer wear stays crease-free! It's no wonder Boris is a long-time favourite among Comfort Design's customers! Also avail in Midback version. Concerned about the wheels scratching your flooring Consider getting a PVC Floor Mat for added protection. Up Close & Personal Imagine It How It Can Accentuate Your Space Product Specifications 2D Headrest: Height adjustable, Multi-angle lock + Jacket Hanger 1D Armrests: Height-adjustable with PU Foam for greater comfort; No-Armrest version also available Breathable Mesh upholstery provides softness, comfort, good heat ventilation and support Height adjustable lumbar gives support to the lower back. Backrest tilts & locks in 3 positions Syncro-tilt Mechanism for an ergonomic experience Anti-Shock Mechanism backrest resumes original position upon sensing pressure Backrest tension can be tightened or loosened with a control knob located on the underside of the seat Nylon Base Tilt Mechanism: Knee-tilt mechanism provides a 1:1 tilt. Synchronised-tilt mechanism provides a 2:1 tilt. Synchronised-tilt provides a better ergonomic experience. It opens the angle of the torso for improved blood circulation to the legs and better spinal alignment. Anti-Shock Mechanism: After unlocking the backrest lock, the backrest will resume its original position only after sensing user's pressure. This ensure a safe and secure experience.   ",
  {'entities': [(85, 112, 'PRODUCT'),
    (148, 153, 'PRODUCT'),
    (544, 549, 'PRODUCT')]}),
 ('Our friendly team will help customers to load bulky furniture items into their cars. Cabinet Planter Box Cabinet Planter Box allows for housing of 3 plants; adding a touch of greenery and nature to the office environment. It can also be placed on top of filing cabinets or mobile caddies; to create privacy in an elusive way. Imagine It How It Can Accentuate Your Space Product Specifications W900 x D450 x H200 Holes Dia200 Internal Height190 OW Overall Width OD Overall Depth OH Overall Height SW Seat Width SD Seat Depth SH Seat Height AH Arm Height Thk -Thickness Dia Diameter IW Internal Width ID Internal Depth IH Internal Height LW Leg Width LD Leg Depth LH Leg Height WH Wood Plank Height FH Footboard Height For all our office chairs, Arm Height (AH) and Overall Chairï¿½s Height (OH) is taken when Seat Height (SH) is fixed at 450mm. Should an office chairï¿½s SH be higher than 450mm, then AH and OH will be taken at the lowest available SH of that chair. * All measurements given are in millimetres (mm).',
  {'entities': [(85, 104, 'PRODUCT'), (105, 124, 'PRODUCT')]}),
 ('Our friendly team will help customers to load bulky furniture items into their cars. Comfort Acrylic Shield W1200 Protect your employees and customers from the potential spread of COVID-19 with minimal impact on your interiorï¿½s aesthetics. The Comfort Shield in acrylic is transparent, preventing visual blocking and allowing stains to be spotted easily. Comes with a stable self-standing capability, the shields can be placed at any table easily and stably; making it great for transient places. No drilling, no blue tags, no accidents The Comfort Acrylic Shield is your perfect tool for enforcing safe distancing. Our table shield allows businesses to remain open as staff are protected and customers are assured. Implementing shields will be the new normalcy for how companies should operate. Download the PDF Catalogue: click here. Want to keep these shields secured Purchase the 3M Command Strips for a semi-permanent fix 3M Command Large Refill Strips 8pc Pack* 1 set (sufficient for 4 acrylic stands) (+SGD5.90) Great for reception counters, cashiers and other counters with frontline staff Great for restaurants, especially on communal tables ',
  {'entities': [(85, 107, 'PRODUCT'),
    (244, 258, 'PRODUCT'),
    (541, 563, 'PRODUCT')]}),
 ('Our friendly team will help customers to load bulky furniture items into their cars. Dynamo Armchair w/ Castors (Nestable) Voted: Best backrest with comfortable tilt function in breathable mesh The petite Dynamo Training Chair is great for the dynamic office as well as the dynamic demands of our bodies. This training or seminar chair comes with wheels, making it easy to shift around. Dynamo also works as an ideal office chair in a space strap work environment. Its mesh backrest comes with back-tilt function, great for a good torso stretch at work! Need more reasons This chair is also nestable for compact storage after an event! Dynamo Castors Chair is available in tablet and non-tablet versions. Product Specifications Tilt-able backrest for long hours of sitting comfort Foldable fabric seat in moulded foam Nestable (horizontal stacking) for storage needs Die-cast Aluminium Connectors on the backrest for added strength Mesh Backrest from Korea Fabric Seat thatï¿½s FR-treated for fire-retardancy Castors (wheels) for mobility and convenience (fixed glides version also available) ',
  {'entities': [(85, 100, 'PRODUCT'),
    (205, 226, 'PRODUCT'),
    (387, 393, 'PRODUCT'),
    (636, 656, 'PRODUCT')]}),
 ('Our friendly team will help customers to load bulky furniture items into their cars. Dynamo Tablet Armchair w/ Castors (Nestable) The petite Dynamo Training Chair is great for the dynamic office as well as the dynamic demands of our bodies. This training or seminar chair comes with wheels, making it easy to shift around. Dynamo also works as an ideal office chair in a space strap work environment. Its mesh backrest comes with back-tilt function, great for a good torso stretch at work! Need more reasons This chair is also nestable for compact storage after an event! Dynamo Castors Chair is available in tablet and non-tablet versions. Product Specifications Writing tablet that comes with lock mechanism to prevent accidents from occurring Comes with retractable cup holder Writing tablet is constructed with strong die-cast aluminium structure on the underside for added strength Tilt-able backrest for long hours of sitting comfort Foldable fabric seat in moulded foam Nestable (horizontal stacking) for storage needs Die-cast Aluminium Connectors on the backrest for added strength Mesh Backrest from Korea Fabric Seat thatï¿½s FR-treated for fire-retardancy Castors (wheels) for mobility and convenience (fixed glides version also available) ',
  {'entities': [(85, 107, 'PRODUCT'),
    (141, 162, 'PRODUCT'),
    (323, 329, 'PRODUCT'),
    (572, 592, 'PRODUCT')]}),
 ('Our friendly team will help customers to load bulky furniture items into their cars. Dynamo Tablet Armchair w/ Fixed Glides (Nestable) The petite Dynamo Training Chair is great for the dynamic office as well as the dynamic demands of our bodies. This version of the training or seminar chair comes with no wheels, ideal for those who prefers a static chair. Dynamo also works as an ideal office chair in a space strap work environment. Its mesh backrest comes with back-tilt function, great for a good torso stretch at work! Need more reasons This chair is also nestable for compact storage after an event! Dynamo Glides Chair is available in tablet and non-tablet versions. Product Specifications Writing tablet that comes with lock mechanism to prevent accidents from occurring Comes with retractable cup holder Writing tablet is constructed with strong die-cast aluminium structure on the underside for added strength Tilt-able backrest for long hours of sitting comfort Foldable fabric seat in moulded foam Nestable (horizontal stacking) for storage needs Die-cast Aluminium Connectors on the backrest for added strength Mesh Backrest from Korea Fabric Seat thatï¿½s FR-treated for fire-retardancy Fixed glides (castor wheels version also available) ',
  {'entities': [(85, 107, 'PRODUCT'),
    (146, 167, 'PRODUCT'),
    (358, 364, 'PRODUCT'),
    (607, 626, 'PRODUCT')]}),
 ("The Edgar Study Table is a 1m wide table in woodgrain laminate and white steel legs that's ideal for smaller study rooms or bedrooms! The laminate table top also makes cleaning easier due to its protective layer that prevents spills from being absorbed. Pair it with a Shonda Mobile Pedestal for a productive WFH setup. OW Overall Width OD Overall Depth OH Overall Height SW Seat Width SD Seat Depth SH Seat Height AH Arm Height Thk -Thickness Dia Diameter IW Internal Width ID Internal Depth IH Internal Height LW Leg Width LD Leg Depth LH Leg Height WH Wood Plank Height FH Footboard Height For all our office chairs, Arm Height (AH) and Overall Chairï¿½s Height (OH) is taken when Seat Height (SH) is fixed at 450mm. Should an office chairï¿½s SH be higher than 450mm, then AH and OH will be taken at the lowest available SH of that chair. ",
  {'entities': [(4, 21, 'PRODUCT')]}),
 ('Voted: Best support as an office chair with no wheels A cantilevered office chair with a bouncy seat feel, providing a little movement to a sedentary position. Hancisco comes with no wheels, perfect as a visitor chair or as a study chair option thatï¿½s without wheels. OW Overall Width OD Overall Depth OH Overall Height SW Seat Width SD Seat Depth SH Seat Height AH Arm Height Thk -Thickness Dia Diameter IW Internal Width ID Internal Depth IH Internal Height LW Leg Width LD Leg Depth LH Leg Height WH Wood Plank Height FH Footboard Height For all our office chairs, Arm Height (AH) and Overall Chairï¿½s Height (OH) is taken when Seat Height (SH) is fixed at 450mm. Should an office chairï¿½s SH be higher than 450mm, then AH and OH will be taken at the lowest available SH of that chair. * All measurements given are in millimetres (mm). ',
  {'entities': [(160, 168, 'PRODUCT')]}),
 ('The low-back, petite Luster Chair is great for SOHO offices, hotels, or anyone looking for a nice looking task chair without arms. This chair will be able to tuck into all types of tables, regardless of the table top thickness; ensuring that passageways are cleared when chairs are not in use. Concerned about the wheels scratching your flooring Consider getting a PVC Floor Mat for added protection. Up Close & Personal Imagine It How It Can Accentuate Your Space Product Specifications Features Fixed Seat Height Non-tilt backrest Available in Polished Aluminium or Hairline Rose Gold base OW Overall Width OD Overall Depth OH Overall Height SW Seat Width SD Seat Depth SH Seat Height AH Arm Height Thk -Thickness Dia Diameter IW Internal Width ID Internal Depth IH Internal Height LW Leg Width LD Leg Depth LH Leg Height WH Wood Plank Height FH Footboard Height For all our office chairs, Arm Height (AH) and Overall Chairï¿½s Height (OH) is taken when Seat Height (SH) is fixed at 450mm. Should an office chairï¿½s SH be higher than 450mm, then AH and OH will be taken at the lowest available SH of that chair. * All measurements given are in millimetres (mm). ',
  {'entities': [(21, 33, 'PRODUCT')]}),
 ('Our friendly team will help customers to load bulky furniture items into their cars. Rib L-Shaped Acoustic Screen Up Close & Personal Imagine It How It Can Accentuate Your Space Product Specifications Sound absorbing and fire retardant E0 (emissions-free) LEED points for using recycled materials Clean manufacturing process (no glue, dye, chemical binder used) 100% recyclable Mildew-proof Thermal insulated Each L-Shaped Screen includes 3 units of L-bracket for installation to table. Installation onto tables are included if the entire table is purchased from us. Recommended height to screw screen is at 350mm above the table; leaving a balance of 240mm from below the top of the table top. ',
  {'entities': [(85, 133, 'PRODUCT'), (414, 429, 'PRODUCT')]}),
 ('Kitson L Shaped Sofa Bed The Kitson L shaped sofa bed is a very smart, chic sofa with extended chaise finished in soft fabric and keeping with the current trend of splayed legs, in a light oak finish. The Kitson is a fantastic three-seater sofa for you to put your feet up and relax on ï¿½ and even take a nap on when extended into a sofa bed. The Kitson can also be swapped from left to right-hand chaise by using the integral fixing brackets. Shop our full range of sofa beds here! L-shape sofa bed Contemporary style Converted from a left to a right-handed sofa Enables to be compatible with all room types. Folds down easily to become a bed when required. Easily transforms from sofa to bed Dimensions: Sofa: W 198 cm x D 113 cm x H 86 cm Sofa Bed: L 198 cm x W 113 cm x D 40 cm',
  {'entities': [(0, 24, 'PRODUCT'),
    (29, 53, 'PRODUCT'),
    (205, 211, 'PRODUCT'),
    (482, 498, 'PRODUCT')]}),
 ('Brighton Velvet Sofa Bed The Brighton sofa bed makes an everyday item a little bit more extra. Upholstered with a soft and cosy velvet this sofa bed will look super sophisticated in any home. Available in four beautiful colours, the Brighton has considered all the details too, even with detail on the legs which are a dark colour so will enhance the materials look. The Brighton will be a relaxing and comfortable piece of furniture whilst also looking good amongst all interior. The Brighton is an effortless day to night option which is easy to use for either yourself or your guests. Available in Green, Orange, Pink and Grey; the Brighton velvet sofa bed will go with any decor or background in any room of your choice. Available in Green, Orange, Pink and Grey Upholstered Velvet Detailed Dark Legs Easy-to-use Dimensions: W 180 cm X H 76 cm X D 85 cm',
  {'entities': [(0, 24, 'PRODUCT'),
    (29, 46, 'PRODUCT'),
    (233, 241, 'PRODUCT'),
    (371, 379, 'PRODUCT'),
    (635, 659, 'PRODUCT')]}),
 ('Vogue Leather Sofa Bed Like the song, the Vogue Futon Sofa Bed is full of style. A contemporary sofa bed that oozes class and sophistication, the Vogue Futon is ideal as a standalone piece of practical and stylish furniture. Wrapped in luxurious black faux leather with button and stitching detail, the slightly raised ends and curved chrome finish legs give a more expensive feel. This sofa easily transforms into a spacious spare bed by simply folding out the back rest. Faux Leather Contemporary Style Pre-Assembled Freestanding Available in 3 colours Dimensions: W 1850 mm X H 860 mm X D 970 mm',
  {'entities': [(0, 22, 'PRODUCT'),
    (42, 62, 'PRODUCT'),
    (146, 157, 'PRODUCT')]}),
 ('Nico Green Sofa Bed Luxury at its finest, the Nico sofa is practical yet stylish. With eye-catching, beautiful cooper effect base and legs, the lavish velvet reflects against this to provide a plush comfy seating area. This is a great asset to have in your home as it is easy and ready to transfer into space for your guests to sleep! Velvet upholstery Copper Legs Modern stylish design Dimensions: W 2100 mm X H 850 mm X D 830 mm',
  {'entities': [(0, 19, 'PRODUCT'), (46, 55, 'PRODUCT')]}),
 ('Lille Extending 4-6 Seater Dining Table ï¿½799 ï¿½699 In stock Available for immediate delivery The Lille dining table is an elegant design. The white coloured tempered glass top is supported by a durable fibre glass tulip style base. The beautifully smooth extending mechanism is very ergonomic, simply using one hand to push the table top, it rotates to reveal the two end extension leaves, then to close the extension use the same method to just push the top and the extension leaves rotate back inwards. Select a colour 3 colours available Select a colour Lille Glass White ï¿½699 Mixed Materials Lille Marble White ï¿½799 Mixed Materials Lille Marble Light Grey ï¿½799 Mixed Materials Dimensions SKU number: 51K1TL5CN Height: 75cm Width: 119cm to 178cm Depth: 89cm Packaged dimensions: box 1 H23cm x W130cm x D100cm, box 2 H73cm x W84cm x D54cm Weight: 104kg More information Main material: White tempered glass, fibre glass, metal Secondary material: Fibreglass Extendable: Yes Assembly instructions: Full assembly required Care instructions: Wipe up spillages immediately to prevent marking. Dust with a soft, clean, lint-free cloth Click and Collect: No Guarantee period: 2 years Product features: Rotating extending mechanism Shape: Oval Dining table size: 4-6 seater',
  {'entities': [(0, 39, 'PRODUCT'),
    (96, 114, 'PRODUCT'),
    (556, 561, 'PRODUCT'),
    (595, 600, 'PRODUCT'),
    (635, 640, 'PRODUCT')]}),
 ('Shabby Sectional Sofa High impact comfort seating, commonly known as our true sloucher Sectional Set Shabby Sectional Sofa Sectional Set High impact comfort seating, commonly known as our true sloucher 360 View Image Gallery Shabby The ultimate sloucher for a guaranteed chill-out High impact comfort seating, commonly known as our true sloucher. The Shabby leather sectional sofa is beautifully cushioned with layers of soft ribs stitched into luxurious handfinished leather to create a distinctive signature look. More views of this product Select Your Preference to Ask Price Shabby Sectional Sofa comes in a range of configurations Group Modules All Sectional Set 70.9L x 47.6W x 29.9H 180cm x 121cm x 76cm Select your hand-finished leather or fabric Featured Finishes Leathers Fabrics DESTROYED BLACK OLD SADDLE BLACK VINTAGE BIANCO SAVAGE SAFARI CHARCOAL VAGABOND BROWN FADED & DEGRADED MELTING PAISLEY FADED & DEGRADED PEAT SMUDGE REVIVAL VELVET GREY Add To Wishlist Request Price Request Price Shabby Sectional Sofa Size Finish Request Price Add To Wishlist Next Request Price Choose your preferred method to request a price from our Ambassador.',
  {'entities': [(0, 21, 'PRODUCT'),
    (225, 231, 'PRODUCT'),
    (351, 380, 'PRODUCT'),
    (579, 600, 'PRODUCT'),
    (1002, 1023, 'PRODUCT')]}),
 ('Nirvana Large Sectional Sofa With classic loft-style proportions, the Nirvana sectional sofa creates a haven in any corner. Sectional Set Nirvana Large Sectional Sofa Sectional Set With classic loft-style proportions, the Nirvana sectional sofa creates a haven in any corner. Nirvana Large With classic loft-style proportions, the Nirvana sectional sofa creates a haven in any corner  When it comes to true lounging, a sectional sofa is perfect for sitting, lying, leaning and draping in comfort. With classic loft-style proportions, the Nirvana sectional sofa creates a haven in any corner. A sandwich of multiple density high quality foam and fibre layers are supported by pocket springs to create a luxuriously loungeful experience. A low back keeps it casual while angled arms are cushioned just the right amount for slouching. More views of this product Select Your Preference to Ask Price Nirvana Large Sectional Sofa comes in a range of configurations Group Modules All Sectional Set 48W x 28.3H 122cm x 72cm Select your hand-finished leather or fabric Featured Finishes Leathers Fabrics DESTROYED BLACK SAFARI CHARCOAL SAFARI TOBACCO REVIVAL VELVET NAVY Add To Wishlist Request Price Request Price Nirvana Large Sectional Sofa Size Finish Request Price Add To Wishlist Next Request Price Choose your preferred method to request a price from our Ambassador.',
  {'entities': [(0, 28, 'PRODUCT'),
    (70, 92, 'PRODUCT'),
    (138, 166, 'PRODUCT'),
    (222, 244, 'PRODUCT'),
    (276, 283, 'PRODUCT'),
    (331, 353, 'PRODUCT'),
    (538, 560, 'PRODUCT'),
    (895, 923, 'PRODUCT'),
    (1206, 1234, 'PRODUCT')]}),
 ('Nest Small Sectional Sofa One of the worldï¿½s most disarmingly comfortable sofas Nest Small Sectional Sofa One of the worldï¿½s most disarmingly comfortable sofas 360 View Image Gallery Nest Small A more compact version of the Nest sectional, offering epic comfort with a generous filling of ultrafine goose feathers swathed under natural linens, dyed using 100% natural vegetable dyes. Some support is offered in the seat and back making Nest a versatile choice for watching, listening, reading or entertaining. More views of this product Select Your Preference to Ask Price Nest Small Sectional Sofa comes in a range of configurations Group Modules All Sectional Set Select your hand-finished fabric Featured Finishes Fabrics OLD LOOM-ABYSS OLD LOOM-OCEAN OLD LOOM-PURPLE OLD LOOM-SKY SOL LINEN-BLUEBELL SOL LINEN-LAVENDER SOL LINEN-NATURAL SOL LINEN-WHITE Add To Wishlist Request Price Request Price Nest Small Sectional Sofa Size Finish Request Price Add To Wishlist Next Request Price Choose your preferred method to request a price from our Ambassador.',
  {'entities': [(0, 25, 'PRODUCT'),
    (80, 105, 'PRODUCT'),
    (183, 193, 'PRODUCT'),
    (224, 238, 'PRODUCT'),
    (436, 440, 'PRODUCT'),
    (573, 598, 'PRODUCT'),
    (900, 925, 'PRODUCT')]}),
 ('Nirvana Medium Sectional Sofa With classic loft-style proportions, the Nirvana sectional sofa creates a haven in any corner. Sectional Set Nirvana Medium Sectional Sofa Sectional Set With classic loft-style proportions, the Nirvana sectional sofa creates a haven in any corner. Nirvana Medium With classic loft-style proportions, the Nirvana sectional sofa creates a haven in any corner  When it comes to true lounging, a sectional sofa is perfect for sitting, lying, leaning and draping in comfort. With classic loft-style proportions, the Nirvana sectional sofa creates a haven in any corner. A sandwich of multiple density high quality foam and fibre layers are supported by pocket springs to create a luxuriously loungeful experience. A low back keeps it casual while angled arms are cushioned just the right amount for slouching. More views of this product Select Your Preference to Ask Price Nirvana Medium Sectional Sofa comes in a range of configurations Group Modules All Sectional Set 39.8W x 28.3H 101cm x 72cm Select your hand-finished leather or fabric Featured Finishes Leathers Fabrics DESTROYED BLACK SAFARI CHARCOAL SAFARI TOBACCO REVIVAL VELVET NAVY ',
  {'entities': [(0, 29, 'PRODUCT'),
    (71, 93, 'PRODUCT'),
    (139, 168, 'PRODUCT'),
    (224, 246, 'PRODUCT'),
    (278, 285, 'PRODUCT'),
    (334, 356, 'PRODUCT'),
    (541, 563, 'PRODUCT'),
    (898, 927, 'PRODUCT')]}),
 ("Zenna Sectional Sofa With feather-filled cushions to sink into on the seat and back, the Zenna sectional sofa sits low... Sectional Set Zenna Sectional Sofa Sectional Set With feather-filled cushions to sink into on the seat and back, the Zenna sectional sofa sits low to the ground for a laid-back feel. Zenna Unwind in serene comfort. The Zenna sectional sofa is made for sprawling; a place to unwind, whatever kind of day you've had. With feather-filled cushions to sink into on the seat and back, the Zenna sectional sits low to the ground for a laid-back feel. Crafted on a hardwood frame, a special suspension system of pocket springs and fibre webbing is topped with foam to ensure core comfort and support, with a sumptuous layer of feathers that are innovatively channel-stitched to eliminate the need for fluffing; all culminating in a super serene experience. More views of this product Select Your Preference to Ask Price Zenna Sectional Sofa comes in a range of configurations Group Modules All Sectional Set 48W x 27.5H 122cm x 70cm Select your hand-finished leather or fabric Featured Finishes Leathers Fabrics OLD SADDLE BLACK VINTAGE CIGAR SAVAGE SAFARI BLACK SAFARI CHARCOAL ",
  {'entities': [(0, 20, 'PRODUCT'),
    (89, 109, 'PRODUCT'),
    (136, 156, 'PRODUCT'),
    (239, 259, 'PRODUCT'),
    (305, 310, 'PRODUCT'),
    (341, 361, 'PRODUCT'),
    (505, 520, 'PRODUCT'),
    (934, 954, 'PRODUCT')]}),
 ('Scrambler Sectional Sofa Scrambler Sectional Sofa Scrambler Sectional Sofa Inspired by the trend of scrambler motorcycles which began in the UK; everyday road bikes customised by racers to perform on any terrain. The same ideology of customisation and versatility was applied to the Scrambler sectional sofa, beginning with the seat comfort, which is designed with an initial soft layer transitioning to a more supportive and incredibly comfortable sit, reflecting the modified seat suspension of the motorcycles. The leather is perforated on the seat and seat back, finished off with a light distress to mimic vintage motorcycle riding leathers. The side table was inspired by the clutch plates of a transmission, with exposed weld marks to embody the raw machinery of the original scramblers. Just as scrambler motorcycles evolved to conquer any terrain, the Scrambler sectional can be customised in a range of configurations to conquer any space. Scrambler is available worldwide. Discover the rest of the Outlaw Collection at our galleries in the UK, US and Hong Kong only. Select Your Preference to Ask Price Group Modules All Featured Finishes Leathers Size Finish',
  {'entities': [(0, 24, 'PRODUCT'),
    (25, 49, 'PRODUCT'),
    (50, 74, 'PRODUCT'),
    (100, 121, 'PRODUCT'),
    (283, 307, 'PRODUCT'),
    (803, 824, 'PRODUCT'),
    (861, 880, 'PRODUCT'),
    (950, 959, 'PRODUCT')]}),
 ('Oasis Sectional Sofa Blending epic comfort with a more supportive sit Oasis Sectional Sofa Blending epic comfort with a more supportive sit 360 View Image Gallery Oasis Part of the Noble Souls collection, offering epic comfort with a generous filling of ultrafine goose feathers swathed under natural linens, dyed using 100% natural vegetable dyes. Oasis offers a more structured seating contour than Realm or Nest, blending disarming comfort with some support, ideal for reading or entertaining. More views of this product Select Your Preference to Ask Price Oasis Sectional Sofa comes in a range of configurations Group Modules All Sectional Set Select your hand-finished fabric Featured Finishes Fabrics OLD LOOM-ABYSS OLD LOOM-OCEAN OLD LOOM-PURPLE SOL LINEN-BRUNNERA SOL LINEN-NATURAL SOL LINEN-WHITE Add To Wishlist Request Price Request Price Oasis Sectional Sofa Size Finish Request Price Add To Wishlist Next Request Price Choose your preferred method to request a price from our Ambassador.',
  {'entities': [(0, 20, 'PRODUCT'),
    (70, 90, 'PRODUCT'),
    (560, 580, 'PRODUCT'),
    (850, 870, 'PRODUCT')]}),
 ('Studio Sectional Sofa A tribute to true softness, the contemporary-styled Studio sectional sofa invites refined relaxation. Sectional Set Studio Sectional Sofa Sectional Set A tribute to true softness, the contemporary-styled Studio sectional sofa invites refined relaxation. Studio A tribute to true softness, the contemporary-styled Studio sectional sofa invites refined relaxation. Fully-loaded feather pillows are wrapped around a gently supportive core, hugging the body for a joyfully sumptuous experience. The perfect sofa for the urban jungle, to relax with friends and let your imagination run wild. More views of this product Select Your Preference to Ask Price Studio Sectional Sofa comes in a range of configurations Group Modules All Sectional Set 39.4W x 27.9H 100cm x 71cm Select your hand-finished leather or fabric Featured Finishes Leathers Fabrics VINTAGE BIANCO VAGABOND BROWN ACID JUNGLE REVIVAL VELVET GREY Add To Wishlist Request Price Request Price Studio Sectional Sofa Size Finish Request Price Add To Wishlist Next Request Price Choose your preferred method to request a price from our Ambassador.',
  {'entities': [(0, 21, 'PRODUCT'),
    (74, 80, 'PRODUCT'),
    (138, 159, 'PRODUCT'),
    (226, 247, 'PRODUCT'),
    (276, 282, 'PRODUCT'),
    (335, 356, 'PRODUCT'),
    (672, 693, 'PRODUCT'),
    (973, 994, 'PRODUCT')]}),
 ('Realm Sectional Sofa Realm Sectional Sofa 360 View Image Gallery Realm Part of the Noble Souls collection, offering epic comfort with a generous filling of ultrafine goose feathers swathed under natural linens, dyed using 100% natural vegetable dyes. Realm offers the most laid-back seating position within the collection, perfect for kicking back, watching an epic movie, or getting lost in music. Note: The Realm Sectional Sofa is not available in the US Select Your Preference to Ask Price Group Modules All Featured Finishes Fabrics Size Finish',
  {'entities': [(0, 20, 'PRODUCT'), (409, 429, 'PRODUCT')]}),
 ('Shabby Curve Sofa High impact comfort seating, commonly known as our true sloucher Sectional Set 43.3L x 43.3W x 29.9H 110cm x 110cm x 76cm Shabby Curve Sofa Sectional Set 43.3L x 43.3W x 29.9H 110cm x 110cm x 76cm High impact comfort seating, commonly known as our true sloucher 360 View Image Gallery Shabby Curve The ultimate sloucher for a guaranteed chill-out High impact comfort seating, commonly known as our true sloucher. The Shabby leather sectional sofa is beautifully cushioned with layers of soft ribs stitched into luxurious handfinished leather to create a distinctive signature look. More views of this product Select Your Preference to Ask Price Shabby Curve Sofa comes in a range of configurations Group Modules All Sectional Set 43.3W x 29.9H 110cm x 76cm Select your hand-finished leather or fabric Featured Finishes Leathers Fabrics DESTROYED BLACK OLD SADDLE BLACK VINTAGE BIANCO SAVAGE SAFARI CHARCOAL VAGABOND BROWN FADED & DEGRADED MELTING PAISLEY REVIVAL VELVET GREY Add To Wishlist Request Price Request Price Shabby Curve Sofa Size Finish Request Price Add To Wishlist Next Request Price Choose your preferred method to request a price from our Ambassador.',
  {'entities': [(0, 17, 'PRODUCT'),
    (140, 157, 'PRODUCT'),
    (303, 315, 'PRODUCT'),
    (435, 464, 'PRODUCT'),
    (663, 680, 'PRODUCT'),
    (1037, 1054, 'PRODUCT')]}),
 ('Fried Egg A unique asymmetrical armchair, designed by the highly acclaimed Danish architect, Hans Olsen. Fried Egg is rooted in 1950s Danish Modernism and appeals to design lovers who like an edgy touch in their homes in the shape of an extraordinary piece of designer furniture that features both personality and warmth. Due to its distinctive shape, this roomy, comfortable chair was christened FRIED EGG, because that is exactly what it looks like when viewed from above. The flat armrest invites you to swing your legs nonchalantly up over the edge, when you really feel like relaxing. To facilitate the ultimate flexibility when it comes to interior design, Fried Egg is now available in both a right-hand and a left-hand version. This also makes it possible to create a lounge corner with two armchairs mirroring each other, thereby creating a perfect spot for long, relaxed conversations. In homage to the 1950sï¿½ love of colour and teak, Fried Egg is manufactured in this elegant wood with upholstery in a range of carefully selected colours from the Warm Nordic universe.',
  {'entities': [(0, 9, 'PRODUCT'),
    (105, 114, 'PRODUCT'),
    (397, 406, 'PRODUCT'),
    (663, 672, 'PRODUCT'),
    (945, 954, 'PRODUCT')]}),
 ('Fried Egg A unique asymmetrical lounge chair, designed by the highly acclaimed Danish architect, Hans Olsen in the 1950s. Fried Egg is also available in an exclusive anniversary edition with sustainable fabric, launched to celebrate the 100th birthday of Hans Olsen. The Fried Egg chair, in all its personality and capriciousness, is at last available for any design lovers who wish to add a bit of edginess to their homes in the shape of a unique item of designer furniture with roots in Danish Modernism. This roomy, comfortable chair was christened Fried Egg, because that is what it looks like when viewed from above. The flat armrests invite you to swing your legs nonchalantly up on them, when you really feel like relaxing. As a tribute to the 1950sï¿½ adoration of colour and love of teak, one of the Fried Egg options is this elegant type of wood and upholstery in a range of stunning colours. You also have the option of playing with two-tone combinations, which are strongly reminiscent of a fried egg. Choose an exclusive furnishing fabric from Kvadrat in a carefully selected shade from the Warm Nordic universe. The frame is available in the highest quality teak. The anniversary edition of Fried Egg is available in two beautiful options, both of which add an extra sensuousness to the Fried Egg chair by combining three different colours on the inside, outside, and on the cushion. The fabrics are a combination of Hero and sustainable textiles, Re-wool and Merit by Maharam, in colours with a modern and light expression.',
  {'entities': [(0, 9, 'PRODUCT'),
    (122, 131, 'PRODUCT'),
    (552, 561, 'PRODUCT'),
    (807, 816, 'PRODUCT'),
    (1203, 1212, 'PRODUCT')]}),
 ('Metallic Tv Console by coaster This radiant TV console will add a handful of dazzle to any home bar or entertainment room. Built from rubber wood, MDF, and mirror, it has a look that might steal the spotlight from whatï¿½s on TV. Features two adjacent open storage spaces beneath the TV and three spacious middle drawers with French dovetail/full extension glides below that. Brilliant tapered legs and metallic platinum finish convey a futuristic, avant-garde sensibility. Console fits a 65-inch television and includes two cabinets that open up for more storage. Attention: Prop 65 Information TV console finished in metallic platinum with mirrored frame and door/drawer fronts Constructed with rubberwood, MDF, and mirror Features two doors with two (2) adjustable shelves, three (3) storage drawers, and two (2) storage compartments Full extension drawer glides Finish: Metallic platinum Max TV size: 65ï¿½; Max TV weight: 150 lbs. Assembly type: KD legs Hardware finish: Champagne Assembly required: Yes',
  {'entities': [(0, 19, 'PRODUCT'),
    (44, 54, 'PRODUCT'),
    (472, 479, 'PRODUCT'),
    (594, 604, 'PRODUCT'),
    (872, 896, 'PRODUCT'),
    (908, 914, 'PRODUCT')]}),
 ('Tv Stand Set | CM5900-TV Curate your living space for the ultimate gaming and entertainment experience. The gray finished entertainment set creates cool tones that instill a relaxing ambiance. Intricate wavy designs create texturized patterns, adding chic edgy detailing. Simply organize with glass paneled cabinets and open shelves that frame the structural design while displaying collectibles in the glass-framed curio and complete the aesthetic with your TV perfectly centered for a seamless entertainment experience. Transitional Style8mm Tempered Glass ShelvesStorage Drawers & ShelvesSolid Wood, Wood Veneer & Others*Gray Finish TV STAND [CM5900-TV] 76ï¿½ï¿½L X 16ï¿½ï¿½W X 31ï¿½ï¿½H PIER CABINET [CM5900-PC] 24ï¿½ï¿½L X 14ï¿½ï¿½W X 78ï¿½ï¿½H BRIDGE [CM5900-BR] 76ï¿½ï¿½L X 14ï¿½ï¿½W X 44 1/2ï¿½ï¿½H',
  {'entities': [(0, 12, 'PRODUCT'), (636, 644, 'PRODUCT')]}),
 ('DescriptionMore Images Wells Entertainment Center | by ID USA Item Color: Distressed Grey Also available in Black Item Features: Media Tower | 171916P 20ï¿½W x 16ï¿½D x 72ï¿½H Two drawers on metal glides Three shelves TV Stand | 171916TV 60ï¿½W x 16ï¿½D x 22ï¿½H Two drawers on metal glides Three shelves holds up to three players Bridge | 171916BR 60ï¿½W x 8ï¿½D x 11ï¿½H Complete Entertainment Center measures: 100ï¿½ in Width *All measurements are approximate',
  {'entities': [(23, 49, 'PRODUCT')]}),
 ('DescriptionDimensionsMore Images Mandy Entertainment Center | by ID USA Item Color: Dark Taupe Also available in Red Cocoa Item Features: All Complete finishTwo side towers with eight shelvesSeven drawersTop shelf storage spaceSix Adjustable shelves Tv Stand | 171919 Width:15.5in Depth:60in Height:33.75in Audio Pier | 171920 Width:11.5in Depth:21.25in Height:82in Bridge | 171921 Width:62.25in Depth:11.5in Height:11.5in *All measurements are approximate',
  {'entities': [(33, 59, 'PRODUCT')]}),
 ('Comfort That Fits Your Life. Available in seven sizes from cot to king, thereï¿½s a Comfort Sleeper that fits your lifestyle and your living space. Each size is a full 80 inches in length ï¿½ five to 10 inches longer than traditional sleepers ï¿½ yet the innovative design requires less floor space for even greater flexibility. Configurations That Fit Your Style. And Your Space. American Leather understands all living spaces are not created equal. The Comfort Sleeper Collection fits effortlessly in any room ï¿½ from refined and sophisticated to casual and cozy. You have full control of size and configuration selection to perfectly fit your space. Here are just a few suggestions: Effortless Style. Matchless Comfort. Comfort Sleeper by American Leather is quite simply the most comfortable AND most versatile sleeper sofa available. With 14 styles, from transitional to modern, youï¿½ re sure to find the stylish centerpiece you want ï¿½ with a better nightï¿½s sleep built right in. Every Comfort Sleeper style has matching pieces available ï¿½ sectionals, sofas, loveseats, chairs and ottomans ï¿½ to complete your room, your way.',
  {'entities': [(82, 97, 'PRODUCT'),
    (449, 464, 'PRODUCT'),
    (716, 731, 'PRODUCT'),
    (983, 998, 'PRODUCT')]}),
 ('Washington DCï¿½s Finest Selection Of Modern Living Room Furniture View our collections of modern living room furniture including sofas, sectionals, chairs, sleepers and sleep sofas all available in leather or fabric. Our modern sofas are American and Italian designed and manufactured. About Theodores Modern Furniture Store Theodores modern furniture showroom and interior design studio, located in upper Georgetown, features 10,000 square feet of the finest European and domestic home furniture. Please take a few moments to review the furniture featured on our website. Each item is selected for its design, quality and originality. We invite you to visit our modern showroom for the latest design ideas  ',
  {'entities': [(36, 64, 'PRODUCT'), (301, 317, 'PRODUCT')]}),
 ("Arm Balcony Stool (with optional seat pad) Collection: Coeur d'Alene Width (in): 23.5 Depth/Radius (in): 23.5 Height (in): 40 Arm Height (in): 34.5 Seat Height (in): 27.5 Weight (lbs): 25 Model Number: M74014BAC",
  {'entities': [(0, 17, 'PRODUCT')]}),
 ("Armless Balcony Stool (with optional seat pad) Collection: Coeur d'Alene Width (in): 23.5 Depth/Radius (in): 23.5 Height (in): 40 Seat Height (in): 27.5 Weight (lbs): 23 Model Number: M74014BC Please be advised that the fulfillment of accessory orders may take up to 21 days from the date of order due to COVID-19 production delays. Some manufacturer delays are also being experienced. Our employees and our manufacturers and vendors appreciate your business and your patience during this International Public Health Emergency. Thank you. Account Wishlist Checkout Login or Register My Cart 0 Express Local Delivery NEED HELP (702) 309-4198 Up to 2 Year Warranty In-home Setup & Installation Product Added to Cart",
  {'entities': [(0, 21, 'PRODUCT')]}),
 ('Backless Balcony Stool Collection: Meza Width (in): 18.125 Depth/Radius (in): 17.25 Height (in): 24 Weight (lbs): 12 Model Number: M5694B Please be advised that the fulfillment of accessory orders may take up to 21 days from the date of order due to COVID-19 production delays. Some manufacturer delays are also being experienced. Our employees and our manufacturers and vendors appreciate your business and your patience during this International Public Health Emergency. Thank you. Account Wishlist Checkout Login or Register My Cart 0 Express Local Delivery NEED HELP (702) 309-4198 Up to 2 Year Warranty In-home Setup & Installation Product Added to Cart',
  {'entities': [(0, 22, 'PRODUCT')]}),
 ('Balcony Height Stool Collection: Vallero Crossweave Width (in): 24.75 Depth/Radius (in): 28 Height (in): 43 Arm Height (in): 32 Seat Height (in): 23.5 Weight (lbs): 16 Model Number: M310014B Please be advised that the fulfillment of accessory orders may take up to 21 days from the date of order due to COVID-19 production delays. Some manufacturer delays are also being experienced. Our employees and our manufacturers and vendors appreciate your business and your patience during this International Public Health Emergency. Thank you. Account Wishlist Checkout Login or Register My Cart 0 Express Local Delivery NEED HELP (702) 309-4198 Up to 2 Year Warranty In-home Setup & Installation Product Added to Cart',
  {'entities': [(0, 20, 'PRODUCT')]}),
 ('Balcony Height Stool Collection: Nottingham Width (in): 25.5 Depth/Radius (in): 25.5 Height (in): 39 Arm Height (in): 30 Seat Height (in): 21.25 Weight (lbs): 27 Model Number: M44014B Please be advised that the fulfillment of accessory orders may take up to 21 days from the date of order due to COVID-19 production delays. Some manufacturer delays are also being experienced. Our employees and our manufacturers and vendors appreciate your business and your patience during this International Public Health Emergency. Thank you. Account Wishlist Checkout Login or Register My Cart 0 Express Local Delivery NEED HELP (702) 309-4198 Up to 2 Year Warranty In-home Setup & Installation Product Added to Cart',
  {'entities': [(0, 20, 'PRODUCT')]}),
 ('Balcony Height Stool Collection: Vision Sling Width (in): 24.5 Depth/Radius (in): 24.5 Height (in): 38 Arm Height (in): 30 Seat Height (in): 22.5 Weight (lbs): 18 Model Number: M4405SB Please be advised that the fulfillment of accessory orders may take up to 21 days from the date of order due to COVID-19 production delays. Some manufacturer delays are also being experienced. Our employees and our manufacturers and vendors appreciate your business and your patience during this International Public Health Emergency. Thank you. Account Wishlist Checkout Login or Register My Cart 0 Express Local Delivery NEED HELP (702) 309-4198 Up to 2 Year Warranty In-home Setup & Installation Product Added to Cart',
  {'entities': [(0, 20, 'PRODUCT')]}),
 ('Balcony Height Stool Collection: Scandia Crossweave Strap Width (in): 26 Depth/Radius (in): 26 Height (in): 39.5 Arm Height (in): 30.25 Seat Height (in): 23.5 Weight (lbs): 18 Model Number: M4505BCW Please be advised that the fulfillment of accessory orders may take up to 21 days from the date of order due to COVID-19 production delays. Some manufacturer delays are also being experienced. Our employees and our manufacturers and vendors appreciate your business and your patience during this International Public Health Emergency. Thank you.',
  {'entities': [(0, 20, 'PRODUCT')]}),
 ('Balcony Height Stool Collection: Shoreline Sling Width (in): 28.5 Depth/Radius (in): 25 Height (in): 41 Arm Height (in): 31.5 Seat Height (in): 23 Weight (lbs): 17 Model Number: M4905SB Please be advised that the fulfillment of accessory orders may take up to 21 days from the date of order due to COVID-19 production delays. Some manufacturer delays are also being experienced. Our employees and our manufacturers and vendors appreciate your business and your patience during this International Public Health Emergency. Thank you. Account Wishlist Checkout Login or Register My Cart 0 Express Local Delivery NEED HELP (702) 309-4198 Up to 2 Year Warranty In-home Setup & Installation Product Added to Cart',
  {'entities': [(0, 20, 'PRODUCT')]}),
 ('Balcony Height Stool Collection: Array Sling Width (in): 27.5 Depth/Radius (in): 29.5 Height (in): 40 Arm Height (in): 28 Seat Height (in): 23 Weight (lbs): 17 Model Number: M5105SB Please be advised that the fulfillment of accessory orders may take up to 21 days from the date of order due to COVID-19 production delays. Some manufacturer delays are also being experienced. Our employees and our manufacturers and vendors appreciate your business and your patience during this International Public Health Emergency. Thank you. Account Wishlist Checkout Login or Register My Cart 0 Express Local Delivery NEED HELP (702) 309-4198 Up to 2 Year Warranty In-home Setup & Installation Product Added to Cart',
  {'entities': [(0, 20, 'PRODUCT')]}),
 ('Balcony Stool Collection: Edge Padded Sling Width (in): 26.5 Depth/Radius (in): 25 Height (in): 42 Arm Height (in): 29 Seat Height (in): 24 Weight (lbs): 24 Model Number: M5205PSB Please be advised that the fulfillment of accessory orders may take up to 21 days from the date of order due to COVID-19 production delays. Some manufacturer delays are also being experienced. Our employees and our manufacturers and vendors appreciate your business and your patience during this International Public Health Emergency. Thank you. Account Wishlist Checkout Login or Register My Cart 0 Express Local Delivery NEED HELP (702) 309-4198 Up to 2 Year Warranty In-home Setup & Installation Product Added to Cart',
  {'entities': [(0, 13, 'PRODUCT')]}),
 ('Equinox L-Shaped Desk Available in 2 Colors Description Items Included L-Shaped Desk The Equinox collection is perfect if you are looking to maximize productivity and space in your office. The functional pieces from this collection feature multiple storage options so you can keep all the essentials on hand and boost your productivity. Our L-Shaped desk features a layered work surface to provide ample room for all your work activities. Pair it with our stylish storage units to create a chic and sophisticated office environment. Dimensions Overall Dimensions: 71.1W x 56D x 29.6H Specifications GOOD QUALITY: Made from high-density commercial-grade particle board compliant with TSCA Title VI. Durable laminate finish. Desktop is 1ï¿½ï¿½ thick. Sturdy construction. Leg levelers. Aluminum handles. Finished back. HIGHLY FUNCTIONAL: The layered work surface provides ample room for activities. Drawers on full-extension ball-bearing slides. Includes 1 grommet for wire management and 1 dual power/USB module. GOOD STORAGE: The credenza features 1 closed storage area with 1 adjustable shelf, 1 open storage area, 1 utility drawer, and 1 file drawer with letter/legal filing system. 5-YEAR LIMITED WARRANTY: Our desk is warranted to the original owner against defects in materials or workmanship. Surface resists scratches, stains and wear. Assembly required (2 people). MADE IN CANADA: Excellent customer support. The unit is fully reversible. Meets or exceeds ANSI/BIFMA performance standards.',
  {'entities': [(0, 21, 'PRODUCT'),
    (71, 84, 'PRODUCT'),
    (341, 354, 'PRODUCT')]}),
 ('Solay Contemporary 3-Piece Set Including an L-Shaped Desk, a Lateral File Cabinet and a Bookcase Available in 2 Colors Description Items Included L-Shaped Desk File Cabinet Bookcase Brighten your workspace with the new Solay workstation. The Solay workstation is the perfect fit for any area surrounding you with everything within armï¿½s reach. The work surface features a durable 1ï¿½ commercial grade work surface with melamine finish that resists scratches, stains and burns. One utility drawer and one file drawer with letter/legal filing system. Fully reversible design. Open storage sections. Metal leg. Layering workstation. The Lateral File features a durable 1ï¿½ commercial grade top surface with melamine finish and letter/legal filing system. The bookcase has three adjustable shelves and two fixed shelves. All drawers on ball-bearing slides for smooth and silent operation. Easy to assemble. Assembly required. Dimensions Desk: 59.3ï¿½W x 59.3ï¿½D x 29.7ï¿½H Lateral File Cabinet: 28ï¿½W x 19.6ï¿½D x 30.4ï¿½H Bookcase: 35.5ï¿½W x 11.9ï¿½D x 65.9ï¿½H',
  {'entities': [(0, 30, 'PRODUCT'),
    (219, 237, 'PRODUCT'),
    (242, 259, 'PRODUCT')]}),
 ('Universel 24ï¿½ x 48ï¿½ Table Desk with Square Metal Legs Available in 10 Colors Save 28% Original price $259.99 Current price $187.99 Description Items Included Table Desk The Universel collection has everything you need. You can furnish several rooms with a large variety of complementary furniture pieces found in this unique collection. Dimensions Overall Dimensions: 47.6W x 24D x 29H Specifications GOOD QUALITY: Made from high-density commercial-grade particle board compliant with TSCA Title VI. Durable laminate finish. Desktop is 1ï¿½ï¿½ thick. CONVENIENT: It can fit in any room and adapt to many uses. Leg levelers to adapt to any floor irregularities. MODERN LOOK: This table desk features square metal legs in matte silver finish to fit perfectly in modern spaces and add a stylish touch to any office. 10-YEAR LIMITED WARRANTY: Warranted to the original owner against defects in materials or workmanship. Surface resists scratches, stains and wear. Assembly required (2 people). MADE IN CANADA: Excellent customer support.',
  {'entities': [(0, 30, 'PRODUCT'), (158, 168, 'PRODUCT')]}),
 ('Pro-Linea L-Shaped Desk with Hutch Available in 2 Colors Description Items Included L-Shaped Desk The Pro-Linea office collection is perfect for anyone who feels that they need more storage space and more help staying organized. Declutter your space with this sturdy and well-designed collection and choose from the multiple configuration options that come with it. With wide, spacious desk surfaces and metal accents, the furniture gives you both functionality and style, the best of both worlds. Revamp your office space with this affordable collection. Dimensions Overall Dimensions: 71.1W x 71.1D x 67.7H Specifications GOOD QUALITY: Made from high-density commercial-grade particle board compliant with TSCA Title VI. Durable laminate finish. Desktop is 1. 5ï¿½ï¿½ thick. Credenza top is 1ï¿½ï¿½ thick. Metal legs with matte silver finish. HIGHLY FUNCTIONAL: This workstation provides an ample work surface, yet everything is within reach. Leg levelers. Return table features a deluxe impact resistant PVC edge. GOOD STORAGE: The credenza features 1 utility drawer and 1 file drawer with a letter/legal filing system. The credenza also offers 3 shelves (1 is adjustable). , and the hutch has 1 large fixed shelf for open storage. 10-YEAR LIMITED WARRANTY: Our desk is warranted to the original owner against defects in materials or workmanship. Surface resists scratches, stains and wear. Assembly required (2 people). MADE IN CANADA: Excellent customer support. The unit is fully reversible. This unit meets or exceeds ANSI/BIFMA performance standards.',
  {'entities': [(0, 23, 'PRODUCT'), (84, 97, 'PRODUCT')]}),
 ('Pro-Linea 2-Piece Set Including an L-Shaped Desk with Hutch and a Bookcase Available in 2 Colors Description Items Included L-Shaped Desk with Hutch Bookcase This office collection is perfect for anyone who feels that they need more storage space in order to stay organized. Decluttering your space is easy with these sturdy and well-designed pieces. In addition, the piecesï¿½ spacious work surfaces and metal accents give you get the best of both worlds: functionality and style. Revamp your office space with this affordable collection! Specifications QUALITY MATERIALS: Made from laminated, high-density commercial-grade particle board. Metal legs with matte silver finish. Desktop is 1. 5ï¿½ thick with melamine finish and impact-resistant PVC edge. Credenza top is 1ï¿½ thick. HIGHLY FUNCTIONAL: The desk provides an ample work surface, yet thanks to its shape, everything is within reach. The Credenza shelves can support up to 25 lb, the drawers can support up to 30 lb and the Credenza top can support up to 50 lb. GOOD STORAGE: Credenza: 1 utility drawer, 1 file drawer with a letter/legal filing system, 1 fixed shelf, 1 adjustable shelf. Hutch: 1 fixed shelf. Bookcase: 3 adjustable shelves and 2 fixed shelves. The bookcase has a finished back. 10YEAR LIMITED WARRANTY: These pieces are warranted to the original owner against defects in materials or workmanship. Easy return policy. Durable work surfaces resist scratches, stains and wear. NOTE: Assembly required (2 people). Excellent customer support. Pieces meet or exceed ANSI/BIFMA performance standards. Fully reversible.',
  {'entities': [(0, 21, 'PRODUCT')]}),
 ("Equinox L-Shaped Office Desk Available in 12 Colors Description Items Included L-Shaped Desk Transform your office into a center of organization and productivity with the Bestar Equinox L-Shaped Office Desk. With a layered work surface, this L-shaped desk provides ample room for all of your work activities. The laminated particle board desktop resists scratches, stains, and wear to last you for many years. With 2 closed and 2 open storage areas and a total of 3 adjustable shelves, the desk offers enough storage solutions for your office accessories. Dimensions Overall Dimensions: 71.5W x 71.1D x 29.7H Specifications Easily keep your files and folders organized thanks to the utility drawer and letter/legal file drawer. The grommet can help you efficiently manage your cables. Keep your phone charged and on hand thanks to the built-in USB/power socket on the desktop. Meets ANSI/BIFMA standards for safety and performance. This Bundle is backed by the 5 Year Manufacturer's Warranty. North American made with local and imported parts.",
  {'entities': [(0, 28, 'PRODUCT'), (171, 207, 'PRODUCT')]}),
 ('Pro-Concept Plus Desk Hutch with Frosted Glass Doors Available in 2 Colors Description Items Included Desk Hutch The Pro-Concept Plus collection offers practical and stylish furniture suitable for any type of office. This furniture is perfect for anyone who wants variety: choose between an L-shaped desk, a U-shaped desk, or a standing desk! This collection also features a keyboard tray, a hutch, a desk shell, a desk bridge, and an assembled pedestal. Arrange the pieces to your liking and create an organized and contemporary office space that reflects your business. Dimensions Overall Dimensions: 71.1W x 12.4D x 40.4H Specifications GOOD QUALITY: Made from high-density commercial-grade particle board compliant with TSCA Title VI. Durable laminate finish. PERFECT ADD-ON: This hutch is the perfect add-on if you want more storage space on your desk or credenza. Matching colors are available in the same collection. GOOD STORAGE: Features 1 wide and 2 small storage compartments behind the frosted glass doors. Accommodates letter size binders. 10-YEAR LIMITED WARRANTY: Warranted to the original owner against defects in materials or workmanship. Assembly required (2 people). MADE IN CANADA: Excellent customer service. Meets or exceeds ANSI/BIFMA performance standards.',
  {'entities': [(0, 52, 'PRODUCT')]}),
 ('Viva L-Shaped Standing Desk Available in 2 Colors Save 29% Original price $1,899.99 Current price $1,357.99 Description Items Included L-Shaped Standing Desk With their high quality commercial-grade work surface, our L-shaped desks are designed for real-world use because they resist stains, scratches and premature wear. You can make extensive use of your desk and still keep its stylish look. Dimensions Overall Dimensions: 71.1W x 68.8D x 28H 45H Specifications Our L-shaped desks are made of high-density particle board panels. Because of their density, the panels we use are rigid and unlikely to sag or warp. The panels are made from recycled wood and they comply with California Air Resources Board (CARB) Phase 2, the highest standards in North America regarding formaldehyde emissions.',
  {'entities': [(0, 27, 'PRODUCT'),
    (135, 157, 'PRODUCT'),
    (469, 483, 'PRODUCT')]}),
 ('Universel Under-Desk Wireless Charger Black Save 29% Original price $179.99 Current price $126.99 Description Items Included Under-Desk Wireless Charger Transform you desk, credenza, or coffee table into a charging station. Install this 5 W wireless charger under any table with a wood, stone, glass, or ceramic top (no metal). that is 0. 8ï¿½ï¿½ to 1. 4ï¿½ï¿½ thick. Specifications ADAPTABLE: Transform you desk, credenza, or coffee table into a charging station. Install this 5 W wireless charger under any table with a wood, stone, glass, or ceramic top (no metal). that is 0. 8ï¿½ï¿½ to 1. 4ï¿½ï¿½ thick. DISCREET: When placed under a desk or table surface, the small emitter (5. 9ï¿½ï¿½ L x 5. 9ï¿½ï¿½W). is barely noticeable. Self-adhesive back. HANDY: Simply place your phone with compatible wireless charging capability on the surface, above the charger. No wire messes. Overcharge protection. EASY INSTALLATION: Upgrade your desk or any other table with a wireless charger in 5 minutes: use the installation template and magnets to position the emitter and receiver on either side of the desk or table top, then simply stick down the parts.',
  {'entities': [(0, 37, 'PRODUCT'), (125, 152, 'PRODUCT')]}),
 ('Universel LED Desk Lamp with Wireless Charger White Description Items Included Desk Lamp This dual purpose lamp and charger in white and silver can be used on your desk, in the bedroom, in the living room or anywhere you need additional lighting. 16ï¿½ H, made of aluminum alloy and PC-ABS, diameter of base is 5. 7ï¿½. WIRELESS AND USB CHARGERS: A 5 W wireless charger that can be used with compatible phones is integrated into the lamp. Specifications VERSATILE: This dual purpose lamp and charger in white and silver can be used on your desk, in the bedroom, in the living room or anywhere you need additional lighting. 16ï¿½ H, made of aluminum alloy and PC-ABS, diameter of base is 5. 7ï¿½. WIRELESS AND USB CHARGERS: A 5 W wireless charger that can be used with compatible phones is integrated into the lamp. A USB and micro USB charging port can be used with all phones. 2 phones can be charged at once. HANDY: Simply place your phone with wireless charging capability on the lamp to charge it. An induction touch switch lets you turn the light on and off quietly. The angle of the lamp can be adjusted to suit your needs. WHITE FLICKER-FREE LIGHT: The lightï¿½s color temperature can be adjusted from 2700K to 6500K. This span covers warm or soft white light ideal for working on the computer to cool white light perfect for reading to natural light. Flicker free. UNLIMITED DIMMER: Find the perfect brightness for your activities thanks to the unlimited dimming levels. 12 V, 1. 5 A DC input.',
  {'entities': [(0, 45, 'PRODUCT')]}),
 ('Embassy Traditional Executive Desk with 2 Pedestals Dark Chocolate Description Items Included Executive Desk with 2 Pedestals The Embassy collection offers a traditional desk style with a twist for your home office or business office. With this unique collection, you can choose to have a regular desk or a standing desk which allows you to go from sitting to standing in no time. Both options come with multiple storage spaces, allowing you to keep your office neat and organized. This collection is well-designed and affordable, making it the perfect addition to your office space. Dimensions Overall Dimensions: 66W x 29.8D x 30.4H Specifications GOOD QUALITY: Made from high-density commercial-grade particle board compliant with TSCA Title VI. Durable laminate finish. Desktop is 1ï¿½ï¿½ thick. Features impact resistant PVC edge, full modesty panel, and classic moldings. SPACE SAVER: This compact, smaller desk is ideal for every type of workplace, including the home office. For efficient wire management, 4 grommets are available on the desk. Features keyboard tray on  extension ball-bearing slides. GOOD STORAGE: Pedestals each offer 2 utility drawers and 1 file drawer with a letter/legal filing system. One lock secures bottom two drawers. Drawers on full-extension ball-bearing slides. 10-YEAR LIMITED WARRANTY: Our desk is warranted to the original owner against defects in materials or workmanship. Surface resists scratches, stains and wear. Assembly required (2 people). MADE IN CANADA: Excellent customer support. The desk shell meets or exceeds ANSI/BIFMA performance standards.',
  {'entities': [(0, 51, 'PRODUCT')]}),
 ('Embassy U-Shaped Executive Desk with Pedestal and Hutch Dark Chocolate Description Items Included U-Shaped Desk with Pedestal and Hutch The Embassy collection offers a traditional desk style with a twist for your home office or business office. With this unique collection, you can choose to have a regular desk or a standing desk which allows you to go from sitting to standing in no time. Both options come with multiple storage spaces, allowing you to keep your office neat and organized. This collection is well-designed and affordable, making it the perfect addition to your office space. Dimensions Overall Dimensions: 66W x 91.7D x 66.8H',
  {'entities': [(0, 55, 'PRODUCT'), (98, 135, 'PRODUCT')]}),
 ('Aquarius 3-Piece Set Including a Desk with Single Pedestal and 2 Storage Units with 8 Cubbies Available in 4 Colors Description Items Included Desk Pedestal 2 Storage Units The Aquarius collection provides solutions for organizing your office in style. With their clean lines and metal accents, the contemporary pieces will help you get that wow effect to impress clients and employees alike. In addition, the desksï¿½ and office storage unitsï¿½ highly functional design will contribute to a more productive and better organized office environment. Dimensions Desk: 66ï¿½ï¿½W x 29.5ï¿½ï¿½D x 29.7ï¿½ï¿½H Storage Units: 35.4ï¿½ï¿½W x 9.9ï¿½ï¿½D x 60.8ï¿½ï¿½H Specifications GOOD QUALITY: Made from high-density commercial-grade particle board compliant with TSCA Title VI. Durable laminate finish. Desktop is 1ï¿½ï¿½ thick. U-shaped steel leg and steel base on desk. Leg levelers on all pieces. MODERN DESIGN: Make your office look good with this asymmetric desk featuring aluminum handles and a layered desktop. The narrow (9. 9ï¿½ï¿½) storage unit is ideal for saving space in tight areas. Finished back on all pieces. GOOD STORAGE: Desk: 1 utility drawer and 1 file drawer with letter/legal filing system. Storage Units: Interior divided vertically into 2 sections (16. 7ï¿½ï¿½W). with 4 cubbies and 3 adjustable shelves each. 5-YEAR LIMITED WARRANTY: Warranted to the original owner against defects in materials or workmanship. Desk surface resists scratches, stains and wear. Assembly required (2 people). MADE IN CANADA: Excellent customer support. The set meets or exceeds ANSI/BIFMA performance standards. Drawers on full-extension ball-bearing slides.',
  {'entities': [(0, 20, 'PRODUCT')]}),
 ('Equinox 3-Piece Set Including 1 L-Shaped Desk and 2 Storage Units with 8 Cubbies Available in 2 Colors Description Items Included L-Shaped Desk 2 Storage Units The Equinox collection is perfect if you are looking to maximize productivity and space in your office. The functional pieces from this collection feature multiple storage options so you can keep all the essentials on hand and boost your productivity. Our L-Shaped desk features a layered work surface to provide ample room for all your work activities. Pair it with our stylish storage units to create a chic and sophisticated office environment. Dimensions L-Shaped Desk: 71.1ï¿½ï¿½W x 56ï¿½ï¿½D x 29.7ï¿½ï¿½H Storage Units: 35.4ï¿½ï¿½W x 9.9ï¿½ï¿½D x 60.8ï¿½ï¿½H Specifications GOOD QUALITY: Made from high-density commercial-grade particle board compliant with TSCA Title VI. Durable laminate finish. Desktop is 1ï¿½ï¿½ thick. Leg levelers. Aluminum handles. Finished backs. Desk is fully reversible. HIGHLY FUNCTIONAL: Drawers on full-extension ball-bearing slides. Desk includes 1 grommet for wire management and 1 dual power/USB module. Reversible sliding door on storage unit conceals part of cabinet to hide clutter. GOOD STORAGE: Desk: 1 closed storage area with 1 adjustable shelf, 1 open storage area, 1 utility drawer, and 1 file drawer with letter/legal filing system. Storage Units: Interior divided vertically into 2 sections with 4 cubbies each. 5-YEAR LIMITED WARRANTY: Warranted to the original owner against defects in materials or workmanship. Surface resists scratches, stains and wear. Assembly required (2 people). MADE IN CANADA: Excellent customer support. Meets or exceeds ANSI/BIFMA performance standards. Each section of the storage units features 3 adjustable shelves so you can customize the cubbies.',
  {'entities': [(0, 19, 'PRODUCT')]}),
 ('Embassy U-Shaped Executive Desk with Pedestal and Hutch Walnut Gray Description Items Included U-Shaped Desk with Pedestal and Hutch The Embassy collection offers a traditional desk style with a twist for your home office or business office. With this unique collection, you can choose to have a regular desk or a standing desk which allows you to go from sitting to standing in no time. Both options come with multiple storage spaces, allowing you to keep your office neat and organized. This collection is well-designed and affordable, making it the perfect addition to your office space. Dimensions Overall Dimensions: 66W x 91.7D x 66.8H',
  {'entities': [(0, 55, 'PRODUCT'), (95, 132, 'PRODUCT')]}),
 ('i3 Plus L-Shaped Desk Available in 2 Colors Description Items Included L-Shaped Desk The I3 Plus collection is ideal for any office space, be it a home office or a workplace office. This collection can easily be arranged to fit perfectly into your space thanks to its multiple configuration options such as additional hutches, storage units, file cabinets, credenzas, and even the option of a standing desk. More options include metal accents which make these desks not only practical but contemporary and stylish as well. Update and organize your space with this affordable office collection. Dimensions Overall Dimensions: 60.2W x 59.3D x 56.8H Specifications GOOD QUALITY: Made from high-density commercial-grade particle board compliant with TSCA Title VI. Durable laminate finish. Desktop is 1ï¿½ï¿½ thick. Square metal legs in matte silver finish. Sturdy construction. SPACE SAVER: Compact and designed for small spaces. GOOD STORAGE: The desk features 1 file drawer with a letter/legal filing system and 5 open storage compartments. The drawer is lockable and on full-extension ball-bearing slides for smooth operation. 10-YEAR LIMITED WARRANTY: Our desk is warranted to the original owner against defects in materials or workmanship. Surface resists scratches, stains and wear. MADE IN CANADA: Excellent customer support. Assembly required (2 people). Fully reversible unit. Meets or exceeds ANSI/BIFMA performance standards.',
  {'entities': [(0, 21, 'PRODUCT'), (71, 84, 'PRODUCT')]}),
 ('Prestige+ 2-Piece Set Including a Standing Desk and a Desk with Hutch Available in 3 Colors Description Items Included Standing Desk Desk with Hutch Prestige+ is a timeless modular collection that will adapt to your needs, over time. With its wide range of products, it is ideal for every type of office. The desk is made of a durable 1ï¿½ commercial grade work surface with melamine finish that resist scratches, stains and wear. The electric mechanism allows you to quickly and quietly adjust the height of the table from 28ï¿½ to 45ï¿½. The control button is located on the front edge of the work surface for an easy access. Voltage : 110V. The L-Desk features an impact resistant PVC edge, classic moldings and a modesty panel on the credenza. Grommet is available on the credenza for efficient wire management. The pedestal offers two utility drawers and one file drawer with letter/legal filing system. One lock secures bottom two drawers. The drawers are on ball-bearing slides for a smooth and quiet operation. The hutch for credenza offers large closed storage space, efficient wire management and side moldings. The opening has 11ï¿½ï¿½ 3/4 high which is ideal for letter format binders. The doors are fitted with strong adjustable hinges. The credenza, the hutch and the height adjustable table meet or exceed ANSI/BIFMA performance standards. The station is fully reversible. Dimensions Height Adjustable Table: 47.6ï¿½W x 24ï¿½D x 28ï¿½ to 45ï¿½H Credenza: 71.1ï¿½W x 22.4ï¿½D x 30.4ï¿½H Overall Dimensions: 71.1ï¿½W x 70.9ï¿½D x 66.9ï¿½H',
  {'entities': [(0, 21, 'PRODUCT')]}),
 ('Prestige+ Modern L-Shaped Office Desk with Two Pedestals and Hutch Available in 3 Colors Description Items Included L-Shaped Desk with 2 Pedestals and Hutch The Prestige Plus collection is ideal for an office with a more traditional style. Give your office a classic look with darker colors. Make your space complete by adding another piece from the collection such as a bookcase, a regular or mobile pedestal, a storage cabinet, or a lateral file cabinet. These options allow you to shape your space to your liking and to show just how competent and serious you are. Dimensions Overall Dimensions: 71.1W x 62.8D x 66.8H Specifications GOOD QUALITY: Our L-shaped desk is made from high-density commercial-grade particle board compliant with TSCA Title VI. Durable laminate finish. Desktop is 1ï¿½ï¿½ thick. Impact resistant PVC edges, a full modesty panel and classic moldings. GOOD STORAGE: Both pedestals offer 2 utility drawers and 1 file drawer with a letter/legal filing system. One lock secures bottom two drawers. The hutch offers 4 closed storage compartments (11ï¿½ï¿½  high). Includes a keyboard tray. HIGHLY FUNCTIONAL: Provides an ample work surface, yet thanks to its shape, everything is within reach. Drawers on full-extension ball-bearing slides and keyboard tray on  extension ball-bearing slides. Includes 3 grommets. 10-YEAR LIMITED WARRANTY: Our modern L-shaped office desk is warranted to the original owner against defects in materials or workmanship. Surface resists scratches, stains and wear. Assembly required (2 people). MADE IN CANADA: Excellent customer support. Workstation is fully reversible. The hutch, the credenza and the return table meet or exceed ANSI/BIFMA performance standards.',
  {'entities': [(0, 66, 'PRODUCT'),
    (116, 156, 'PRODUCT'),
    (654, 667, 'PRODUCT'),
    (1365, 1385, 'PRODUCT')]}),
 ('Prestige+ 2-Piece Set Including a Standing Desk and a Desk Available in 3 Colors Description Items Included Standing Desk Desk Prestige+ is a timeless modular collection that will adapt to your needs, over time. With its wide range of products, it is ideal for every type of office. The desk is made of a durable 1ï¿½ commercial grade work surface with melamine finish that resist scratches, stains and wear. The electric mechanism allows you to quickly and quietly adjust the height of the table from 28ï¿½ to 45ï¿½. The control button is located on the front edge of the work surface for an easy access. Voltage : 110V. The L-Desk features an impact resistant PVC edge, classic moldings and a modesty panel on the credenza. Grommet is available on the credenza for efficient wire management. The pedestal offers two utility drawers and one file drawer with letter/legal filing system. One lock secures bottom two drawers. The drawers are on ball-bearing slides for a smooth and quiet operation. The credenza and the height adjustable table meet or exceed ANSI/BIFMA performance standards. The station is fully reversible. Also available in Chocolate. Dimensions Height Adjustable Table: 47.6ï¿½W x 24ï¿½D x 28ï¿½ to 45ï¿½H Credenza: 71.1ï¿½D x 22.4ï¿½D x 30.4ï¿½H Overall Dimensions: 71.1ï¿½W x 70.9ï¿½D',
  {'entities': [(0, 21, 'PRODUCT')]}),
 ("Prestige+ 72W L-Shaped Desk with Pedestal and Hutch Available in 3 Colors Description Items Included L-Shaped Desk The work surface provides sturdiness while the modesty panel and classic moldings bring elegance to any office. The keyboard tray helps you work from a comfortable position. Thanks to the CPU stand and wire management solutions, it is easy to keep your desktop organized. The utility drawers and the file drawer are perfect for storing supplies and sensitive documents. The lock secures the two bottom drawers to keep your files safe. The closed storage areas accommodate items of various sizes, including letter size binders. The hutch, the credenza, and the return table are tested to meet ANSI/BIFMA quality standards for safety and performance. This L-Shaped Desk is backed by the 10 Year Manufacturer's Warranty. North American made with local and imported parts. Dimensions Overall Dimensions: 71.1W x 62.8D x 66.8H",
  {'entities': [(0, 51, 'PRODUCT'),
    (101, 114, 'PRODUCT'),
    (769, 782, 'PRODUCT')]}),
 ('Prestige+ U-Shaped Executive Desk with Pedestal Available in 3 Colors Description Items Included U-Shaped Desk with Pedestal The Prestige Plus collection is ideal for an office with a more traditional style. Give your office a classic look with darker colors. Make your space complete by adding another piece from the collection such as a bookcase, a regular or mobile pedestal, a storage cabinet, or a lateral file cabinet. These options allow you to shape your space to your liking and to show just how competent and serious you are. Dimensions Overall Dimensions: 71.1W x 92.6D x 30.4H',
  {'entities': [(0, 47, 'PRODUCT'), (97, 124, 'PRODUCT')]}),
 ('Prestige+ U-Shaped Executive Desk with Hutch and 2 Pedestals Available in 3 Colors Description Items Included U-Shaped Desk with 2 Pedestals The Prestige Plus collection is ideal for an office with a more traditional style. Give your office a classic look with darker colors. Make your space complete by adding another piece from the collection such as a bookcase, a regular or mobile pedestal, a storage cabinet, or a lateral file cabinet. These options allow you to shape your space to your liking and to show just how competent and serious you are. Dimensions Overall Dimensions: 71.1W x 92.6D x 66.8H',
  {'entities': [(0, 60, 'PRODUCT'), (110, 140, 'PRODUCT')]}),
 ('Prestige+ Desk Bridge Available in 3 Colors Description Items Included Desk Bridge The Prestige Plus collection is ideal for an office with a more traditional style. Give your office a classic look with darker colors. Make your space complete by adding another piece from the collection such as a bookcase, a regular or mobile pedestal, a storage cabinet, or a lateral file cabinet. These options allow you to shape your space to your liking and to show just how competent and serious you are. Dimensions Overall Dimensions: 39.4W x 19.8D x 30.4H Specifications GOOD QUALITY: Made from high-density commercial-grade particle board compliant with TSCA Title VI. Durable laminate finish. Top is 1ï¿½ï¿½ thick with a deluxe PVC edge. PERFECT ADD-ON: This desk bridge is designed as an add-on to connect 2 desks from the Prestige + collection and form a U-Shaped desk (product numbers: 99400, 99612). Matching colors available in the same collection. PRACTICAL: This desk bridge is the perfect solution to increase your work surface. Comes with 1 grommet for efficient wire management. 10-YEAR LIMITED WARRANTY: Warranted to the original owner against defects in materials or workmanship. Surface resists scratches, stains, and wear. Assembly required (2 people). MADE IN CANADA: Excellent customer service. Meets or exceeds ANSI/BIFMA performance standards.',
  {'entities': [(0, 21, 'PRODUCT'),
    (71, 82, 'PRODUCT'),
    (748, 759, 'PRODUCT')]}),
 ('Prestige+ Credenza Desk with 2 Pedestals and Hutch Bordeaux & Graphite Description Items Included Credenza Desk with 2 Pedestal and Hutch The Prestige Plus collection is ideal for an office with a more traditional style. Give your office a classic look with darker colors. Make your space complete by adding another piece from the collection such as a bookcase, a regular or mobile pedestal, a storage cabinet, or a lateral file cabinet. These options allow you to shape your space to your liking and to show just how competent and serious you are. Dimensions Overall Dimensions: 71.1W x 23.4D x 66.8H Specifications GOOD QUALITY: Made from high-density commercial-grade particle board compliant with TSCA Title VI. Durable laminate finish. Desktop is 1ï¿½ï¿½ thick. Features PVC edge, modesty panel, and classic moldings. Sturdy construction. HIGHLY FUNCTIONAL: Provides an ample work surface, yet everything is within reach. Drawers on full-extension ball-bearing slides for smooth operation. A rubber strip and 2 grommets for wire management. GOOD STORAGE: Pedestals each offer 2 utility drawers and 1 file drawer with letter/legal filing systems. One lock secures bottom two drawers. Hutch offers 4 closed storage areas (11ï¿½ï¿½  high). behind doors. 10-YEAR LIMITED WARRANTY: Our desk is warranted to the original owner against defects in materials or workmanship. Surface resists scratches, stains and wear. Assembly required (2 people). MADE IN CANADA: Excellent customer support. Doors are fitted with strong adjustable hinges. The hutch and the desk shell meet or exceed ANSI/BIFMA performance standards.',
  {'entities': [(0, 50, 'PRODUCT'), (98, 137, 'PRODUCT')]}),
 ('Connexion 2-Piece Set Including a Standing Desk and a Desk with Hutch Available in 3 Colors Description Items Included Standing Desk Desk with Hutch Connexion is a contemporary and durable collection that features a wide variety of configuration options that will adapt to your specific needs. The desk is made of a durable 1ï¿½ commercial grade work surface with melamine finish that resist scratches, stains and wear. The electric mechanism allows you to quickly and quietly adjust the height of the table from 28ï¿½ to 45ï¿½. The control button is located on the front edge of the work surface for an easy access. Voltage : 110V. The L-Desk features an impact resistant PVC edge, classic moldings and a modesty panel on the credenza. Grommet is available on the credenza for efficient wire management. The pedestal offers two utility drawers and one file drawer with letter/legal filing system. Drawers on ball-bearing slides for smooth and quiet operation. One lock secures bottom two drawers. The hutch for credenza offers large closed storage space, efficient wire management and two large paper shelves. The doors are fitted with strong lift up hardware. The large flip up doors bring a modern and original touch. The credenza, the hutch and the height adjustable table meet or exceed ANSI/BIFMA performance standards. The station is fully reversible. Also available in Bordeaux and Slate finish. Dimensions Height Adjustable Table: 47.6ï¿½W x 24ï¿½D x 28ï¿½ to 45ï¿½H Credenza: 71.1ï¿½W x 22.4ï¿½D x 30.4ï¿½H Overall Dimensions: 71.1ï¿½W x 70.9ï¿½D x 65.9ï¿½H',
  {'entities': [(0, 69, 'PRODUCT'), (119, 158, 'PRODUCT')]}),
 ('Connexion 2-Piece Set Including a Standing Desk and a Desk Available in 3 Colors Description Items Included Standing Desk Desk Connexion is a contemporary and durable collection that features a wide variety of configuration options that will adapt to your specific needs. The desk is made of a durable 1ï¿½ commercial grade work surface with melamine finish that resist scratches, stains and wear. The electric mechanism allows you to quickly and quietly adjust the height of the table from 28ï¿½ to 45ï¿½. The control button is located on the front edge of the work surface for an easy access. Voltage : 110V. The L-Desk features an impact resistant PVC edge, classic moldings and a modesty panel on the credenza. Grommet is available on the credenza for efficient wire management. The pedestal offers two utility drawers and one file drawer with letter/legal filing system. Drawers on ball-bearing slides for smooth and quiet operation. One lock secures bottom two drawers. The credenza and the height adjustable table meet or exceed ANSI/BIFMA performance standards. The station is fully reversible. Also available in Slate and Sandstone finish. Dimensions Height Adjustable Table: 47.6ï¿½W x 24ï¿½D x 28ï¿½ to 45ï¿½H Credenza: 71.1ï¿½W x 22.4ï¿½D x 30.4ï¿½H Overall Dimensions: 71.1ï¿½W x 70.9ï¿½D x 30.4ï¿½H',
  {'entities': [(0, 21, 'PRODUCT')]}),
 ('Connexion 2-Piece Set Including an L-Shaped Desk and a Lateral File Cabinet Available in 3 Colors Description Items Included L-Shaped Desk Lateral File Cabinet Connexion is a contemporary and durable collection that features a wide variety of configuration options that will adapt to your specific needs. The desk is made of durable 1ï¿½ commercial grade work surface with melamine finish that resist scratches, stains and wear. It features an impact resistant 0.25 cm PVC edge. Grommets are available on the station for efficient wire management. The credenza and the return table meet or exceed ANSI/BIFMA performance standards. The pedestal offers two utility drawers and one file drawer with letter/legal filing system. One lock secures bottom two drawers. The lateral file offers two file drawers with letter/legal filing system. The drawers are on ball-bearing slides for a smooth and quiet operation. The station is fully reversible. Also available in Slate and Sandstone finish. Dimensions L-Shaped Desk: 71.1ï¿½W x 82.9ï¿½D x 30.4ï¿½H Lateral File Cabinet: 30.8ï¿½W x 19.6ï¿½D x 30.4ï¿½H',
  {'entities': [(0, 21, 'PRODUCT')]}),
 ('Connexion U-Shaped Executive Desk with Pedestal and Hutch Available in 3 Colors Description Items Included U-Shaped Desk with Pedestal and Hutch The Connexion collection is made for people who like to have options. Choose from an L-shaped or U-shaped desk, a credenza desk, or even a standing desk. You also have the option of adding a keyboard tray, a lateral file cabinet, or a hutch. This Bestar collection offers more than enough options that will suit your needs, help you stay organized, and help you stay on top of your work! Dimensions Overall Dimensions: 71.1W x 92.6D x 65.9H',
  {'entities': [(0, 57, 'PRODUCT'), (107, 144, 'PRODUCT')]}),
 ('Connexion L-Shaped Desk with Lateral File Cabinet and Hutch Available in 3 Colors Description Items Included L-Shaped Desk Lateral File Cabinet Hutch The Connexion office collection is made for people who like to have options. Choose from an L-shaped or U-shaped desk, a credenza desk, or even a standing desk. You also have the option of adding a keyboard tray, a lateral file cabinet, or a hutch. This Bestar collection offers more than enough options that will suit your needs, help you stay organized, and help you stay on top of your work! Dimensions Overall Dimensions: 71.1W x 82.9D x 65.9H Specifications GOOD QUALITY: Made from high-density commercial-grade particle board compliant with TSCA Title VI. Durable laminate finish. Desktop is 1ï¿½ï¿½ thick. Impact resistant PVC edges. Sturdy construction. Finished back. HIGHLY FUNCTIONAL: Features a keyboard tray to maximize workspace. Includes 2 grommets and a rubber strip for wire management. The drawers are on full-extension ball-bearing slides. GOOD STORAGE: The hutch offers 2 large closed storage areas with 2 flip up doors, efficient wire management, and 2 large paper shelves. The lateral file cabinet offers 2 file drawers with letter/legal filing systems. 10-YEAR LIMITED WARRANTY: Our desk is warranted to the original owner against defects in materials or workmanship. Surface resists scratches, stains and wear. Assembly required (2 people). MADE IN CANADA: Excellent customer support. The workstation is fully reversible. The credenza, the hutch, and the return table meet or exceed ANSI/BIFMA performance standards.',
  {'entities': [(0, 59, 'PRODUCT'), (109, 149, 'PRODUCT')]}),
 ('Connexion U-Shaped Executive Desk with Lateral File Cabinet Available in 3 Colors Description Items Included U-Shaped Desk Lateral File Cabinet The Connexion collection is made for people who like to have options. Choose from an L-shaped or U-shaped desk, a credenza desk, or even a standing desk. You also have the option of adding a keyboard tray, a lateral file cabinet, or a hutch. This Bestar collection offers more than enough options that will suit your needs, help you stay organized, and help you stay on top of your work! Dimensions Overall Dimensions: 71.1W x 92.6D x 30.4H',
  {'entities': [(0, 59, 'PRODUCT'), (109, 143, 'PRODUCT')]}),
 ('Connexion U-Shaped Executive Desk with Lateral File Cabinet and Hutch Available in 3 Colors Description Items Included U-Shaped Desk Lateral File Cabinet Hutch The Connexion collection is made for people who like to have options. Choose from an L-shaped or U-shaped desk, a credenza desk, or even a standing desk. You also have the option of adding a keyboard tray, a lateral file cabinet, or a hutch. This Bestar collection offers more than enough options that will suit your needs, help you stay organized, and help you stay on top of your work! Dimensions Overall Dimensions: 71.1W x 92.6D x 65.9H',
  {'entities': [(0, 69, 'PRODUCT'), (119, 159, 'PRODUCT')]}),
 ('Connexion L-Shaped Desk with Lateral File Cabinet Available in 3 Colors Description Items Included L-Shaped Desk Lateral File Cabinet The Connexion office collection is made for people who like to have options. Choose from an L-shaped or U-shaped desk, a credenza desk, or even a standing desk. You also have the option of adding a keyboard tray, a lateral file cabinet, or a hutch. This Bestar collection offers more than enough options that will suit your needs, help you stay organized, and help you stay on top of your work! Dimensions Overall Dimensions: 71.1W x 82.9D x 30.4H',
  {'entities': [(0, 49, 'PRODUCT'), (99, 133, 'PRODUCT')]}),
 ('Connexion Credenza Desk with 2 Pedestals and Hutch Available in 2 Colors Description Items Included Credenza Desk with 2 Pedestals Hutch The Connexion office collection is made for people who like to have options. Choose from an L-shaped or U-shaped desk, a credenza desk, or even a standing desk. You also have the option of adding a keyboard tray, a lateral file cabinet, or a hutch. This Bestar collection offers more than enough options that will suit your needs, help you stay organized, and help you stay on top of your work! Dimensions Overall Dimensions: 71.1W x 23.4D x 65.9H Specifications GOOD QUALITY: Made from high-density commercial-grade particle board compliant with TSCA Title VI. Durable laminate finish. Desktop is 1ï¿½ï¿½ thick. Features impact resistant PVC edges. Sturdy construction. HIGHLY FUNCTIONAL: Drawers on full-extension ball-bearing slides and keyboard tray on  extension ball-bearing slides for smooth operation. A rubber strip and 1 grommet available for efficient wire management. GOOD STORAGE: Pedestals each offer 2 utility drawers and 1 file drawer with letter/legal filing systems. One lock secures bottom two drawers. Hutch features 2 large closed storage areas behind flip-up doors, and 2 large paper shelves. 10-YEAR LIMITED WARRANTY: Our desk is warranted to the original owner against defects in materials or workmanship. Surface resists scratches, stains and wear. Assembly required (2 people). MADE IN CANADA: Excellent customer support. Hutch doors are fitted with strong lift up hardware. Finished back. The credenza and the hutch meet or exceed ANSI/BIFMA performance standards.',
  {'entities': [(0, 50, 'PRODUCT'), (100, 136, 'PRODUCT')]}),
 ('Connexion L-Shaped Desk with Pedestal and Hutch Available in 3 Colors Description Items Included L-Shaped Desk with Pedestal Hutch The Connexion office collection is made for people who like to have options. Choose from an L-shaped or U-shaped desk, a credenza desk, or even a standing desk. You also have the option of adding a keyboard tray, a lateral file cabinet, or a hutch. This Bestar collection offers more than enough options that will suit your needs, help you stay organized, and help you stay on top of your work! Dimensions Overall Dimensions: 71.1W x 82.9D x 65.9H',
  {'entities': [(0, 47, 'PRODUCT'), (97, 130, 'PRODUCT')]}),
 ('Connexion Executive Desk Available in 3 Colors Description Items Included Executive Desk The Connexion office collection is made for people who like to have options. Choose from an L-shaped or U-shaped desk, a credenza desk, or even a standing desk. You also have the option of adding a keyboard tray, a lateral file cabinet, or a hutch. This Bestar collection offers more than enough options that will suit your needs, help you stay organized, and help you stay on top of your work! Dimensions Overall Dimensions: 71.1W x 29.8D x 30.4H Specifications GOOD QUALITY: Made in North America from high-density commercial-grade particle board compliant with TSCA Title VI. Durable laminate finish. Desktop is 1ï¿½ï¿½ thick. Features full modesty panel and impact resistant PVC edge. Finished back. SPACE SAVER: This compact, executive desk is ideal for every type of workplace, including the home office. For efficient wire management, 4 grommets are available on the desk. GOOD STORAGE: Each pedestal has 2 utility drawers and 1 file drawer with letter/legal filing system, all on full-extension ball-bearing slides. One lock secures bottom two drawers. Keyboard tray on  extension ball-bearing slides. 10-YEAR LIMITED WARRANTY: Our executive desk is warranted to the original owner against defects in materials or workmanship. Surface resists scratches, stains and wear. Assembly required (2 people). MADE IN CANADA: Excellent customer support. The desk shell meets or exceeds ANSI/BIFMA performance standards.',
  {'entities': [(0, 24, 'PRODUCT'), (74, 88, 'PRODUCT')]}),
 ('Connexion Desk Bridge Available in 3 Colors Description Items Included Desk Bridge The Connexion office collection is made for people who like to have options. Choose from an L-shaped or U-shaped desk, a credenza desk, or even a standing desk. You also have the option of adding a keyboard tray, a lateral file cabinet, or a hutch. This Bestar collection offers more than enough options that will suit your needs, help you stay organized, and help you stay on top of your work! Dimensions Overall Dimensions: 39.4W x 19.7D x 30.4H Specifications GOOD QUALITY: Made from high-density commercial-grade particle board compliant with TSCA Title VI. Durable laminate finish. Top is 1ï¿½ï¿½ thick with a deluxe PVC edge. PERFECT ADD-ON: This desk bridge is designed as an add-on to connect 2 desks from the Connexion collection and form a U-Shaped desk (product numbers: 93440, 93612). Matching colors available in the same collection. PRACTICAL: This desk bridge is the perfect solution to increase your work surface. Comes with 1 grommet for efficient wire management. 10-YEAR LIMITED WARRANTY: Warranted to the original owner against defects in materials or workmanship. Surface resists scratches, stains, and wear. Assembly required (2 people). MADE IN CANADA: Excellent customer service. Meets or exceeds ANSI/BIFMA performance standards.',
  {'entities': [(0, 21, 'PRODUCT'),
    (71, 82, 'PRODUCT'),
    (732, 743, 'PRODUCT')]}),
 ('Connexion Add-On Desk Pedestal with 3 Drawers Available in 3 Colors Description Items Included Desk Pedestal The Connexion office collection is made for people who like to have options. Choose from an L-shaped or U-shaped desk, a credenza desk, or even a standing desk. You also have the option of adding a keyboard tray, a lateral file cabinet, or a hutch. This Bestar collection offers more than enough options that will suit your needs, help you stay organized, and help you stay on top of your work! Dimensions Overall Dimensions: 15.4W x 17.7D x 29.4H Specifications GOOD QUALITY: Made from high-density commercial-grade particle board compliant with TSCA Title VI. Durable laminate finish. Leg levelers. PERFECT ADD-ON: This pedestal is designed as an add-on for a desk from the Connexion collection. Matching colors available in the same collection. GOOD STORAGE: Features 2 utility drawers and 1 file drawer with letter/legal filing system. Drawers on ball-bearing slides. One lock secures bottom two drawers. 10-YEAR LIMITED WARRANTY: Warranted to the original owner against defects in materials or workmanship. Surface resists scratches, stains, and wear. Assembly required (2 people). MADE IN CANADA: Excellent customer service.',
  {'entities': [(0, 45, 'PRODUCT'), (95, 108, 'PRODUCT')]}),
 ('Innova U-Shaped or L-Shaped Desk with Hutch Available in 2 Colors Description Items Included U-Shaped Desk with Hutch The Innova collection is perfect for keeping your office space clean and organized. The multiple storage options will help you stay clutter-free. If you still find yourself searching for more storage solutions, try pairing a desk with an Innova bookshelf. Your office space will feel cohesive and stylish with the pieces from this collection. Specifications GOOD QUALITY: Made from high-density commercial-grade particle board compliant with TSCA Title VI. Durable laminate finish. Desktops are 1ï¿½ï¿½ thick. Finished back. The desk shell features leg levelers. HIGHLY FUNCTIONAL: Can be set up as a U-shaped desk or as 2 desks: an L-Shaped Desk and a Desk Shell. File drawer on full-extension ball-bearing slides and keyboard tray on  extension ball-bearing slides. Rubber strip for wire management. GOOD STORAGE: The pedestal offers 2 utility drawers and 1 file drawer with a letter/legal filing system. One lock secures bottom two drawers. The hutch offers 4 closed storage compartments behind doors and 4 paper shelves. 5-YEAR LIMITED WARRANTY: Warranted to the original owner against defects in materials or workmanship. Surfaces resist scratches, stains and wear. Assembly required (2 people). MADE IN NORTH AMERICA: Excellent customer support. Desk is fully reversible. Desk shell meets or exceeds ANSI/BIFMA performance standards.',
  {'entities': [(0, 43, 'PRODUCT'), (93, 117, 'PRODUCT')]}),
 ('Innova U-Shaped or L-Shaped Desk with Pedestal and Hutch, 1 Lateral File Cabinet, and 1 Bookcase Available in 2 Colors Save 29% Original price $2,159.99 Current price $1,539.99 Description Items Included Desk with Pedestal and Hutch Lateral File Cabinet Bookcase This bundle is perfect for anyone who wants to enhance their office. With modern designs and clean lines, this set is sure to give your office a sophisticated feel. You can also easily maintain cleanliness and organization in your office thanks to the abundance of storage areas available in this set.',
  {'entities': [(0, 56, 'PRODUCT'), (204, 232, 'PRODUCT')]}),
 ('Innova L-Shaped Desk with Pedestal and Hutch, 1 Lateral File Cabinet, and 1 Bookcase Bark Gray Description Items Included L-Shaped Desk with Pedestal and Hutch Lateral File Cabinet Bookcase The Innova collection features practical office furniture with a modern look and clean lines. The L-shaped desk with pedestal and hutch comes with an abundance of open and closed storage areas, a convenient keyboard tray, and a rubber strip for wire management. The lateral file cabinet offers 2 letter/legal file drawers to keep your work files and papers organized. The bookcase features 5 shelves to provide you with enough space to store your books and office essentials. This bundle is the perfect choice when it comes to creating a cohesive and stylish office space. Specifications GOOD QUALITY: Made from high-density commercial-grade particle board compliant with TSCA Title VI. Durable laminate finish. Desktop is 1ï¿½ï¿½ thick. Leg levelers on the file cabinet. Drawers and keyboard tray on ball-bearing slides. CONVENIENT 3-PIECE SET: File cabinet features 2 file drawers with letter/legal filing systems. One lock secures both drawers. Bookcase includes 2 fixed shelves (including base). and 3 adjustable shelves. GOOD STORAGE: The pedestal offers 2 utility drawers and 1 file drawer with a letter/legal filing system. One lock secures bottom two drawers. Hutch offers 4 closed storage compartments and 4 paper shelves. 10-YEAR LIMITED WARRANTY: Warranted to the original owner against defects in materials or workmanship. Assembly required (2 people). MADE IN CANADA: Excellent customer support. Desk comes with rubber strip for wire management.',
  {'entities': [(0, 44, 'PRODUCT'), (122, 159, 'PRODUCT')]}),
 ('Innova Desk Shell Available in 2 Colors Description Items Included Desk Shell The Innova collection is perfect for keeping your office space clean and organized. The multiple storage options will help you stay clutter-free. If you still find yourself searching for more storage solutions, try pairing a desk with an Innova bookshelf. Your office space will feel cohesive and stylish with the pieces from this collection. Dimensions Overall Dimensions: 59.4W x 22D x 30.4H Specifications GOOD QUALITY: Made from high-density commercial-grade particle board compliant with TSCA Title VI. Durable laminate finish. Desktop is 1ï¿½ï¿½ thick. PERFECT ADDITION TO YOUR SPACE: This compact, smaller desk is ideal for every type of workplace, including the home office. Can be used on its own or as a complement to a standing desk, a credenza desk, or even an L-shaped desk. HIGHLY FUNCTIONAL: Features leg levelers to adjust to any floor irregularities. The finished back allows for the desk to be placed anywhere in a room. 5-YEAR LIMITED WARRANTY: Our desk is warranted to the original owner against defects in materials or workmanship. Surface resists scratches, stains and wear. Assembly required (2 people). MADE IN CANADA: Excellent customer support. Meets or exceeds ANSI/BIFMA performance standards.',
  {'entities': [(0, 17, 'PRODUCT'), (67, 77, 'PRODUCT')]}),
 ('Dayton 3-Piece Set Including an L-Shaped Desk with Hutch, a Lateral File Cabinet, and a Bookcase Bark Gray Description Items Included L-Shaped Desk with Hutch Lateral File Cabinet Bookcase Dayton is distinguishable by its large work surfaces and multiple areas of open and closed storage. Durable 1ï¿½ commercial grade work surfaces with melamine finish that resist scratches, stains and wear. Pedestal with one utility drawer and one file drawer. File drawer with letter/legal filing system. Side section with open storage and 1 fixed shelf. The keyboard shelf has double extension ball-bearing slides for smooth and quiet operation. Sliding frosted glass doors with metals handles. Wire management system. Fully reversible. The Lateral File has one lock that secures both drawers. Two file drawers with letter/legal filing system. The bookcase has 5 shelves, three of which are adjustable. Assembly required. Dimensions L-Shaped Desk: 62.6ï¿½W x 65ï¿½D x 62.8ï¿½H Bookcase: 35.5ï¿½W x 11.9ï¿½D x 65.5ï¿½H Lateral File: 28.4ï¿½W x 19.6ï¿½D x 30.4ï¿½H',
  {'entities': [(0, 18, 'PRODUCT'), (189, 195, 'PRODUCT')]}),
 ('Lumina Full Murphy Wall Bed with Desk and 2 Storage Units (107ï¿½ï¿½) White Chocolate Description Items Included Murphy Wall Bed with Desk 2 Storage Units The Lumina collection is a brilliant way to optimize any space. The Lumina collection is the ideal solution when it comes to furnishing condos, lofts, guest rooms, and offices! The modern designs and colors found in this collection are sure to enhance any decor. Bring a touch of elegance and multi-functionality to any room in your home. The bed and cabinets must be anchored to a wall. Dimensions Overall Dimensions: 106.3W x 19.5D x 80.8H Specifications GOOD QUALITY: Made in North America from quality laminated particle board compliant with TSCA Title VI. Set features 4 CSA-approved LED lights (120V). Mattress not included. Thanks to dual piston system, bed folds up and down safely. SPACE SAVER: Ideal for small spaces and multifunctional rooms. Clearance of 9. 6ï¿½ under bed when open. Assign a clearance of 30-36ï¿½ï¿½ around open bed. EASY TO USE: Features a desk attached to the Murphy bed. Remove all items from desk, fold desk and snap safety catch, open two panels on the ends, use handle to pull and lower bed towards you, then flip leg onto floor. CONVENIENT: Accommodates most standard Full mattresses (8-10ï¿½ï¿½ thick, 60-90 lb, 54ï¿½ï¿½ x 75ï¿½ï¿½). No futons. Both storage units have 2 fixed and 3 adjustable shelves, 2 drawers, and 1 grommet. Drawers on ball-bearing slides. MADE IN NORTH AMERICA: Assembly required (3 people). Maximum weight supported is 1000 lb (including mattress). Minimum height of anchoring wall is 83. 5ï¿½. Desk weight limit is 50 lb. Box spring not required. Bed must be anchored to a wall.',
  {'entities': [(0, 37, 'PRODUCT'),
    (109, 134, 'PRODUCT'),
    (1037, 1048, 'PRODUCT')]}),
 ('Lumina Full Murphy Wall Bed with Desk White Chocolate Description Items Included Murphy Wall Bed with Desk The Lumina collection is a brilliant way to optimize any space. The Lumina collection is the ideal solution when it comes to furnishing condos, lofts, guest rooms, and offices! The modern designs and colors found in this collection are sure to enhance any decor. Bring a touch of elegance and multi-functionality to any room in your home. The bed must be anchored to a wall. Dimensions Overall Dimensions: 59.3W x 19.5D x 80.8H Specifications GOOD QUALITY: Made in North America from quality laminated particle board compliant with TSCA Title VI. The Murphy bed features 2 CSA-approved LED lights (120V). Mattress not included, box spring not required. SPACE SAVER: Ideal for small spaces and multifunctional rooms. Assign a clearance of 30-36ï¿½ï¿½ around open bed. Clearance of 9. 6ï¿½ under bed when open. EASY TO USE: Features a desk attached to the Murphy bed. Simply remove all items from desk, fold desk and snap safety catch, open two panels on the ends, use the handle to pull and lower the bed towards you, then flip the leg down onto the floor. CONVENIENT: Accommodates most standard Full mattresses (8-10ï¿½ï¿½ thick, 60-90 lb, 54ï¿½ï¿½ x 75ï¿½ï¿½). No futons. Thanks to dual piston system, the bed folds up and down safely in seconds without heavy lifting. MADE IN NORTH AMERICA: Assembly required (3 people). Maximum weight supported is 1000 lb (including mattress). Minimum height of anchoring wall is 83. 5ï¿½. Desk weight limit is 50 lb. Bed must be anchored to a wall.',
  {'entities': [(0, 37, 'PRODUCT'),
    (81, 106, 'PRODUCT'),
    (658, 668, 'PRODUCT'),
    (955, 966, 'PRODUCT')]}),
 ('Hampton L-Shaped Corner Gaming Desk Available in 2 Colors Description Items Included L-Shaped Corner Gaming Desk This L-shaped gaming desk is the perfect desk to help you conquer the gaming world! With storage options specifically designed to accommodate electronics and gaming accessories, this desk is sure to suit your needs. Everything you need will be within reach, but you will always have more than enough space to manoeuvre. This gaming desk is sturdy and carefully constructed to support your equipment. Both your friends and foes will be jealous of your cool gaming desk. Dimensions Overall Dimensions: 65.94W x 65.94D x 35.56H',
  {'entities': [(0, 35, 'PRODUCT'), (85, 112, 'PRODUCT')]}),
 ('Hampton 71W Corner Desk Available in 2 Colors Description Items Included Corner Desk This unique corner design is ideal for creating a private workstation in a busy office or a gaming corner in your home. The sturdy and durable work surface resists the wear and tear from a busy workday or an intense gaming session. The large work surface easily accommodates up to 3 monitors, which makes the desk ideal for jobs that require multiple monitors or for gaming setups. The open and closed storage areas on the pedestal are perfect for storing office or gaming supplies of various sizes. The keyboard tray frees up the work surface and helps you work from a comfortable position. The large wings and deep center unit ensure that you have ample space for your monitors. Dimensions Overall Dimensions: 70.1W x 70.1D x 35.4H Specifications Unique corner design is ideal for creating a private workstation or gaming corner. High-density commercial-grade particle board work surface is compliant with TSCA Title VI and has a durable laminate finish that resists scratches, stains, and wear. 1-inch thick desktop with shock resistant PVC edge detail provides a sturdy surface to work on. Large work surface supports up to 200 lb of evenly distributed weight and accommodates up to 3 monitors; perfect for PC gaming. Integrated monitor stand with 3 storage nooks allows you to easily reach items on your desk. 1 open and 1 closed storage area on the pedestal can hold various office or gaming supplies. Keyboard tray on  extension ball-bearing slides allows for the utmost comfort and flexibility when working (large enough to fit a keyboard and a mouse). Left and right wings are 35.5 inches wide x 23.75 inches deep, center unit is 34.6 x 34.6 inches in the back, and 39 inches deep in the center, at its widest point. Backed by the 10 Year Manufacturerï¿½s Warranty. North American made with local and imported parts.',
  {'entities': [(0, 23, 'PRODUCT'), (73, 84, 'PRODUCT')]}),
 ('Universel 4-Piece Set Including Four 30ï¿½  60ï¿½ Table Desks with Square Metal Legs Available in 3 Colors Save 28% Original price $1,239.99 Current price $886.99 Description Items Included 4 Desks This collection is perfect for any business trying to maximize their workspace. You can easily find multiple furniture options to help you and your employees make the most of your space. Specifications QUALITY MATERIALS: Table tops made from laminated, high-density commercial-grade particle board with melamine finish. 1ï¿½ thick top. The square metal legs feature levelers to adapt to any floor irregularities. CONVENIENT BUNDLE: If you are furnishing a large office space or a large dining area, this bundle is the ideal solution. Do not risk a mismatched decor when Bestar has the perfect bundle for you. STYLISH DESIGN: Our table desks come in several colors and they all feature square metal legs in matte silver finish. With contemporary features and clean lines, this furniture is sure to complement any space. 10-YEAR LIMITED WARRANTY: These pieces are warranted to the original owner against defects in materials or workmanship. Durable work surfaces resist scratches, stains and wear. NOTE: 2 people required for assembly. Excellent customer support.',
  {'entities': [(0, 21, 'PRODUCT')]}),
 ('Madura Bench New New Item O9490O Overall Dimensions W: 66 D: 22 H: 15 W: 167.64 D: 55.88 H: 38.1 Fabric Shown 6002-000 (grade P) COM Yardage 2.4 Finish Shown Description Frame: Solid Teak Frame Finish: Smoked Truffle Construction Solid Carbonized Teak, Exteriors Grade Wood and Materials Miscellaneous Optional fabric pad available. Specifications subject to change without notice. Due to differences in screen resolutions, the fabrics and finishes displayed may vary from the actual fabric and finish colors. ALL RELATED PRODUCTS Madura Sofa New O9457',
  {'entities': [(0, 16, 'PRODUCT'), (531, 546, 'PRODUCT')]}),
 ('Bali Daybed New New Item O1289 Overall Dimensions W: 57 D: 68 H: 35 SH: 16 AH: 29 SD: 57 BA: 54-1/2 W: 144.78 D: 172.72 H: 88.9 SH: 40.64 AH: 73.66 SD: 144.78 BA: 138.43 Fabric Shown 6002-000 (grade P) 6032-010 (grade C) COM Yardage 14.6 Finish Shown Description Material: Split Cane All-Weather Wicker Material Finish: Peppercorn Base: Powder Coated Aluminum Base Finish: Lava Pillows (4) Loose Fiber Back pillows knife edge with welt; Polyester fiber in waterproof tick Construction Powder Coated Aluminum Frame Cushion HR foam with polyester fiber wrap encased in water resistant ticking Specifications subject to change without notice. Due to differences in screen resolutions, the fabrics and finishes displayed may vary from the actual fabric and finish colors. ALL RELATED PRODUCTS Bali Swivel Chair OP2012S Bali Swivel Chair OP2023S',
  {'entities': [(0, 15, 'PRODUCT'),
    (789, 806, 'PRODUCT'),
    (815, 832, 'PRODUCT')]}),
 ('Madura Sofa New New Item O9457 Overall Dimensions W: 85-1/2 D: 36 H: 32 SH: 18 AH: 28-1/2 SD: 22 BA: 79-1/2 W: 217.17 D: 91.44 H: 81.28 SH: 45.72 AH: 72.39 SD: 55.88 BA: 201.93 Fabric Shown 6002-000 (grade P) COM Yardage 11.2 Finish Shown Description Material: Solid Teak Arm panels with Exteriors Grade Wood and Materials Material Finish: Smoked Truffle Pillows (2) 4 Loose Fiber Back pillows box edge with topstitch; Polyester fiber in waterproof ticking Construction Carbonized Teak, Exteriors Grade Wood and Materials Cushion Exteriors Plush Specifications subject to change without notice. Due to differences in screen resolutions, the fabrics and finishes displayed may vary from the actual fabric and finish colors. ALL RELATED PRODUCTS Madura Bench New O9490O',
  {'entities': [(0, 15, 'PRODUCT'), (744, 760, 'PRODUCT')]}),
 ('Tanah Sofa New New Item O1207 Overall Dimensions W: 80 D: 36-1/2 H: 26 SH: 18 AH: 26 SD: 23 BA: 76 W: 203.2 D: 92.71 H: 66.04 SH: 45.72 AH: 66.04 SD: 58.42 BA: 193.04 Fabric Shown 6048-000 (grade L) COM Yardage 10.5 Description Material: Teak Material Finish: Weathered Pillows (2) Loose Fiber Back pillows knife edge; Polyester fiber in waterproof tick Construction Teak, Teak Slat Seat Deck Cushion Exteriors Plush Specifications subject to change without notice. Due to differences in screen resolutions, the fabrics and finishes displayed may vary from the actual fabric and finish colors. ALL RELATED PRODUCTS Tanah Chair New O1202 Tanah Swivel Chair New O1202S',
  {'entities': [(0, 14, 'PRODUCT'),
    (615, 630, 'PRODUCT'),
    (637, 659, 'PRODUCT')]}),
 ('Wailea Daybed New New Item O2019 Overall Dimensions W: 70-1/2 D: 53 H: 28-1/2 SH: 16 SD: 42 W: 179.07 D: 134.62 H: 72.39 SH: 40.64 SD: 106.68 Fabric Shown 6049-000 (grade Q) COM Yardage 5.4 Finish Shown Description Material: Sock Weave Material Finish: Nordic Gray Base: Powder Coated Aluminum Base Finish: Oyster Pillows (5) Loose Fiber Back pillows knife edge weltless; Polyester fiber in waterproof tick Construction Powder Coated Aluminum Frame Cushion HR foam with polyester fiber wrap encased in water resistant ticking Specifications subject to change without notice. Due to differences in screen resolutions, the fabrics and finishes displayed may vary from the actual fabric and finish colors. ALL RELATED PRODUCTS Wailea Swivel Chair OP2013S Wailea Swivel Chair New OP2014S',
  {'entities': [(0, 17, 'PRODUCT'),
    (724, 743, 'PRODUCT'),
    (752, 775, 'PRODUCT')]}),
 ('Capri Swivel Chair New New Item OP1012S Overall Dimensions W: 41 D: 38 H: 25-1/2 SH: 19 AH: 26 SD: 22 BA: 27 W: 104.14 D: 96.52 H: 64.77 SH: 48.26 AH: 66.04 SD: 55.88 BA: 68.58 Fabric Shown 6048-000 (grade L) COM Yardage 4.6 Finish Shown Description Material: Rope Material Finish: Mist Gray Legs: Aluminum Leg Finish: Raven Pillows (1) Loose Fiber Back pillow boxed edge weltless; Polyester fiber in waterproof tick Construction Powder Coated Aluminum Frame Cushion Exteriors Plush Specifications subject to change without notice. Due to differences in screen resolutions, the fabrics and finishes displayed may vary from the actual fabric and finish colors. ALL RELATED PRODUCTS Capri Ottoman OP1011 Capri Chair OP1012 Capri Sofa OP1017',
  {'entities': [(0, 22, 'PRODUCT'),
    (681, 694, 'PRODUCT'),
    (702, 713, 'PRODUCT'),
    (721, 731, 'PRODUCT')]}),
 ('Lovina Chair New New Item O8713 Overall Dimensions W: 27-1/2 D: 35-1/2 H: 34 SH: 18 AH: 24 SD: 22 BA: 23 W: 69.85 D: 90.17 H: 86.36 SH: 45.72 AH: 60.96 SD: 55.88 BA: 58.42 Fabric Shown 6031-002 (grade L) COM Yardage 2.4 Finish Shown Description Frame: Teak Frame Finish: Smoked Truffle Construction Solid Teak with Exterior Grade Materials Cushion Tight seat, quick dry foam Specifications subject to change without notice. Due to differences in screen resolutions, the fabrics and finishes displayed may vary from the actual fabric and finish colors.',
  {'entities': [(0, 16, 'PRODUCT')]}),
 ('Savaii Swivel Chair New New Item O1402S Overall Dimensions W: 40 D: 41 H: 32 SH: 17-1/2 SD: 23 W: 101.6 D: 104.14 H: 81.28 SH: 44.45 SD: 58.42 Fabric Shown 6031-002 (grade L) COM Yardage 2.6 Finish Shown Description Material: Round Rope Material Finish: Marled Clay Base: Powder Coated Aluminum Base Finish: Stone Pillows (1) Loose Fiber Back pillow knife edge; Polyester fiber in waterproof tick Construction Exterior Grade Wood & Materials Cushion Exteriors Plush Specifications subject to change without notice. Due to differences in screen resolutions, the fabrics and finishes displayed may vary from the actual fabric and finish colors.',
  {'entities': [(0, 23, 'PRODUCT')]}),
 ('Tanah Chair New New Item O1202 Overall Dimensions W: 34 D: 36-1/2 H: 26 SH: 18 AH: 26 SD: 23 BA: 30 W: 86.36 D: 92.71 H: 66.04 SH: 45.72 AH: 66.04 SD: 58.42 BA: 76.2 Fabric Shown 6048-000 (grade L) COM Yardage 4.8 Description Material: Teak Material Finish: Weathered Pillows (1) Loose Fiber Back pillow knife edge; Polyester fiber in waterproof tick Construction Teak, Teak Slat Seat Deck Cushion Exteriors Plush Specifications subject to change without notice. Due to differences in screen resolutions, the fabrics and finishes displayed may vary from the actual fabric and finish colors. ALL RELATED PRODUCTS Tanah Swivel Chair New O1202S Tanah Sofa New O1207',
  {'entities': [(0, 15, 'PRODUCT'),
    (612, 634, 'PRODUCT'),
    (642, 656, 'PRODUCT')]}),
 ('Tanah Swivel Chair New New Item O1202S Overall Dimensions W: 32 D: 31 H: 26 SH: 18-1/2 AH: 26 SD: 23 BA: 28 W: 81.28 D: 78.74 H: 66.04 SH: 46.99 AH: 66.04 SD: 58.42 BA: 71.12 Fabric Shown 6045-010 (grade C) COM Yardage 4.8 Description Material: Teak Material Finish: Weathered Pillows (1) Loose Fiber Back pillow knife edge; Polyester fiber in waterproof tick Construction Teak, Teak Slat Seat Deck Cushion Exteriors Plush Specifications subject to change without notice. Due to differences in screen resolutions, the fabrics and finishes displayed may vary from the actual fabric and finish colors. ALL RELATED PRODUCTS Tanah Chair New O1202 Tanah Sofa New O1207',
  {'entities': [(0, 22, 'PRODUCT'),
    (621, 636, 'PRODUCT'),
    (643, 657, 'PRODUCT')]}),
 ('Wailea Swivel Chair New New Item OP2014S Overall Dimensions W: 36-1/2 D: 30-1/2 H: 33-1/2 SH: 19 AH: 32-1/2 SD: 18 BA: 28 W: 92.71 D: 77.47 H: 85.09 SH: 48.26 AH: 82.55 SD: 45.72 BA: 71.12 Fabric Shown 6048-000 (grade L) COM Yardage 3.4 Finish Shown Description Material: Sock Weave Material Finish: Cadet Gray Base: Powder Coated Aluminum Base Finish: Dove Grey Pillows (1) Loose Fiber Back pillow knife edge with welt; Polyester fiber in waterproof ticking Construction Powder Coated Aluminum Frame Cushion Exteriors Plush Miscellaneous Suitable for indoor and outdoor use. Specifications subject to change without notice. Due to differences in screen resolutions, the fabrics and finishes displayed may vary from the actual fabric and finish colors. ALL RELATED PRODUCTS Wailea Daybed New O2019 Wailea Swivel Chair OP2013S',
  {'entities': [(0, 23, 'PRODUCT'),
    (774, 791, 'PRODUCT'),
    (798, 817, 'PRODUCT')]}),
 ('Hyannis Cocktail Table New New Item X02-028 Overall Dimensions W: 49.13 D: 32.19 H: 16.13 W: 124.78 D: 81.76 H: 40.96 Description Glass reinforced cast concrete in London Fog finish Polyurethane protective top coat Open front-to-back, tapered design bottom Stainless steel adjustable glides Specifications subject to change without notice. Due to differences in screen resolutions, the fabrics and finishes displayed may vary from the actual fabric and finish colors. ALL RELATED PRODUCTS Hyannis Accent Table New X02-107',
  {'entities': [(0, 26, 'PRODUCT'), (489, 513, 'PRODUCT')]}),
 ('Leeward Cocktail Table New New Item X02-009 Overall Dimensions W: 53.94 D: 28 H: 18 W: 137.00 D: 71.12 H: 45.72 Finish Shown Description Carbonized teak in Smoked Truffle finish Slat top with small open space between slats Galvanized adjustable glides Specifications subject to change without notice. Due to differences in screen resolutions, the fabrics and finishes displayed may vary from the actual fabric and finish colors. ALL RELATED PRODUCTS Leeward Cocktail Table New X02-019 Leeward Side Table New X02-159',
  {'entities': [(0, 26, 'PRODUCT'),
    (450, 476, 'PRODUCT'),
    (485, 507, 'PRODUCT')]}),
 ('Leeward Cocktail Table New New Item X02-019 Overall Dimensions W: 42.13 D: 42.13 H: 18 W: 107.00 D: 107.00 H: 45.72 Finish Shown Description Carbonized teak in Smoked Truffle finish Slat top with small open space between slats Galvanized adjustable glides Specifications subject to change without notice. Due to differences in screen resolutions, the fabrics and finishes displayed may vary from the actual fabric and finish colors. ALL RELATED PRODUCTS Leeward Cocktail Table New X02-009 Leeward Side Table New X02-159',
  {'entities': [(0, 26, 'PRODUCT'),
    (454, 480, 'PRODUCT'),
    (489, 511, 'PRODUCT')]}),
 ('Montecito Cocktail Table New New Item X02-024 Overall Dimensions H: 18 Diameter: 40 H: 45.72 Diameter: 101.60 Description Glass reinforced cast concrete with Silver Slate finish Polyurethane protective top coat Stainless steel adjustable glides Specifications subject to change without notice. Due to differences in screen resolutions, the fabrics and finishes displayed may vary from the actual fabric and finish colors. ALL RELATED PRODUCTS Montecito Side Table New X02-124',
  {'entities': [(0, 28, 'PRODUCT'), (443, 467, 'PRODUCT')]}),
 ('Paseo Cocktail Table New New Item X02-007 Overall Dimensions H: 17 Diameter: 42 H: 43.18 Diameter: 106.68 Finish Shown Description Inset carbonized suar wood inset top in Smoked Truffle finish Cast resin case in White finish Glides Specifications subject to change without notice. Due to differences in screen resolutions, the fabrics and finishes displayed may vary from the actual fabric and finish colors. ALL RELATED PRODUCTS Paseo Accent Table New X02-158',
  {'entities': [(0, 24, 'PRODUCT'), (430, 452, 'PRODUCT')]}),
 ('Tambora Cocktail Table New New Item X02-018 Overall Dimensions H: 15.75 Diameter: 38 H: 40.01 Diameter: 96.52 Description Laminated and hand hewn striped stone with smooth top and textured sides Polyurethane protective top coat Adjustable glides Specifications subject to change without notice. Due to differences in screen resolutions, the fabrics and finishes displayed may vary from the actual fabric and finish colors. ALL RELATED PRODUCTS Tambora Accent Table New X02-108',
  {'entities': [(0, 26, 'PRODUCT'), (444, 468, 'PRODUCT')]}),
 ('Casitas Accent Table New New Item X02-150 Overall Dimensions W: 16 D: 16 H: 20 W: 40.64 D: 40.64 H: 50.80 Description Natural stonecast with smooth square top and rattan texture tapered case bottom Polyurethane protective top coat Adjustable glides Specifications subject to change without notice. Due to differences in screen resolutions, the fabrics and finishes displayed may vary from the actual fabric and finish colors.',
  {'entities': [(0, 24, 'PRODUCT')]}),
 ('Hyannis Accent Table New New Item X02-107 Overall Dimensions W: 19.31 D: 19.00 H: 20.06 W: 49.05 D: 48.26 H: 50.96 Description Glass reinforced cast concrete in London Fog finish Polyurethane protective top coat Open front-to-back, tapered design bottom Stainless steel adjustable glides Specifications subject to change without notice. Due to differences in screen resolutions, the fabrics and finishes displayed may vary from the actual fabric and finish colors. ALL RELATED PRODUCTS Hyannis Cocktail Table New X02-028',
  {'entities': [(0, 24, 'PRODUCT')]}),
 ('Leeward Side Table New New Item X02-159 Overall Dimensions W: 23.13 D: 23.13 H: 25 W: 58.74 D: 58.74 H: 63.50 Finish Shown Description Carbonized teak in Smoked Truffle finish Slat top with small open space between slats Galvanized adjustable glides Specifications subject to change without notice. Due to differences in screen resolutions, the fabrics and finishes displayed may vary from the actual fabric and finish colors. ALL RELATED PRODUCTS Leeward Cocktail Table New X02-009 Leeward Cocktail Table New X02-019',
  {'entities': [(0, 22, 'PRODUCT'),
    (448, 474, 'PRODUCT'),
    (483, 509, 'PRODUCT')]}),
 ('Voile Accent Table New New Item X02-154 Overall Dimensions H: 21.88 Diameter: 16 H: 55.56 Diameter: 40.64 Description Natural stonecast with smooth top and Coco Twig texture base Polyurethane protective top coat Adjustable glides Specifications subject to change without notice. Due to differences in screen resolutions, the fabrics and finishes displayed may vary from the actual fabric and finish colors.',
  {'entities': [(0, 22, 'PRODUCT')]}),
 ('Madura Dining Table New New Item X02-272, X02-273 Pieces Shown X02-272 Dining Table Top X02-273 Dining Table Base Overall Dimensions H: 30 Diameter: 60 H: 76.20 Diameter: 152.40 Finish Shown Description Carbonized sliced teak in Smoked Truffle finish Powder-coated stainless steel metal supports Galvanized adjustable glides Specifications subject to change without notice. Due to differences in screen resolutions, the fabrics and finishes displayed may vary from the actual fabric and finish colors.',
  {'entities': [(0, 23, 'PRODUCT'),
    (71, 83, 'PRODUCT'),
    (96, 108, 'PRODUCT')]}),
 ('Corfu Side Chair New New Item X02-547X Overall Dimensions W: 20.63 D: 23.75 H: 33.88 SH: 18.38 SD: 19 W: 52.39 D: 60.33 H: 86.04 SH: 46.67 SD: 48.26 Fabric Shown 6503-000 (grade N) COM Yardage 1.00 Finish Shown Description Base: Carbonized teak frame Base Finish: Smoked Truffle Construction Galvanized adjustable glides Cushion Upholstered slip seat Specifications subject to change without notice. Due to differences in screen resolutions, the fabrics and finishes displayed may vary from the actual fabric and finish colors.',
  {'entities': [(0, 20, 'PRODUCT')]}),
 ('Santa Cruz Arm Chair New New Item X02-545 Overall Dimensions W: 26.25 D: 28 H: 30 SH: 18 AH: 30 SD: 18.19 W: 66.68 D: 71.12 H: 76.20 SH: 45.72 AH: 76.20 SD: 46.20 (Note: Seat Depth dimensions are with pillow) Chair Height with back pillow 36.50 in. | 92.71 cm. Fabric Shown 6048-000 (grade L) COM Yardage 3.50 Finish Shown Description Material: Sock Weave Material Finish: Nordic Gray Base: Powder-coated stainless steel Base Finish: Silver Mist Construction Reticulated foam Cushion Loose seat cushion; Loose back pillow; Adjustable glides. Specifications subject to change without notice. Due to differences in screen resolutions, the fabrics and finishes displayed may vary from the actual fabric and finish colors. ALL RELATED PRODUCTS Santa Cruz Arm Chair New X02-549',
  {'entities': [(0, 24, 'PRODUCT'), (740, 764, 'PRODUCT')]}),
 ('Santa Cruz Arm Chair New New Item X02-549 Overall Dimensions W: 26.25 D: 28 H: 30 SH: 18 AH: 30 SD: 18.19 W: 66.68 D: 71.12 H: 76.20 SH: 45.72 AH: 76.20 SD: 46.20 (Note: Seat Depth dimensions are with pillow) Chair Height with back pillow 36.50 in. | 92.71 cm. Fabric Shown 6048-000 (grade L) COM Yardage 3.50 Finish Shown Description Material: Sock Weave Material Finish: Cadet Gray Base: Powder-coated stainless steel Base Finish: Silver Mist Construction Reticulated foam Cushion Loose seat cushion; Loose back pillow; Adjustable glides. Specifications subject to change without notice. Due to differences in screen resolutions, the fabrics and finishes displayed may vary from the actual fabric and finish colors. ALL RELATED PRODUCTS Santa Cruz Arm Chair New X02-545',
  {'entities': [(0, 24, 'PRODUCT'), (739, 763, 'PRODUCT')]}),
 ('Description Anthracite Painted Aluminum Frame Canvas Granite Cushions Acrylic Rope Wicker Backrest Lounge comfortably with the Cuddle Outdoor Chair! With anthracite painted aluminum legs and its stylish rope backrest, this chair is perfect for your patio or backyard. Use it with the Cuddle ottoman to complete the set. |Dimensions||30  31  30 in| |Available Colors| Canvas Granite |Minimum Order| 1',
  {'entities': [(127, 147, 'PRODUCT'), (284, 298, 'PRODUCT')]}),
 ('Description Anthracite Painted Aluminum Legs Canvas Granite Cushion Lounge in style with the Cuddle ottoman! It has an aluminum frame and a thick textured cushion perfect for relaxing under the sun. Pair it with Cuddle outdoor chair to complete the set. |Dimensions||23  18  17 in| |Available Colors| Canvas Granite |Minimum Order| 1',
  {'entities': [(93, 107, 'PRODUCT'), (212, 232, 'PRODUCT')]}),
 ('Description Anthracite Painted Aluminum Legs Acrylic Rope Wicker Backrest Canvas Granite Cushion The Diva armchair lets you relax under the sun. With durable fabric and an aluminum frame, itï¿½s perfect for family fun. Pair it with the rest of the Diva collection to complete the set. |Dimensions||30  31  30 in| |Available Colors| Canvas Granite |Minimum Order| 1',
  {'entities': [(101, 114, 'PRODUCT')]}),
 ('Description Anthracite Painted Aluminum Frame Canvas Granite Seat Cushion Acrylic Rope Wicker Backrest The Diva Barstool is the perfect seat for some drinks under the sun. With durable fabric and an aluminum frame, itï¿½s perfect for family fun. Pair it with the rest of the Diva collection to complete the set. |Dimensions||22  23  39 in| |Available Colors| Canvas Granite |Minimum Order| 1',
  {'entities': [(107, 120, 'PRODUCT')]}),
 ('Description Anthracite Painted Aluminum Frame Acrylic Rope Wicker Backrest Canvas Granite Seat and back Cushions The Diva Daybed lets you relax under the sun. With durable fabric and an aluminum frame, itï¿½s perfect for . Pair it with the rest of the Diva collection to complete the set. |Dimensions||54  62  28 in| |Available Colors| Canvas Granite |Minimum Order| 1',
  {'entities': [(117, 128, 'PRODUCT')]}),
 ('Description Anthracite Painted Aluminum Frame Canvas Granite Seat and Back Cushions The Diva Dining Chair lets you relax under the sun. With durable fabric and an aluminum frame, itï¿½s perfect for family fun. Pair it with the rest of the Diva collection to complete the set. |Dimensions||24  24  31 in| |Available Colors| Canvas Granite |Minimum Order| 1',
  {'entities': [(88, 105, 'PRODUCT')]}),
 ('Description Anthracite Painted Aluminum Frame Canvas Granite Cushion The Diva ottoman lets you relax under the sun, with durable fabric and aluminum frame, itï¿½s perfect for family fun. Pair it with the rest of the Diva outdoor collection! |Dimensions||23  18  17 in| |Available Colors| Canvas Granite |Minimum Order| 1',
  {'entities': [(73, 85, 'PRODUCT')]}),
 ('Description Aluminum Anthracite Painted Frame Frosted Glass Top Seats up to 6 The Diva Dining Table lets you enjoy meals under the sun. With frosted glass and an aluminum frame, itï¿½s perfect for family dinners. Pair it with the rest of the Diva collection to complete the set. |Dimensions||73  37  30 in| |Available Colors| Anthracite |Minimum Order| 1',
  {'entities': [(82, 99, 'PRODUCT')]}),
 ('Description Anthracite Painted Aluminum Frame Acrylic Rope Wicker Backing Canvas Granite Seat and Back Cushions The Diva Sofa is perfect for relaxing under the sun. With durable fabric and an aluminum frame, itï¿½s perfect for family fun. Pair it with the rest of the Diva collection to complete the set. |Dimensions||71  31  30 in| |Available Colors| Canvas Granite |Minimum Order| 1',
  {'entities': [(116, 125, 'PRODUCT')]}),
 ('Description Aluminum Legs and Top Matching Low Side Table Sleek and Stylish the DJ High Side Table is made up beautifully painted aluminum and is versatile enough to be added to any patio set. Buy it with the DJ Low Side Table to complete the set. |Dimensions||20  20 in| |Available Colors| White, Anthracite, Beige |Minimum Order| 1',
  {'entities': [(80, 98, 'PRODUCT'), (209, 226, 'PRODUCT')]}),
 ('Description Aluminum Frame Available with matching high side table Sleek and Stylish the DJ Low Side Table is made up beautifully painted aluminum and is versatile enough to be added to any patio set. Buy it with the DJ High Side Table to complete the set. |Dimensions||24  18 in| |Available Colors| White, Anthracite, Beige |Minimum Order| 1',
  {'entities': [(89, 106, 'PRODUCT'), (217, 235, 'PRODUCT')]}),
 ('Description Taupe or White Painted Aluminum Frame Batyline ISO Fabric Cover Attached Wheels The Ella Lounger is perfect for tanning and relaxing in the sun. With an aluminum frame and attached wheels, this lounger is comfortable and practical. |Dimensions||80  30  11 in| |Available Colors| Taupe, White',
  {'entities': [(96, 108, 'PRODUCT')]}),
 ('Description Taupe or White Painted Aluminum Frame Frosted Glass Top The Ella Side Table is a durable and versatile outdoor living piece. With a neutral colored aluminum frame and a frosted glass top, the Ella side table is perfect for backyard or patio. Pair with the Ella lounger to complete the set. |Dimensions||18  18  18 in| |Available Colors| Taupe, White |Assembled| Yes |Minimum Order| 1',
  {'entities': [(72, 87, 'PRODUCT'),
    (204, 219, 'PRODUCT'),
    (268, 280, 'PRODUCT')]}),
 ('Description Anthracite Aluminum Upholstered Frame Built-in Tray Top The Flexi Chair is a uniquely practical piece. Its fully upholstered aluminum frame comes with a built in wooden finished frame making it perfect for backyard dinner and drinks. |Dimensions||34  34  31 in| |Available Colors| Taupe, White |Minimum Order| 1',
  {'entities': [(72, 83, 'PRODUCT')]}),
 ('Description Polypropylene Frame Wicker Backing Powder-coated Metal legs Stylish and modern, the Piazza outdoor armchair is made of durable polypropylene, with a wicker backrest and powder-coated metal legs. |Weight||10.5 lbs| |Dimensions||23  22  31.5 in| |Assembled| Yes |Available Colors| Black, Grey, White, blue |Minimum Order| 1',
  {'entities': [(96, 119, 'PRODUCT')]}),
 ('Kyra Wall Mirror Features: Rectangular Sustainable elm wood Interlocked grain Measurements: Height:100 cm Width:40 cm Depth:3 cm Product Code:PHW The Furniture Megastore Bradgate house Derby Road DE75 7QL 01773714816 The Furniture Megastore Bradgate House Derby Road DE75 7QL 01773 714816 Rectangular',
  {'entities': [(0, 16, 'PRODUCT')]}),
 ('Mirrored Glass Octagonal Wall Mirror 61cm Reflective and geometric, this contemporary wall mirror is a real focal piece. This wall mirror creates a sense of depth thanks to the three layers of reflective glass that combine to create its octagonal frame. Each individual pane features bevelled edges which will help the mirror to catch the light. Adding some geometric drama to its surroundings, it looks great styled with opulent boutique hotel furnishings.',
  {'entities': [(0, 36, 'PRODUCT')]}),
 ("Glitz Sun Burst Mirrored Glass Wall Mirror 102cm Make a style statement in the living room or bedroom with the Glitzy wall mirror. Its tiled frame is comprised of individual rectangular mirrors that taper out like sunrays from its circular centre. Its uneven outer rim contributes to the abstract solar effect. Each tile's bevelled edge lends the mirror texture and will refract the light beautifully. ",
  {'entities': [(0, 42, 'PRODUCT'), (111, 130, 'PRODUCT')]}),
 ('Serpent Round Gold Wall Mirror Ideal for homes with bold dcor and luxe finishing, this Serpent wall mirror is a striking piece. Round in shape, the frame consists of a gold metallic, textured serpent that encompasses a central mirror. Add an interesting focal point with the round, serpent framed wall mirror ',
  {'entities': [(0, 30, 'PRODUCT'),
    (87, 106, 'PRODUCT'),
    (282, 308, 'PRODUCT')]}),
 ('Riva Gold Hexagonal Wall Mirror The metallic toned Riva wall mirror can be displayed as part of a luxury modern retro design theme. Add an interesting focal point with the round, serpent framed wall mirror. The seven hexagonal forms that are clustered together combine to create a reflective array. The sides of the mirror are finished in a warm gold colour, complementing the slim lines that house the individual mirrored panes on the surface of this contemporary piece.',
  {'entities': [(0, 31, 'PRODUCT'), (51, 67, 'PRODUCT')]}),
 ('Simi Arabesque Wall Mirror 86cm The striking Simi wall mirror is inspired by exotic arabesque design heritage. Details Its flowing lines and symmetrical design make this mirror an eye-catching statement piece for the wall. The multi-layered frame recesses the reflective surface, its grey finish weathered with flecks of black. Its distinctive design makes it a suitable accent piece for traveller/fusion furniture and lighting.',
  {'entities': [(0, 26, 'PRODUCT'), (45, 61, 'PRODUCT')]}),
 ('Square Inlaid Crystal Wall Mirror 60cm square wall mirror featuring a glamorous inlaid crystal frame. Measurements: Height: 60 cm Width: 60 cm Depth: 5 cm Product Code:CICM The Furniture Megastore Bradgate house Derby Road DE75 7QL 01773714816 The Furniture Megastore Bradgate House Derby Road DE75 7QL 01773 714816 60cm square wall mirror featuring a glamorous inlaid crystal frame.',
  {'entities': [(0, 33, 'PRODUCT')]}),
 ('Mustique Parquet Inlaid Wall Mirror With a nod towards the french colonial style this plain glass wall mirror is made from mindy ash and features parquet inlaid veneer work in a lightly brushed natural finish. Measurements: Height: 90 cm Width: 110 cm Depth: 3 cm Availability: Estimated to be available from 29 July 2021 Product Code: GD',
  {'entities': [(0, 35, 'PRODUCT')]}),
 ('Antique Bronze Curved Wall Mirror This is the Antique Bronze Ornate Curved Mirror, at 60cm in height and 40cm in width this is a practical styled mirror that is capable of being placed in various areas in an interior too. Featuring an ornate shape and complementing antique bronze finish this piece gives a subtle vintage yet timeless feel and look ',
  {'entities': [(0, 33, 'PRODUCT'), (46, 81, 'PRODUCT')]}),
 ('Bohemian Boutique Round Wall Mirror ', {'entities': [(0, 35, 'PRODUCT')]}),
 ('Casablanca Bronze Triple Wall Mirror This is the Casablanca Triple Wall Mirror, it features four individual mirrors to create a wall feature that is sure to capture the attention of many. This style also comes as a single piece or as a quad wall mirror. ',
  {'entities': [(0, 36, 'PRODUCT'), (49, 78, 'PRODUCT')]}),
 ('Cube Mango Wood Wall Mirror A Modern Style Collection Made From Solid Mango Wood In Light Natural Finish. ',
  {'entities': [(0, 27, 'PRODUCT')]}),
 ('Antique Silver Quarter foil Wall Mirror This Antique Silver Quarter foil Wall Mirror is sure to make a stylish impression in your space. ',
  {'entities': [(0, 39, 'PRODUCT'), (45, 84, 'PRODUCT')]}),
 ('Arthur Grey Large Wooden Wall Mirror Large Wooden Wall Mirror. A substantial piece skillfully hand-carved by Indian artisans, this well-proportioned piece has a beautifully simple design that will appeal to a wide variety of customers thanks to its ability to complement a variety of decor schemes. ',
  {'entities': [(0, 36, 'PRODUCT'), (37, 62, 'PRODUCT')]}),
 ("Limi Ornate Vintage-Inspired Rectangular Wall Mirror Intricate floral and leaves decorate this beautifully ornate and vintage-inspired rectangular wall mirror. It's ready to hang on your wall using the D-ring fittings on the back of the mirror.",
  {'entities': [(0, 52, 'PRODUCT')]}),
 ('Celeste Decagon Wall Mirror Gold 105 cm Diameter Decorative mirrors are ideal for homes of all shapes and sizes. Not only this but wall mirrors are easy to hang and can bring an instant sense of style. This Celeste Decagon Painted Gold Mirror is an incredibly artistic mirror that features a beautiful repeated geometric pattern in a painted gold finish. This mirror is ideal when you want to create a feature point. Measurements: Width 105cm x Height 105cm Out of Stock Available To Pre-Order Availability: Estimated to be available from 8 June 2021 Product Code: GD',
  {'entities': [(0, 27, 'PRODUCT'), (207, 242, 'PRODUCT')]}),
 ('Chelsea Wall Mirror 164 cm Wide in White High Gloss Chelsea Wall Mirror 164 cm Wide in White. Unconventional design a proposal for the followers of the latest trends. The clever mix of scratch resistant white melamine with beautiful high gloss fronts Pair with matching items from our Chelsea ',
  {'entities': [(0, 19, 'PRODUCT'),
    (52, 71, 'PRODUCT'),
    (285, 292, 'PRODUCT')]}),
 ('Crushed Diamond Mirrored LED Angel Wings Wall Art Features: Touch On and OFF Measurements: Height: 120 cm Width:90 cm Pair with matching items from our Diamond Crush Mirrored Collection. ****Delivery Est: 2 3 Weeks **** ProductCode: ASI',
  {'entities': [(0, 49, 'PRODUCT')]}),
 ('Decorative Large Crushed Diamond Mirrored Wall Clock 60cm Easy access battery housing and time set Hanging brackets and instructions attached A very large clock that will make a good focal point Measurements: Height: 60 cm Width: 60 cm Depth: 5 cm Pair with matching items from our Diamond Crush Mirrored Collection. ****Delivery Est: 2 3 Weeks **** Product Code: ASI',
  {'entities': [(17, 52, 'PRODUCT')]}),
 ('Designed with the home office in mind, the MODICA collection includes a Desk and Mobile File Pedestal. The unique platform desk stands securely on a black steel trestle base and features an open storage shelf flanked with two storage drawers. There is even a home for a small power strip to keep cables organized. The mobile file pedestal keeps hanging files and supplies close at hand and makes a great home for a printer or all-in-one. Model: 6341 Desk Design: Matthew Weatherly',
  {'entities': [(72, 102, 'PRODUCT')]}),
 ('Everyone needs a work space that is comfortable and inviting, and if it can be beautifully designed and neatly organized, all the better. Designed with an emphasis on ergonomics and organization, Sequel Office is a versatile and beautiful component collection offering the ultimate in style, function and flexibility for any home or small office application. The Sequel DESK is the cornerstone of the collection. The multi-function system drawer provides the ideal spot for a keyboard, mouse and rechargeable items. The Desk includes magnetically attached front and back panels making it easy to keep wires controlled and out of view. When positioned against a wall the back panel may be removed for direct access to wall outlets. Designer: Matthew Weatherly Model: 6001',
  {'entities': [(363, 374, 'PRODUCT'), (520, 524, 'PRODUCT')]}),
 ('Available in left and right-facing models, the Sequel Peninsula is perfect for a collaborative work environment with a chair on either side. The micro-etched glass top provides a smooth and durable work surface. The Peninsula attaches securely to Cabinets 6014, 6015, 6016 & 6017 with steel supports located under the glass top. Wire management channels are integrated into the steel frame and legs to keep cables organized and out of sight. Model 6018R is right-facing, 6018L is left-facing. Designer: Matthew Weatherly Model: 6018R/6018L',
  {'entities': [(47, 63, 'PRODUCT'), (216, 225, 'PRODUCT')]}),
 ('The Sequel Corner Desk makes full use of the corner of a room. The micro-etched glass top provides a smooth work surface and the neoprene-lined multifunction drawer holds supplies, a keyboard & mouse and/or rechargeable items. The Corner Desk attaches securely to Cabinets 6014, 6015, 6016 & 6017 with steel supports located under the glass top. Wire management channels are integrated into the steel frame and the support column conceals wires, a power strip and/or a router (not included). Designer: Matthew Weatherly Model: 6019 Corner Desk',
  {'entities': [(4, 22, 'PRODUCT'), (231, 242, 'PRODUCT')]}),
 ('The SEQUEL LIFT DESK is the best of both worlds. The deskï¿½s adjustable height allows you to keep yourself in motion with periods of sitting and standing throughout the workday. Combining beautiful styling, thoughtful engineering and ergonomic design, the SEQUEL sit+stand desk provides the versatility and movement that every workday requires. Features Micro-etched, black glass top Programmable digital keypad Integrated cable management Wire management ledge Partners beautifully with other sequel components',
  {'entities': [(4, 20, 'PRODUCT'),
    (53, 59, 'PRODUCT'),
    (255, 276, 'PRODUCT')]}),
 ('The SEQUEL LIFT DESK is the best of both worlds. The deskï¿½s adjustable height allows you to keep yourself in motion with periods of sitting and standing throughout the workday. Combining beautiful styling, thoughtful engineering and ergonomic design, the SEQUEL sit+stand desk provides the versatility and movement that every workday requires. Features Micro-etched, black glass top Programmable digital keypad Integrated cable management Wire management ledge Partners beautifully with other sequel components Note: Optional storage drawer sold seperate',
  {'entities': [(4, 20, 'PRODUCT'),
    (53, 59, 'PRODUCT'),
    (255, 276, 'PRODUCT')]}),
 ('Designed by Verner Panton. The Flowerpot lamp became synonymous with the Flower Power movement from the late 60s. Breaking from convention to embrace a more open, modern mentality. Such was the mindset of its designer, Verner Panton, one of the most forward-thinking talents of his time. The Flowerpot series consists of several different models, including three pendants and two table lamps all in a wide range of colours. Available in 9 colours. Details Materials Spun, Lacquered Aluminium. Colour-matched textile cord. Dimensions 500  H350mm 3m fabric cord Light Source (not included) E27 Max 60 Watt / 12 Watt CFL In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title Lead time 3-4 weeks Lead times are provided as a general guide to what is typical for this product but can vary depending on the specific variant and quantity you choose as well as current availability.',
  {'entities': [(31, 45, 'PRODUCT')]}),
 ('Designed by Arne Jacobsen and Flemming Lassen. The Mayor sofa was created for Sollerod City Hall in 1939 by Danish architects Arne Jacobsen and Flemming Lassen. The tight upholstery and the two rows of elegant buttons that extend over the armrests give the sofa a simple and stylish look with three loose cushions that comprise the seating area. The Mayor sofa has won the award for ï¿½Best Reissueï¿½ï¿½ in the prestigious Wallpaper* Design Award 2013. Made to order in a choice of fabric ranges. Add a note of your colour choice at checkout or email us. Details Materials Solid oak base available in three finishes: white oiled, smoked oiled or black stained. Kvadrat fabric or CA-MO leather upholstery. Dimensions H82cm  D62cm  L200cm. Seat height 44 cm',
  {'entities': [(51, 61, 'PRODUCT'), (350, 360, 'PRODUCT')]}),
 ("Designed by Jorn Utzon. The Utzon JU1 lamp is a reflection of Jorn Utzon's fascination with ships and boats as well as light. Designed ten years before his masterpiece the Sydney Opera House. The lamp combines the smoothness of the lines of a ship with the source of the light. Available in white or copper finish. Details Material White lacquered steel with white fabric cord, White and Brass or polished brass with black cord.Dimensions D220mm  H230mm In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title Lead time 3-4 weeks Lead times are provided as a general guide to what is typical for this product but can vary depending on the specific variant and quantity you choose as well as current availability.",
  {'entities': [(28, 42, 'PRODUCT')]}),
 ("Designed by Space Copenhagen. The Fly Lounge Series was created with the ultimate comfort in mind. The lovely loose cushions that comprise the back can be arranged and rearranged to suit your personal preferences. The design alludes to a Nordic aesthetic, such as the slatted back, the handcrafted solid oak frame and the organic shaped elements. Offered here in &Tradition selected timber / Hot Madison fabric combinations. Further upholstery options available under 'Details' tab please select the appropriate price Group and then add a note of your fabric choice & timber finish at Checkout. Details Materials Oiled oak frame and base in White Oiled Oak or Smoked Oiled Oak. CMHR foam and feather pillows.",
  {'entities': [(34, 51, 'PRODUCT')]}),
 ('Designed by Space Copenhagen. The Fly series by Space Copenhagen is a lounge collection made up of two sofas, an armchair, two tables and a pouf. Pushing the idea of lounging to its fullest, Fly is a series that allows the user to sit, lie, rest, and relax. 3 alternative lounge table sizes available: SC4 / SC5 / SC11. Details Materials Base: Solid oak, oak veneer and plywood treated with either smoked oil or white pigmented oil. Top: Honed Bianco Carrara marble, polished Nero Marquina marble or honed Azul Valverde marble. Dimensions H26cm  D80cm  L80cm Weight 45kg Fly High Legs available to give table height of 46cm contact us for more information. In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(34, 44, 'PRODUCT')]}),
 ('Designed by Space Copenhagen. The Fly series by Space Copenhagen is a lounge collection made up of two sofas, an armchair, two tables and a pouf. Pushing the idea of lounging to its fullest, Fly is a series that allows the user to sit, lie, rest, and relax. 3 alternative lounge table sizes available: SC4 / SC5 / SC11. Details Materials Base: Solid oak, oak veneer and plywood treated with either smoked oil or white pigmented oil. Top: Honed Bianco Carrara marble, polished Nero Marquina marble or honed Azul Valverde marble. Dimensions H26cm  D60cm  L120cm Weight 45kg Fly High Legs available to give table height of 46cm contact us for more information. In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(34, 44, 'PRODUCT')]}),
 ('Space Copenhagenï¿½s ambition with the Copenhagen pendant was to design a lamp with a simple industrial feel, but which was still elegant and poetic. The Copenhagen Pendant is an exercise in contrasts. Combining the classic and the modern, the maritime and the industrial. The series started with one pendant in three sizes in 2014 and was extended in 2018 with two table lamps, one wall lamp and one floor lamp. Details Designed by Space Copenhagen. Materials & available colours Lacquered steel shade with bronzed brass suspension. Satinized opal glass with bronzed brass suspension. SC6: Matt Black, Matt Stone, Matt Plum, Opal Glass SC7: Matt Black, Matt Stone, Matt Plum, Opal Glass SC8: Matt Black, Matt White Dimensions SC6: D200mm  H250mm SC7: D350mm  H400mm SC8: D600mm  H530mm Light Source (not included) SC6: G9 Max 40 Watt light bulb 3m fabric cord SC7: E27 Max 100 Watt light bulb 4m fabric cord SC8: E27 Max 150 Watt light bulb 6m fabric cord In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(37, 55, 'PRODUCT'), (152, 170, 'PRODUCT')]}),
 ("Designed by Space Copenhagen. The Fly Lounge Series was created with the ultimate comfort in mind. Offered here in &Tradition selected timber / Hot Madison fabric combinations. Further upholstery options available under 'Details' tab please select the appropriate price Group and then add a note of your fabric choice & timber finish at Checkout. Details Materials Oiled oak base in White Oiled Oak or Smoked Oiled Oak. CMHR foam and feathers.",
  {'entities': [(34, 51, 'PRODUCT')]}),
 ('Designed by Sami Kallio. The In Between table is beautifully crafted from solid timber with a functional round shape. The table legs use traditional wood turning techniques. Available in a range of finishes and two sizes SK3 and SK4. Details Materials Solid oak treated with either lacquer or oil / Solid Walnut treated with oil Dimensions SK3 D90cm  H73cm In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title This item is made specially for you and special terms apply (see Terms and Conditions). ',
  {'entities': [(29, 45, 'PRODUCT')]}),
 ('Designed by Luca Nichetto. The Cloud range of furniture combines the the defined shapes of the Danish design tradition and the soft and cosy pillows of the Italian one. Ultimately, the highest priority has been comfort and cosiness. Because of the compact dimensions of the furniture in the Cloud series, they are perfectly adaptable to contemporary living spaces. Cloudï¿½s base feature no-sag springs and the pillows are made using a mix of HR foam and feathers, providing ultimate comfort. Details Materials Plywood frame, solid wood and steel base with no-zag springs, HR foam and feather pillows, Kvadrat upholstery with leg options in either powder coated black or chromed steel. Dimensions Width 100 cm  Depth 84 cm  Height 75 cm. Seat height 40 cm.',
  {'entities': [(31, 55, 'PRODUCT'),
    (291, 303, 'PRODUCT'),
    (365, 372, 'PRODUCT')]}),
 ('Designed by Sami Kallio. In a desire to design a table that blends into a space without taking up much space, Sami Kallio created the In Between Table. A partner to his In Between Chair, the legs of the table echo the same angle as the front legs of the chair. At the same time, the tableï¿½s slim silhouette and absence of ornamentation make it an ideal match for any chair in any setting. The In Between Table is a mix of traditional wood craftsmanship, involving compression moulding and woodturning, together with innovative veneer techniques. As such, it is an homage to Kallioï¿½s Nordic heritage and craft training, as well as his modern Scandinavian sense of simplicity. Details Available in two sizes and three Oak finishes. Table top high quality MDF board with oak veneer on the top and edges. Legs solid wood involving a woodturning technique using a lathe to create the conical shape. Wood blocks are used to attach the legs to the table to make it more stable when mounted Materials Oak and Oak veneer Dimensions SK5 Length 200cm  Depth 90cm  Height 73cm SK6 Length 250cm  Depth 100cm  Height 73cm In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(134, 151, 'PRODUCT'),
    (169, 185, 'PRODUCT'),
    (393, 409, 'PRODUCT')]}),
 ('Designed by Jaime Hayon. Passepartout lamps enhance the visual appeal of whatever they are illuminating. Suitable for every type of architectural setting, these ceiling & wall lamps come in three different shapes where you can choose between either gold or a bronzed brass finish. All 3 variants of Passepartout lights are now set up as dim-to-warm, meaning that as you dim the light, the colour temperature is gradually reduced to a warmer colour. Details Materials Aluminium, polycarbonate, ABSDimensions JH10 Diameter 15.5 cm  Height 17 cmLight Source 11,5 Watt dimmable LED with dim-to-warm technology CRI: >90 Light output: 350 Lumen In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(25, 43, 'PRODUCT')]}),
 ("Designed by Jaime Hayon. &tradition are pleased to present the Palette tables designed by Jaime Hayon. A series of three multi-tiered tables mixing marble, ash veneer, brass and satin brushed stainless steel inspired by Alexander's Calder's kinetic sculptures. Hayon creates a harmonious composite of disparate sizes, shapes, colours and materials which elegantly come together in a cohesive design. The Palette table JH7 juxtaposes a rounded rectangle with an elongated oval and a circle. Details Materials Brass, black Marquina marble and black stained AshDimensions Width 115.2 cm  Depth 67.8 cm  Height 47 cm In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ",
  {'entities': [(63, 77, 'PRODUCT'), (404, 421, 'PRODUCT')]}),
 ('Designed by Jaime Hayon. Dating back centuries, paper lanterns have featured in Asia amongst the privileged and the poor, where different colours, shapes and sizes connote different meanings. Here Hayon has dispensed with cultural formalities to create a series of lanterns hand crafted with rice paper merging various sizes and shapes together. The Paper Lantern series is comprised of three different versions in ivory white rice paper with black stained Oak accents in sizes from 46 cm to 70 cm in diameter. Each weighing only 1 kilo, they hang delicately in any indoor setting. Details Also in the Paper lantern series are the Formakami JH4 and Formakami JH5 . Materials Ivory white paper, black stained OakDimensions Diameter 45cm x Height 65 cm 4m black fabric cordLight Source E27 Max 60 Watt / 12 Watt CFL In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(350, 370, 'PRODUCT'), (602, 622, 'PRODUCT')]}),
 ('Designed by Verner Panton. The Flowerpot lamp became synonymous with the Flower Power movement from the late 60s. Breaking from convention to embrace a more open, modern mentality. Such was the mindset of its designer, Verner Panton, one of the most forward-thinking talents of his time. The Flowerpot series consists of several different models, including three pendants and two table lamps all in a wide range of colours. Details Materials Lacquered steel, brass, stainless steel. Dimensions W23 cm  H49 cm 2m Clear PVC cord Light Source (not included) E27 Max 40 Watt / 7 Watt CFL In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(31, 45, 'PRODUCT')]}),
 ('Designed by Norm Architects. When traveling cities such as Paris, Barcelona and New York, Norm.Architects Kasper Rnn and Jonas Bjerre-Poulsen were fascinted by the street lamps; the characteristic cast iron top with a semi transparent glass sphere at the base. Back in the studio Kasper and Jonas detached the street lamp from its original use and transferred the shape and combination of materials in order to design an outstanding indoor pendant ï¿½ the Mass Light. The well-known shape lets you focus on the beauty of the materials, its aesthetic structures and colours. Details Materials Marble or brass with white acid-etched opal glassDimensions Diameter 12.5 cm  Height 18 cm 4m black fabric cordLight source (not included) G9 max 40 Watt In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(454, 465, 'PRODUCT')]}),
 ('Designed by Arne Jacobsen. Arne Jacobsenï¿½s preference for graceful lines and simple, organic forms are readily recognisable in the Bellevue Lamp. The re-launch of this iconic lamp involves a return to the original Bellevue shape in three styles: floor, table and wall lamp. Bellevue floor lamp is available in four finishes. Details Material Satin polished brass with lacquered aluminium and steel, cast iron baseDimensions AJ7 Diameter base 27.5 cm  Depth 60 cm  Height 130 cm 2m fabric cord with on-off switchLight Source (not included) E27 Max 25 Watt / 12 Watt CFL In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(131, 145, 'PRODUCT'), (274, 293, 'PRODUCT')]}),
 ('With the Bellevue Lamp, Danish architect and designer Arne Jacobsen embodies the minimalism of the Modernist Movement with the functionalism of Bauhaus. Viewed as the driving force behind Danish Modernism, Jacobsenï¿½s preference for graceful lines and simple, organic forms are readily recognisable in the Bellevue Lamp. The re-launch of this iconic lamp involves a return to the original Bellevue shape in three styles: floor, table and wall lamp. Bellevue table lamp is available in four finishes. Details Material Satin polished brass with lacquered aluminium and steel, cast iron baseDimensions Diameter base 17.5 cm  Depth 10 cm  Height 12.5 cm 2m textile cordLight Source (not included) E14 Max 25 Watt / 7 Watt CFL In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(9, 22, 'PRODUCT'),
    (305, 319, 'PRODUCT'),
    (448, 467, 'PRODUCT')]}),
 ("Designed by Space Copenhagen. Endearing in size and shape, the Fly Chair SC10 is the newest addition to the Fly Lounge series. The Fly Lounge Series was created with the ultimate comfort in mind. The lovely loose cushions that comprise the back can be arranged and rearranged to suit your personal preferences. The design alludes to a Nordic aesthetic, such as the slatted back, the handcrafted solid oak frame and the organic shaped elements. Offered here in &Tradition selected timber / Hot Madison fabric combinations. Further upholstery options available under 'Details' tab please select the appropriate price Group and then add a note of your fabric choice & timber finish at Checkout. Details Materials Oiled oak frame and base in White Oiled Oak or Smoked Oiled Oak. CMHR foam and feather pillows.",
  {'entities': [(63, 77, 'PRODUCT')]}),
 ('Designed by Verner Panton. Sometimes the simplest things are the most lasting and memorable, the Topan VP6 Pendant being a brilliant example. It was the first mass produced lamp by Verner Panton in 1959, later to be followed by the FlowerPot. Topanï¿½s simplicity and coolness proves to be of lasting value. The shade is spun into shape from a single piece of aluminium. Available lacquered in five colours with matching textile cord: Light Blue, Mustard, Red Brown, Matt White, Matt Black. Details Materials Lacquered aluminium, 3 meter colour-matched fabric cord. Canopy included. Dimensions H19  21 cm Light source E27 Max 40W (not included) In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(97, 114, 'PRODUCT'), (243, 250, 'PRODUCT')]}),
 ("Designed by All The Way To Paris. The Moor rugs are the second series of rugs designed by All The Way To Paris and produced by &Tradition. This time, ATWTPï¿½s departure point came from the various shapes of gemstones. The result is a compelling sense of cohesion, especially when you arrange all the rugs together as a cluster. There's a subtle shimmer to the surface of each rug, to echo the qualities of a gemstone. Available in four different sizes. AP5 and AP7 have square edges, AP6 and AP8 have a diagonal edge Details Materials Handloom-woven from 85% New Zealand wool and 15% viscose Dimensions AP5: W240  D170 cm AP6: W240  D240 cm AP7: W300  D200 cm AP8: W300  D300 cm In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ",
  {'entities': [(38, 47, 'PRODUCT')]}),
 ('Designed by Anderssen and Voll. The Pavilion chair has an airy feeling, with slim legs that give it a light appearance. Stackable up to 10 chairs version with arms also available. These must be bought as a pair, but are priced individually. Details Materials Steel tubes and formpressed veneer in black lacquered oak, lacquered oak, lacquered walnut. Comes with plastic glides suitable for most floor types. Black base: Black fittings for the front side of the back rest installed as standard. Brass & Chrome fittings available upon request. Chrome base: Chrome fittings for the front side of the back rest installed as standard. Brass & Black fittings available upon request. Dimensions W52.5  D50  H76 cm. Seat height 46 cm In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(36, 50, 'PRODUCT')]}),
 ('Designed by Signe Hytte. The Journey lamp is a lovely merging of two classic geometric shapes, the sphere and the rectangle. The round, opal glass shade is reminiscent of the moon, with a new dimmer feature to enhance your mood and sense of well-being.Journey is both a table lamp and a wall lamp, with subtle colour combinations that make it easy to place anywhere. Details Materials Mouth blown glass and steel. Dimensions L26cm  W18cm  H24cm In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(29, 41, 'PRODUCT')]}),
 ("Designed by Viggo Boesen. The Little Petra lounge chair reflects Viggo Boesen's approach to design ï¿½ with its fluid, organic expression designed to naturally embrace the body. Initially introduced back in 1938, Little Petra won instant praise at the Copenhagen Cabinetmakers Guild Exhibition, subsequently winning awards at exhibits in New York and Berlin. It's one of just a few designs by Viggo Boesen, who was associated with Denmark's signature design aesthetic in the 1930s. A matching Little Petra Pouf and Sofa are now available for an elegant lounge seating arrangement. Details Available in &Tradition selected upholstery options: Kvadrat Hallingdal 130, Dedar Karakorum 003, Scandilock Sheepskin Moonlight 17mm and Scandilock Sheepskin Sahara 17mm with White oiled oak or Oiled walnut. Materials The frame is made from solid beech and plywood and subsequently covered in HR foam and polyester wadding. The solid wood legs are turned and CNC milled in Solid White Oiled oak or Oiled walnut. The legs are fitted with felt glides as standard. Dimensions W81  D83  H75 cm. Seat height 40 cm In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ",
  {'entities': [(30, 55, 'PRODUCT'),
    (211, 223, 'PRODUCT'),
    (491, 517, 'PRODUCT')]}),
 ('Designed by Sami Kallio and Jacob Thau. Named after the Betty Nansen theater in Copenhagen, this stackable chair comes in two versions: one with a seat made from exposed webbing crafted from natural linen fibers, and another with a solid wooden seat. It is available in four contemporary frame colours: Black, Twilight, Maroon, as well as Natural Oak. Betty TK1 must be ordered in pairs price is per chair. Details Materials Solid wood and formpressed veneer with natural linen webbing Dimensions H77  W51  D48 cm Seat height 47 cm In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(352, 361, 'PRODUCT')]}),
 ('Designed by Space Copenhagen. A lighter, narrower version of its predecessor, the Loafer chair is crafted with a higher seat for dining and gently sloping sides to comfortably accommodate your arms. The fully upholstered chair is soft yet sturdy and is fine-tuned for upscale eating environments and private homes alike. . ',
  {'entities': [(82, 94, 'PRODUCT')]}),
 ("Designed by Samuel Wilkinson. The distinctive quilted surface of the Blown table lamp reflects and refracts the light to create an intimate, almost artistic expression. It's crafted from translucent silver-hued glass set on a Nero Marquina marble base with a dimmer switch to adjust the light. Ideal on a bedside table, a desk or windowsill, it illuminates any space to striking effect. Details Materials Mouth blown glass and powder coated metal Dimensions W23  H23 cm. Base 12cm Light source E14 Max 60W (not included) In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ",
  {'entities': [(69, 85, 'PRODUCT')]}),
 ('Designed by Jaime Hayon. Each Palette Desk partners two shapes together: an elongated bean shape with a simple round shape. The two merge a marvellous mix of regal materials available in three different combinations. For the table tops, choose from polished Nero Marquina marble or honed Bianco Carrara marble, satin polished brass or stained ash veneer in black, red or grey-green. The base is made from welded and powder coated steel tubes, adding a contemporary touch to this collage of contrasts. Details Materials Satin polished brass, polished Nero Marquina marble, walnut or lacquered ash veneer, black powder-coated steel. Dimensions L146  W62  H83 cm In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(30, 42, 'PRODUCT')]}),
 ('Designed by Jaime Hayon. Whether by a sofa or bedside, in an entrance, hallway, reception area or part of an in-store display, the Palette side table creates a unified sense of design in a space filled with different design lingos. Details Materials Polished Nero Marquina marble, walnut or black lacquered ash veneer, black powder-coated steel. Dimensions L46  W47  H67 cm In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(131, 149, 'PRODUCT')]}),
 ("Designed by Viggo Boesen. Based on Viggo Boesenï¿½s original drawings from the 1930s, Little Petra VB2 promises the same exceptional craftsmanship as the Lounge Chair & Poufï¿½ but with even more space for you to sink in. From the hand-stitched upholstery to the wooden frame built by hand, Little Petra is made from premium materials. Initially introduced back in 1938, Little Petra won instant praise at the Copenhagen Cabinetmakers Guild Exhibition, subsequently winning awards at exhibits in New York and Berlin. It's one of just a few designs by Viggo Boesen, who was associated with Denmark's signature design aesthetic in the 1930s. Details Available in &Tradition selected upholstery options: Dedar Karakorum 003, Scandilock Sheepskin Moonlight 17mm and Scandilock Sheepskin Sahara 17mm with White oiled oak or Oiled walnut. Materials The frame is made from solid beech and plywood and subsequently covered in HR foam and polyester wadding. The solid wood legs are turned and CNC milled in Solid White Oiled oak or Oiled walnut. The legs are fitted with felt glides as standard. Dimensions W150  H75  D80 cm In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ",
  {'entities': [(84, 100, 'PRODUCT'),
    (152, 172, 'PRODUCT'),
    (287, 299, 'PRODUCT'),
    (367, 379, 'PRODUCT')]}),
 ('Designed by Sami Kallio and Jacob Thau. Named after the Betty Nansen Theatre in Copenhagen, the new TK2 version of the Betty chair is available with a solid wooden seat and in four different finishes. Stackable up to 6 chairs high. The Betty TK1 chair with exposed linen webbing is available here. Details Materials Lacquered solid wood and form-pressed veneer. Available in Lacquered Oak, Black, Twilight & Maroon. Felt gliders installed as standard. Dimensions H77  W51  D48 cm Seat height 47 cm In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(119, 130, 'PRODUCT'), (236, 251, 'PRODUCT')]}),
 ("Designed by Viggo Boesen. Soft and nurturing, the Little Petra Pouf is an ideal companion. With rounded legs in either oak walnut, and upholstery in a wide variety of fabrics that include sheepskin, it lends an organic warmth to any interior. Initially introduced back in 1938, Little Petra won instant praise at the Copenhagen Cabinetmakers Guild Exhibition, subsequently winning awards at exhibits in New York and Berlin. It's one of just a few designs by Viggo Boesen, who was associated with Denmark's signature design aesthetic in the 1930s. Pair with the Little Petra Lounge Chair or Sofa for an elegant lounge seating arrangement. Details Available in &Tradition selected upholstery options: Kvadrat Hallingdal 130, Dedar Karakorum 003, Scandilock Sheepskin Moonlight 17mm and Scandilock Sheepskin Sahara 17mm with White oiled oak or Oiled walnut. Materials The frame is made from solid beech and plywood and subsequently covered in HR foam and polyester wadding. The solid wood legs are turned in Solid White Oiled oak or Oiled walnut. The legs are fitted with felt glides as standard. Dimensions W53  D53  H75 cm. Seat height 37 cm In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ",
  {'entities': [(50, 67, 'PRODUCT'),
    (278, 290, 'PRODUCT'),
    (561, 594, 'PRODUCT')]}),
 ('Designed by Sami Kallio and Jacob Thau. Named after the Betty Nansen Theatre in Copenhagen, the latest TK3 iteration of the Betty chair is available with a solid wooden frame in four different finishes, and an upholstered seat for superior comfort. See also the Betty TK1 chair with exposed linen webbing and unupholstered Betty TK2 chair.  Details Materials Lacquered solid wood and formpressed veneeer with upholstered HR foam seat. Available in Lacquered Oak, Black, Twilight & Maroon. Felt gliders installed as standard.',
  {'entities': [(124, 135, 'PRODUCT'),
    (262, 277, 'PRODUCT'),
    (323, 339, 'PRODUCT')]}),
 ('Designed by Sami Kallio and Jacob Thau. Named after the Betty Nansen Theatre in Copenhagen, this bench is made from exposed webbing hand-crafted from natural linen fibers. Its solid oak frame is available in three contemporary colours: Black, Twilight, and Maroon, and comes in two lengths: 105cm and 157cm. See Betty TK1 for coordinating chairs. Details Materials Made from lacquered solid wood. Seat is webbed by hand with natural linen. Felt glides provided as standard. Dimensions TK4 W105cm  D36cm x H46cm TK5 W157cm  D36cm  H46cm In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(312, 321, 'PRODUCT')]}),
 ('Designed by Sami Kallio. The In Between Lounge Table series carefully echoes many aspects of the In Between chairs and dining tables, from their gentle curvature to the outward splay of the legs. There are three variations of proportion available and all come in three finishes. Details Materials Solid oak treated with either lacquer or oil. Dimensions SK13 D48cm  H48cm SK14 D60cm  H40cm SK15 D90cm  H48cm In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(29, 52, 'PRODUCT'), (97, 132, 'PRODUCT')]}),
 ('The 1956 Drawn chair by Hvidt & Mlgaard stands out as a definitive piece of Danish design. Relying upon traditional craftsmanship techniques and built out of organic materials, it brings a sense of wholesome honesty to any space. Available in oak or walnut, it comes in two versions: a wider model with armrests HM4, or one without HM3. Peter Hvidt (1916-1986) and Orla Mlgaard-Nielsen (1907-1993) were pioneers of Danish mid-century design and the founders of Copenhagen-based firm Hvidt & Mlgaard. Details Materials The chair is made from turned and milled solid wood with a formpressed backrest. The seat is hand-woven from more than 100 meter natural paper cord Dimensions H78cm  W49cm  D46cm Seat Height 46cm Seat Depth: 42cm In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title Read More Made to Order 6-8 weeks This item is made specially for you in the options you choose and special terms apply. ',
  {'entities': [(4, 20, 'PRODUCT'), (522, 527, 'PRODUCT')]}),
 ('The 1956 Drawn chair by Hvidt & Mlgaard stands out as a definitive piece of Danish design. Relying upon traditional craftsmanship techniques and built out of organic materials, it brings a sense of wholesome honesty to any space. Available in oak or walnut, it comes in two versions: a wider model with armrests HM4, or one without HM3. Peter Hvidt (1916-1986) and Orla Mlgaard-Nielsen (1907-1993) were pioneers of Danish mid-century design and the founders of Copenhagen-based firm Hvidt & Mlgaard. Details Materials The chair is made from turned and milled solid wood with a formpressed backrest. The seat is hand-woven from more than 100 meter natural paper cord Dimensions H78cm  W59cm  D54cm Seat Height 46cm Seat Depth: 49cm In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title Read More Made to Order 6-8 weeks This item is made specially for you in the options you choose and special terms apply. ',
  {'entities': [(4, 20, 'PRODUCT'), (522, 527, 'PRODUCT')]}),
 ('This extendable dining table by Hee Welling employs a butterfly extension technique to provide seating for more people. Its wooden base is topped with a smooth laminate tabletop. Drop-shaped legs and rounded corners add subtle design details, as do the metal patches on each leg that hold it securely together. The table comes in two sizes: HW1 180  90cm, or HW2 240  100cm and are both extendable by 100cm. Details Materials Tabletop in Fenix Nano Laminate in selected colours. Solid wood frame and legs in White Oiled Oak, Smoked Oiled Oak or Oiled Walnut. Extension leaves with veneered and chamfered edges. Brass fittings. Dimensions HW1 W180cm (280cm)  D90cm  H75 HW2 W240cm (340cm)  D100cm  H75 In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title Read More Made to Order 6-8 weeks This item is made specially for you in the options you choose and special terms apply. ',
  {'entities': [(5, 28, 'PRODUCT'), (315, 320, 'PRODUCT')]}),
 ('Designed by Luca Nichetto. At first glance, Lato resembles a sculpture, with its slim, oval or round table top balanced by an oval-shaped base. Striking, graphic and poetic, its purity of form is proof that simple is sophisticated. Now available with luxurious timber tabletops. Details Materials Base in honed, semi-matt marble. Top in laser cut and milled powder coated steel, or FSC 100% certified lacquered wood. Finishes Warm Black Steel top with Emperador marble base Ivory White Steel top with Crema Diva marble base Lacquered Walnut with Emperador marble base Lacquered Oak with Crema Diva marble base Dimensions LN8 Round table top 40, H50 cm LN9 Oval table top 50, H45 cm In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title Read More Made to Order 6 weeks This item is made specially for you in the options you choose and special terms apply. ',
  {'entities': [(44, 48, 'PRODUCT')]}),
 ('Designed by Space Copenhagen. Match lighting to mood with Como SC53, a portable table lamp from Space Copenhagen. Crafted from anodised aluminium, Comoï¿½s sturdy base tapers up towards a softy curved, mushroom-shaped shade. This battery-powered lamp can operate for eleven hours at the highest setting, with an extra battery option that allows additional operating. It is easily recharged with a magnetic USB cable or a charging tray. Details Materials Extruded anodised aluminium with bronzed brass, black or brass finish. Dimensions H21 x 15.6 cm Light Source 1W LED light bulb / 100 lumen / 2700K Life time: 25,000 hours Magnetic micro USB Charging Cable: 200cm In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(58, 67, 'PRODUCT'), (147, 153, 'PRODUCT')]}),
 ('Designed by Space Copenhagen. Ensure the ambiance is right with Manhattan SC52, a portable table lamp from Space Copenhagen. Its industrial-inspired base is made from anodised aluminium with a columned detailing; a cream-coloured canvas shade sits atop. This battery-powered lamp can operate for eleven hours at the highest setting, with an extra battery option that allows additional operating. It is easily recharged with a magnetic USB cable or a charging tray. Details Materials Extruded anodised aluminium with bronzed brass finish. Canvas shade. Dimensions H24.5  Shade 13 cm x Base 8.4 cm Light Source 1W LED light bulb / 100 lumen / 2700K Life time: 25,000 hours Magnetic micro USB Charging Cable: 200cm In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(64, 78, 'PRODUCT')]}),
 ('Designed by Verner Panton. The new vividly colourful and portable Flowerpot VP9 embodies the experimental 1960s attitude of Verner Panton. Slightly smaller in size than the table lamp version, and deliberately lightweight, the VP9 comes equipped with a USB charging cable. It is available in a wide range of 10 contemporary colours and will bring a cozy atmosphere to indoor and outdoor settings. Details Dimmable by 3 level touch dimmer: 100% ï¿½ 50% ï¿½ 20 % Charging time: 8 hours Operating time in 100% brightness: 11 hours Materials The base and the lamp shade are moulded in Polycarbonate. The dimmer is made from steel. Dimensions 16cm  H29.5cm Light Source 3W integrated LED, 2700k, 325 lumen 200 cm Micro USB charging cable In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(66, 79, 'PRODUCT')]}),
 ('Designed by Hvidt and Mlgaard. Tripod Floor Lamp from 1953 reflects the clean minimalism of Old Masters Hvidt & Mlgaard and is now in production for the first time. The shade is spun from a single piece of aluminium. The base is made from steel tubes, bent and welded together. The lamp is lacquered in a satin matt finish either in Black or Maroon. Peter Hvidt (1916-1986) and Orla Mlgaard-Nielsen (1907-1993) were pioneers of Danish mid-century design and the founders of Copenhagen-based firm Hvidt & Mlgaard. Also see Tripod HM9 Table Lamp Details Materials Lacquered metal shade and base in either Maroon (RAL 8012) or Black (RAL 9005). Lacquered brass detail. 2m fabric cord with on/off switch. Dimensions H134  W47  D63cm 2m fabric cord Light Source E27 Max 60 Watt (not included) In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(31, 48, 'PRODUCT'), (282, 286, 'PRODUCT')]}),
 ('Designed by Space Copenhagen. Space Copenhagenï¿½s ambition with their Copenhagen lighting was to design lamps with a simple industrial feel, but which were still elegant and poetic. The Copenhagen lighting range is an exercise in contrasts. Combining the classic and the modern, the maritime and the industrial. The series started with one pendant in three sizes in 2014 and was extended in 2018 with two table lamps, one wall lamp and one floor lamp. Details Materials Mouth blown satinized opal glass shade. Bronzed brass base and stem. 2m Black textile cord with dimmer switch. Dimensions Glass shade: 16  H12.5 cm Baseplate: . 9  H10 cm Projection depth: 21.65 cm Light Source (not included) G9 max 25W In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(185, 210, 'PRODUCT')]}),
 ('Designed by Space Copenhagen. Space Copenhagenï¿½s ambition with their Copenhagen lighting was to design lamps with a simple industrial feel, but which were still elegant and poetic. The Copenhagen lighting range is an exercise in contrasts. Combining the classic and the modern, the maritime and the industrial. The series started with one pendant in three sizes in 2014 and was extended in 2018 with two table lamps, one wall lamp and one floor lamp. Details Materials Mouth blown satinized opal glass shade. Bronzed brass base and stem. Dimmer switch on baseplate. Dimensions Glass shade: 33cm Base: 17.5  H42cm 350cm PVC Light Source (Integrated) 8 Watt Integrated LED board, Dimmable. Color temperature (Kelvin): 2200K-2800K Light output: 700 lumen In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(185, 210, 'PRODUCT')]}),
 ('Designed by Verner Panton. The Flowerpot lamp became synonymous with the Flower Power movement from the late 60s. Breaking from convention to embrace a more open, modern mentality. Such was the mindset of its designer, Verner Panton, one of the most forward-thinking talents of his time. The Flowerpot series consists of several different models, including three pendants and two table lamps all in a wide range of colours. Details Available in White, Matt Light Grey, Grey Beige, Dark Green, Mustard, Beige Red, Red Brown, Light Blue, Matt White, Matt Black, Stone Blue, Signal Green and Pale Sand. Materials Lacquered steel Dimensions 37 cm  H27 cm 3m fabric cord Light Source (not included) E27 Max 100 Watt / 5 watt LED recommended In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(31, 45, 'PRODUCT')]}),
 ('Designed by Jaime Hayon. Named after the Spanish word for mushroom (ï¿½setaï¿½) and the suffix ï¿½goï¿½, this quirky portable lamp is unburdened by an electrical cord and can literally move anywhere. Additional design features include a discreet gold dimmer and a stylish upward curve on the lamp shadeï¿½s brim. Details Charging time: 8 hours Operating time in highest light setting: 10 hours Dimmability: 3 level touch dimmer on base 100% 70% 40 Material The base and the lamp shade are moulded in polycarbonate (PC). The dimmer button is made from brass. Dimensions Base 14 cm  Shade 23 cm  H22 cm Light Source G4 Max 3 Watt LED bulb, dimmable LED bulb included replaceable (spare part) Micro USB charging cable 140cm/55.1in, included Environment This lamp is for indoors use only. It can be taken outdoors, but must be kept and stored indoors. In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(109, 122, 'PRODUCT')]}),
 ('Designed by Hvidt and Mlgaard. After the success of the Tripod HM8 Floor Lamp, &Tradition has expanded the Tripod series to include a table lamp with the same timeless design. Tripod, a table lamp built from powder-coated steel with brass details, reflects the clean minimalism of historic Copenhagen-based duo Hvidt & Mlgaard. Considered Old Masters of mid-century Danish design, their pieces often lend a sculptural element to the spaces they inhabit. Details Colours This scaled-down table lamp version is available in three colours: Black, Maroon, and Moss. Materials Lacquered metal shade and base, lacquered brass, 2 meter fabric cord with on/off switch. Dimensions H 45 cm  W 26 cm  D 32 cm Light Source E14 Max 25 Watt In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(56, 77, 'PRODUCT'), (176, 182, 'PRODUCT')]}),
 ("Introducing Wulff, a lounge chair whose upholstered form pays tribute to the hand-crafted designs of the 1930's. Wulff promises superior craftsmanship and outstanding comfort with luxuriantly soft upholstery that covers its seat and backrest. Intended as a lounge chair for the living room or bedroom, its architectural shape will add a stylish statement to any interior. Details Materials Oiled wood, HR foam, polyester wadding, and fabric upholstery.Frame Oiled Oak Oiled WalnutDimensions H: 85 cm X W85 cm X D84 cm Seat height: 41 cm Armrest height: 59 cm Fabric Ranges: Quickship (2-4 weeks) Fabric Ranges: Made To Order (6-8 weeks)Group 1 Hallingdal 376 Karakorum 003 Karandash 005 Sheepskin Moonlight 17mm Group 2 Group 3 Group 4 Group 5 In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ",
  {'entities': [(12, 17, 'PRODUCT'), (113, 118, 'PRODUCT')]}),
 ('Designed by Luca Nichetto. The Cloud range of furniture combines the the defined shapes of the Danish design tradition and the soft and cosy pillows of the Italian one. Ultimately, the highest priority has been comfort and cosiness. Because of the compact dimensions of the furniture in the Cloud series, they are perfectly adaptable to contemporary living spaces. Cloudï¿½s base feature no-sag springs and the pillows are made using a mix of HR foam and feathers, providing ultimate comfort. Details Materials Plywood frame, solid wood and steel base with no-zag springs, HR foam and feather pillows, upholstery with leg options in either powder coated black or chromed steel.',
  {'entities': [(31, 55, 'PRODUCT'), (291, 303, 'PRODUCT')]}),
 ('Designed by Arne Jacobsen and Flemming Lassen. The Mayor sofa was created for Sollerod City Hall in 1939 by Danish architects Arne Jacobsen and Flemming Lassen. The tight upholstery and the two rows of elegant buttons that extend over the armrests give the sofa a simple and stylish look with two loose cushions that comprise the seating area. The Mayor sofa has won the award for ï¿½Best Reissueï¿½ï¿½ in the prestigious Wallpaper* Design Award 2013. Made to order in a choice of fabric ranges. Add a note of your colour choice at checkout or email us. Details Materials Solid oak base available in three finishes: white oiled, smoked oiled or black stained. Kvadrat fabric or CA-MO leather upholstery. Dimensions H82cm  D62cm  L138cm. Seat height 44 cm',
  {'entities': [(51, 61, 'PRODUCT'), (348, 358, 'PRODUCT')]}),
 ('Debuted in 1956, the lounge chair by duo Hvidt & Mlgaard has a no-frills, streamlined form. From the hand-polished wooden frame, to its slender brass legs, the Boomerang endures as a masterpiece of Danish design. Version with armrests also available Boomerang HM2. Details Materials Solid wood, brass, HR foam, polyester wadding and fabric upholstery Oiled Oak, Soaped Oak or Oiled Walnut frame.',
  {'entities': [(160, 169, 'PRODUCT'), (250, 264, 'PRODUCT')]}),
 ('Debuted in 1956, the lounge chair by duo Hvidt & Mlgaard has a no-frills, streamlined form. From the hand-polished wooden frame, to its slender brass legs, the Boomerang endures as a masterpiece of Danish design. Version without armrests also available Boomerang HM1. Details Materials Solid wood, brass, HR foam, polyester wadding and fabric upholstery Oiled Oak, Soaped Oak or Oiled Walnut frame & armrests.',
  {'entities': [(160, 169, 'PRODUCT'), (253, 267, 'PRODUCT')]}),
 ('Designed by Sami Kallio. The In Between Lounge Table series carefully echoes many aspects of the In Between chairs and dining tables. These new SK23 & SK24 rectangular additions to the lounge table range provide a practical console for many rooms in the home. Details Materials Solid oak treated with either lacquer or oil. Dimensions SK23 H48cm  W50cm  D110cm In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title This item is made specially for you and special terms apply (see Terms and Conditions). ',
  {'entities': [(29, 52, 'PRODUCT'), (97, 133, 'PRODUCT')]}),
 ('Award-winning design duo Anderssen & Voll drew their design cues from the iconic Langelinie Pavilion in Copenhagen. This light and lyrical aesthetic has been translated into a compact desk intended to suit every home office space. Also available with back Pavilion Desk AV17 Details Materials Steel, wood veneer, formpressed wood veneer, Forbo linoleum. Plastic gliders supplied as standard.Dimensions H74cm  W132cm  D66cm Tabletop height: 74cm In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(256, 274, 'PRODUCT')]}),
 ('Drip is a highly versatile and practical dining table made from 80% recycled aluminium. Easily assembled within a few minutes, Dripï¿½s base comes in three options: black, bronzed, or a polished aluminium. The tabletop is available in durable Nanolaminate or timber veneer. Please enquire for optional cable management. Details Materials Aluminium base, MDF table top with Fenix Nano laminate or Oak veneer with matching colour high gloss edge bands for laminate and matching edge bands for wood veneer.Dimensions W140cm  D80cm  H74cm In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(0, 4, 'PRODUCT'), (127, 133, 'PRODUCT')]}),
 ("Designed by Edward van Vliet. Develius is Edward van Vlietï¿½s idea of the archetypal modular sofa. With its elongated seat, itï¿½s reminiscent of a chaise longue, inviting you to luxuriate at length for hours on end. The modular system is composed of individual sofa components, pouf, and lush, loose pillows. Two depths are available. Select from the modules to design your own piece of paradise. Click here to see & download Develius Modules PDF. Made to order in a choice of fabric ranges. Add a note of your colour choice at checkout or email us. We'd be delighted to help with your configuration design. Details Materials Wooden frame, no-zag springs, CMHR foam and polyester wadding. Metal brackets for connecting modules.Dimensions Overall dimensions vary depending on configuration. EV1 modules: Seating Depth: 70cm / 27.2in, Seating Height: 42cm / 16.5in, Armrest Height: 60cm / 23.6in, Backrest Height: 70cm / 27.6in EV2 modules: Seating Depth: 91cm / 35.8in, Seating Height: 42cm / 16.5in, Armrest Height: 60cm / 23.6in, Backrest Height: 70cm / 27.6in View fabrics Group 1 Re-Wool click here to view fabrics Remix click here to view fabrics Canvas click here to view fabrics Group 2 Hallingdal 65 click here to view fabrics Linara click here to view fabrics Divina click here to view fabrics Divina MD click here to view fabrics Divina Melange click here to view fabrics Fiord click here to view fabrics Steelcut click here to view fabrics Steelcut Trio click here to view fabrics Maple click here to view fabrics Group 3 &Tradition Velvet click here to view colours Ruskin click here to view fabrics Balder click here to view fabrics Colline click here to view fabrics Twill Weave click here to view fabrics Vidar click here to view fabrics Group 4 Clay click here to view fabrics Fuse click here to view fabrics Karakorum click here to view fabrics Sonar click here to view fabrics Ria click here to view fabrics Zero click here to view fabrics In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ",
  {'entities': [(84, 97, 'PRODUCT')]}),
 ('1  Copenhagen Pendant SC7 / black. New condition. Space Copenhagenï¿½s ambition with the Copenhagen pendant was to design a lamp with a simple industrial feel, but which was still elegant and poetic. The Copenhagen Pendant is an exercise in contrasts. Combining the classic and the modern, the maritime and the industrial. The series started with one pendant in three sizes in 2014 and was extended in 2018 with two table lamps, one wall lamp and one floor lamp. Details Materials & available colours Lacquered steel shade with bronzed brass suspension. Dimensions SC7: D350mm  H400mm Light Source (not included) SC7: E27 Max 100 Watt light bulb 4m fabric cord In Stock In stock and ready to despatch! UK delivery usually 3-7 days. Larger items and non mainland addresses will take a little longer. Quick Ship false weeks Good news! This popular item is usually available on a quick lead time in the finish you selected (other options may have different lead time). Default two man delivery notice Default two man delivery information Bespoke terms & Conditions Title ',
  {'entities': [(3, 25, 'PRODUCT'),
    (87, 105, 'PRODUCT'),
    (202, 220, 'PRODUCT')]}),
 ('Susy side table is handwoven with weatherproof maintenance-free Art Fibers on a strong hand-bent aluminum frame. The garden table is made in the colour antique, which is original and suitable for all outdoor environments. The Georgia Garden collection is inspired by the British colonial style and by the old Danish wicker furniture. The Susy table is a Danish design by Sika Design. Susy is strong, lovely and very suitable for private use as well as in hotels and restaurants.The Georgia Garden collection is made for life in the garden and the cozy wicker furniture requires no maintenance. Then there is time and for lots of cozy moments with friends and family on the terrace. These moments together with good friends remind us all what really matters in a busy weekday. Width: 55 cm | 21.7 inches * Depth: 55 cm | 21.7 inches * Height: 51 cm | 20.1 inches * Weight: 5.3 kg | 0 lbs * * ',
  {'entities': [(0, 15, 'PRODUCT'),
    (338, 348, 'PRODUCT'),
    (384, 388, 'PRODUCT')]}),
 ('The Dawn Exterior Foot Stool is designed by Sika-Design and is part of our romantic Georgia Garden collection. The Georgia Garden collection is inspired by the British colonial style and by classical Danish wicker furniture. The Dawn stool is strong, comfortable and very suitable for private use as well as for use in hotels and restaurants. Use the Dawn foot stool with a Dawn lounge chair, or as an extra seat. The Dawn foot stool is handcrafted of aluminum and fully woven with our weather-resistant ArtFibre. It is designed to stand in the open air and to make garden life as easy as possible. It can be left outdoor all year round and requires no maintenance.',
  {'entities': [(4, 28, 'PRODUCT'),
    (229, 239, 'PRODUCT'),
    (351, 366, 'PRODUCT'),
    (374, 391, 'PRODUCT')]}),
 ('The Hazel coffee table with glass top is designed by Sika-Design and is part of our romantic Georgia Garden collection. The Georgia Garden collection is inspired by the British colonial style and by classical Danish wicker furniture. The coffee table is strong and very suitable for private use as well as for use in hotels and restaurants. It is handcrafted of aluminum and fully woven with our weather-resistant ArtFibre. The tables glass top is made from 8 mm tempered glass which makes it easy to clean and more durable than ordinary glass. The Hazel coffee table is designed to stand in the open air and to make garden life as easy as possible. It can be left outdoor all year round and requires no maintenance.',
  {'entities': [(4, 22, 'PRODUCT'),
    (238, 250, 'PRODUCT'),
    (549, 567, 'PRODUCT')]}),
 ('The Anna Exterior side table with glass top is designed by Sika-Design and is part of our romantic Georgia Garden collection. The Georgia Garden collection is inspired by the British colonial style and by classical Danish wicker furniture. The side table is strong and very suitable for private use as well as for use in hotels and restaurants. It is handcrafted of aluminum and fully woven with our weather-resistant ArtFibre. The tables glass top is made from 8 mm tempered glass which makes it easy to clean and more durable than ordinary glass. The Anna side table is designed to stand in the open air and to make garden life as easy as possible. It can be left outdoor all year round and requires no maintenance.',
  {'entities': [(4, 28, 'PRODUCT'),
    (244, 254, 'PRODUCT'),
    (553, 568, 'PRODUCT')]}),
 ("Caroline table is designed by Sika-Design and is made in Alu-Rattan and ArtFibre materials. Alu-Rattan is weatherproof powder coated aluminum and ArtFibre is artificial fibers made of maintenance-free polyethylene that can withstand the sun's UV radiation. Due to the weatherproof materials, the furniture can withstand being outside throughout the year without maintenance. The fine bindings in the seat and back are made on a frame of aluminum of skilled wicker workers. The Caroline table contributes to a naturally beautiful element in any outdoor environment. Caroline has with the simple bindings a very elegant expression. Width: 60 cm | 23.7 inches * Depth: 60 cm | 23.7 inches * Height: 43 cm | 17 inches * Weight: 6.2 kg | 0 lbs * * ",
  {'entities': [(0, 14, 'PRODUCT'), (477, 491, 'PRODUCT')]}),
 ("FA-E16-DO Ottoman puff size large is designed by Franco Albini and is made in Alu-rattan and ArtFibre materials. Ottoman is made in the colour dove white, which is fine and elegant. Alu-Rattan is weatherproof powder coated aluminum and ArtFibre is artificial fibers made of maintenance-free polyethylene that can withstand the sun's UV radiation. Due to the weatherproof materials, the furniture can withstand being outside throughout the year without maintenance. The fine bindings in the seat are made on a frame of aluminum of skilled wicker workers. The Ottoman is available in two sizes. Franco Albini (1905-1977) was both an architect and designer and had his heyday in the 1950s, producing many acclaimed furniture designs of, among other things, the furniture design. Width: 65 cm | 25.6 inches * Depth: 65 cm | 25.6 inches * Height: 40 cm | 15.8 inches * Weight: 4.4 kg | 0 lbs * * ",
  {'entities': [(0, 17, 'PRODUCT'),
    (113, 120, 'PRODUCT'),
    (558, 565, 'PRODUCT')]}),
 ('The round Grace garden table with glass top is designed by Sika-Design and is part of our romantic Georgia Garden collection. The Georgia Garden collection is inspired by the British colonial style and by classical Danish wicker furniture. The garden table is strong and very suitable for private use as well as for use in hotels and restaurants. It is handcrafted of aluminum and fully woven with our weather-resistant ArtFibre. The tables glass top is made from 8 mm tempered glass which makes it easy to clean and more durable than ordinary glass. The table fits 4 persons around. The Grace table is designed to stand in the open air and to make garden life as easy as possible. It can be left outdoor all year round and requires no maintenance.',
  {'entities': [(10, 28, 'PRODUCT'),
    (244, 256, 'PRODUCT'),
    (588, 599, 'PRODUCT')]}),
 ('The Carrie Exterior 3-seater sofa is designed by Sika-Design and is part of our romantic Georgia Garden collection. The Georgia Garden collection is inspired by the British colonial style and by classical Danish wicker furniture. The Carrie sofa is strong, comfortable and very suitable for private use as well as for use in hotels and restaurants. It is handcrafted of aluminum and fully woven with our weather-resistant ArtFibre. The Carrie 3-seater sofa is designed to stand in the open air and to make garden life as easy as possible. It can be left outdoor all year round and requires no maintenance. Cushions can be ordered in optional fabric, handcrafted and made to order in our Danish sewing department.',
  {'entities': [(4, 33, 'PRODUCT'),
    (234, 245, 'PRODUCT'),
    (436, 456, 'PRODUCT')]}),
 ('Our George Teak Extendable Table is a beautiful wooden garden table. The table has a Dutch extension, where the extension plate is integrated in the table. On our outdoor tables the wooden boards are placed with a small space between so the wood can expand and contract depending on the weather. Over time teak furniture exposed to sun and rain will turn grey. The table requires no maintenance but if you want to preserve the dark brown color, the teak must be treated with oil. The table is made of teak, which is especially known for its high strength and beautiful glow. Due to its natural oil content, the wood is very durable, resistant to wind and weather, and is not attacked by insects and rot. Therefore, our outdoor teak furniture can withstand all kinds of weather and be left outside all year round. At Sika-Design, we only use premium teak from Indonesia for our teak furniture. Our exterior teak products are made using plantation-grown teak marked with the Indonesian Legal Woodï¿½ stamp. This teak sourcing is overseen by the Indonesian government, making sure that no tree too immature or too small, is removed from the forest. Without extension the table measures 200 cm. With the extension plate the table measures 280 cm. and will be able to accommodate 8-10 people. The table is also available in a non-extendable version in the size 240x100 cm.',
  {'entities': [(4, 32, 'PRODUCT'),
    (484, 489, 'PRODUCT'),
    (1218, 1223, 'PRODUCT'),
    (1290, 1295, 'PRODUCT')]}),
 ('Our George Bench is a beautiful wooden bench made of teak wood which is known for its high strength and beautiful glow. Due to its natural oil content, the wood is very durable, resistant to wind and weather, and is not attacked by insects and rot. Therefore, our outdoor teak furniture can withstand all kinds of weather and be left outside all year round. On our exterior teak furniture the wooden boards are placed with a small space between so the wood can expand and contract depending on the weather. Over time teak furniture exposed to sun and rain will turn grey. The bench requires no maintenance but if you want to preserve the dark brown color, the teak must be treated with oil. At Sika-Design we only use premium teak from Indonesia for our teak furniture. Our exterior teak products are made using plantation-grown teak marked with the Indonesian Legal Woodï¿½ stamp. This teak sourcing is overseen by the Indonesian government, making sure that no tree too immature or too small, is removed from the forest. The George teak bench measures 220 cm and fits 4 persons.',
  {'entities': [(4, 16, 'PRODUCT'),
    (576, 581, 'PRODUCT'),
    (1025, 1042, 'PRODUCT')]}),
 ('Our Julian Coffee Table is a beautiful wooden table made of teak wood which is known for its high strength and beautiful glow. On our outdoor tables the wooden boards are placed with a small space between so the wood can expand and contract depending on the weather. Over time teak furniture exposed to sun and rain will turn grey. The table requires no maintenance but if you want to preserve the dark brown color, the teak must be treated with oil. Due to its natural oil content, the wood is very durable, resistant to wind and weather, and is not attacked by insects and rot. Therefore, our outdoor teak furniture can withstand all kinds of weather and be left outside all year round. At Sika-Design we only use premium teak from Indonesia for our teak furniture. Our exterior teak products are made using plantation-grown teak marked with the Indonesian Legal Woodï¿½ stamp. This teak sourcing is overseen by the Indonesian government, making sure that no tree too immature or too small, is removed from the forest. The Julian Coffee Table measures 140x55 cm and looks beautiful in outdoor lounge areas.',
  {'entities': [(4, 23, 'PRODUCT'),
    (134, 148, 'PRODUCT'),
    (336, 341, 'PRODUCT'),
    (1023, 1042, 'PRODUCT')]}),
 ("ND-E25-NAT Chill lounge chair was designed by Nanna Ditzel and is made in Alu-Rattan and ArtFibre materials. The armchair is made in the colour natural which is original. Alu-Rattan is weatherproof powder coated aluminum and ArtFibre is artificial fibers made of maintenance-free polyethylene that can withstand the sun's UV radiation. Due to the weatherproof materials, the furniture can withstand being outside throughout the year without maintenance. The fine bindings in the seat and back are made on a frame of aluminum of skilled wicker workers. Chill contributes to a naturally beautiful element in any outdoor environment. The armchair is strong, comfortable and suitable for use privately. Chill, which was originally named Kaminstol, is a sculptural installation that almost fluently integrates the furniture with the body and with the surroundings. It was originally used in collaboration with Robert W., who was the pioneer of rattan production in Denmark and characterized by exceptional craftsmanship as well as innovative ideas and the use of materials. Width: 69 cm | 27.2 inches * Depth: 75 cm | 29.6 inches * Height: 55 cm | 21.7 inches * Seat height: 17 cm | 6.7 inches * Weight: 14 kg | 0 lbs * * ",
  {'entities': [(0, 29, 'PRODUCT'),
    (113, 121, 'PRODUCT'),
    (635, 643, 'PRODUCT'),
    (699, 704, 'PRODUCT')]}),
 ('The Carrie Exterior lounge chair is designed by Sika-Design and is part of our romantic Georgia Garden collection. The Georgia Garden collection is inspired by the British colonial style and by classical Danish wicker furniture. The Carrie chair is strong, comfortable and very suitable for private use as well as for use in hotels and restaurants. It is handcrafted of aluminum and fully woven with our weather-resistant ArtFibre. The Carrie Exterior lounge chair is designed to stand in the open air and to make garden life as easy as possible. It can be left outdoor all year round and requires no maintenance. Cushions can be purchased for the Carrie chair in any optional fabric, handcrafted and made to order in our Danish sewing department.',
  {'entities': [(4, 32, 'PRODUCT'),
    (233, 245, 'PRODUCT'),
    (436, 464, 'PRODUCT'),
    (648, 654, 'PRODUCT')]}),
 ('The James Exterior Trolley is designed by Sika-Design and is part of our romantic Georgia Garden collection. The Georgia Garden collection is inspired by the British colonial style and by classical Danish wicker furniture. The James Trolley is strong and very suitable for private use as well as for use in hotels and restaurants. The trolley is fitted with wheels and is easy to move around. Use the trolley to serve drinks on a hot summer day or decorate it with flowers and vases as a neat decoration. James is handcrafted of aluminum and fully woven with our weather-resistant ArtFibre. The outdoor bar trolley is designed to stand in the open air and to make garden life as easy as possible. It can be left outdoor all year round and requires no maintenance.',
  {'entities': [(4, 26, 'PRODUCT'),
    (227, 240, 'PRODUCT'),
    (335, 342, 'PRODUCT'),
    (401, 408, 'PRODUCT'),
    (595, 614, 'PRODUCT')]}),
 ('The Victoria Exterior coffee table with glass top is designed by Sika-Design and is part of our romantic Georgia Garden collection. The Georgia Garden collection is inspired by the British colonial style and by classical Danish wicker furniture. The coffee table is strong and very suitable for private use as well as for use in hotels and restaurants. It is handcrafted of aluminum and fully woven with our weather-resistant ArtFibre. The tables glass top is made from 8 mm tempered glass which makes it easy to clean and more durable than ordinary glass. The Victoria coffee table is designed to stand in the open air and to make garden life as easy as possible. It can be left outdoor all year round and requires no maintenance.',
  {'entities': [(4, 34, 'PRODUCT'),
    (250, 262, 'PRODUCT'),
    (440, 446, 'PRODUCT'),
    (561, 582, 'PRODUCT')]}),
 ('Our George Dining Table is a beautiful wooden table made of teak wood which is known for its high strength and beautiful glow. On our outdoor tables the wooden boards are placed with a small space between so the wood can expand and contract depending on the weather. Over time teak furniture exposed to sun and rain will turn grey. The table requires no maintenance but if you want to preserve the dark brown color, the teak must be treated with oil. Due to its natural oil content, the wood is very durable, resistant to wind and weather, and is not attacked by insects and rot. Therefore, our outdoor teak furniture can withstand all kinds of weather and be left outside all year round. At Sika-Design we only use premium teak from Indonesia for our teak furniture. Our exterior teak products are made using plantation-grown teak marked with the Indonesian Legal Woodï¿½ stamp. This teak sourcing is overseen by the Indonesian government, making sure that no tree too immature or too small, is removed from the forest. The garden table measures 240x100 cm and fits 8 persons. The table is also available in an extendable version in the size 200/280x100 cm.',
  {'entities': [(4, 23, 'PRODUCT'),
    (336, 341, 'PRODUCT'),
    (1030, 1035, 'PRODUCT'),
    (1080, 1085, 'PRODUCT')]}),
 ("Valerie is a charming outdoor dining chair and part of Sika-Design's Alu-Affire collection. Our Alu-Affire collection represents a unique selection of outdoor bistro and caf furniture. This furniture is inspired by Sika-Design's own designs from the 1950s and by the charming Parisian bistro atmosphere. Our Alu-Affire chairs are handwoven with ArtFibre, which is an extremely durable and weather-resistant material made of polyethylene. The chairs can therefore withstand heavy rain, frosty weather, and the sun's UV radiation and be left outdoor all year round without compromising the design or quality. The combination of ArtFibre and aluminum gives the chair a remarkable strength and comfort. The Valerie chair is further maintenance-free and easy to clean. The Alu-Affire collection was designed for brasseries, cafï¿½s, and restaurants, but is also suitable for private homes. This Valerie chair has armrests, which provides extra support and comfort. The chair is stackable and therefore easy to store.",
  {'entities': [(0, 7, 'PRODUCT'),
    (703, 716, 'PRODUCT'),
    (888, 901, 'PRODUCT')]}),
 ("Sofie is a charming outdoor dining chair and part of Sika-Design's Alu-Affire collection. Our Alu-Affire collection represents a unique selection of outdoor bistro and caf furniture. This furniture is inspired by Sika-Design's own designs from the 1950s and by the charming Parisian bistro atmosphere. Our Alu-Affire chairs are handwoven with ArtFibre, which is an extremely durable and weather-resistant material made of polyethylene. The chairs can therefore withstand heavy rain, frosty weather, and the sun's UV radiation and be left outdoor all year round without compromising the design or quality. The combination of ArtFibre and aluminum gives the chair a remarkable strength and comfort. The Sofie Exterior chair is further maintenance-free and easy to clean. The Alu-Affire collection was designed for brasseries, cafï¿½s, and restaurants, but is also suitable for private homes. The chair is stackable and therefore easy to store.",
  {'entities': [(0, 5, 'PRODUCT'), (701, 721, 'PRODUCT')]}),
 ('Nicole is a small classic caf table inspired by the charming French caf style. The table is designed for restaurants, brasseries, and cafes, but is also suitable for the private garden, terrace, and balcony. The table base is made of powder coated aluminum and the tabletop of teak wood which is known for its high strength and beautiful glow. Due to the teak woods natural oil content, our teak products are very durable, resistant to wind and weather, and is not attacked by insects and rot. Our exterior teak products are made using plantation-grown teak marked with the Indonesian Legal Woodï¿½ stamp, which ensures that the furniture is part of a sustainable production. This teak sourcing is overseen by the Indonesian government, making sure that no tree too immature or too small, is removed from the forest. The table can withstand all kinds of weather and be left outside all year round. Over time teak wood exposed to sun and rain will turn into a beautiful grey color. The table requires no maintenance but if you want to preserve the dark brown color, the teak must be treated with oil. Choose between different colors for the table base.',
  {'entities': [(0, 6, 'PRODUCT')]}),
 ('The round Victoria coffe table with glass top is designed by Sika-Design and is part of our romantic Georgia Garden collection. The Georgia Garden collection is inspired by the British colonial style and by classical Danish wicker furniture. The caf table is strong and very suitable for private use as well as for use in hotels and restaurants. It is handcrafted of aluminum and fully woven with our weather-resistant ArtFibre. The tables glass top is made from 8 mm tempered glass which makes it easy to clean and more durable than ordinary glass. The Victoria table is designed to stand in the open air and to make garden life as easy as possible. It can be left outdoor all year round and requires no maintenance.',
  {'entities': [(10, 30, 'PRODUCT'), (554, 568, 'PRODUCT')]}),
 ('Our Colonial Dining Table is made of teak wood known for its high strength and beautiful glow. On our outdoor tables the wooden boards are placed with a small space between so the wood can expand and contract depending on the weather. Over time teak furniture exposed to sun and rain will turn grey. The table requires no maintenance but if you want to preserve the dark brown color, the teak must be treated with oil. Due to its natural oil content, the wood is very durable, resistant to wind and weather, and is not attacked by insects and rot. Therefore, our outdoor teak furniture can withstand all kinds of weather and be left outside all year round. At Sika-Design, we only use premium teak from Indonesia for our teak furniture. Our exterior teak products are made using plantation-grown teak marked with the Indonesian Legal Woodï¿½ stamp. This teak sourcing is overseen by the Indonesian government, making sure that no tree too immature or too small, is removed from the forest. The garden table measures 200x100 cm and fits 6 persons.',
  {'entities': [(4, 25, 'PRODUCT'),
    (304, 309, 'PRODUCT'),
    (999, 1004, 'PRODUCT')]}),
 ('Our round George table 160 cm is a beautiful wooden garden table made of teak which is known for its high strength and beautiful glow. In the middle of the tabletop you find an umbrella hole which can be covered with the elegant included top. Due to its natural oil content, the wood is very durable, resistant to wind and weather, and is not attacked by insects and rot. Therefore, our outdoor teak furniture can withstand all kinds of weather and be left outside all year round. On our exterior teak furniture the wooden boards are placed with a small space between so the wood can expand and contract depending on the weather. Over time teak furniture exposed to sun and rain will turn grey. The garden table requires no maintenance but if you want to preserve the dark brown color, the teak must be treated with oil. At Sika-Design we only use premium teak from Indonesia for our teak furniture. Our exterior teak products are made using plantation-grown teak marked with the Indonesian Legal Woodï¿½ stamp. This teak sourcing is overseen by the Indonesian government, making sure that no tree too immature or too small, is removed from the forest. The round George table has a width of 160 cm and fits 6 persons.',
  {'entities': [(10, 22, 'PRODUCT'),
    (699, 711, 'PRODUCT'),
    (1161, 1173, 'PRODUCT')]}),
 ('Carlo Trolley signals both aesthetics and hospitality. It is ideal for storage and serving, and the wheels make it easy to move around the terrace. The decorative and practical trolley is highly requested in the indoor version in rattan. Now we are presenting it as a novelty in Alu-Rattan, where it is available in the new natural color.',
  {'entities': [(0, 13, 'PRODUCT')]}),
 ('Our new Maggie modular sofa adapts to your outdoor needs. Use it as a gorgeous lounge sofa or create a cozy corner in the sun for two. With its three different modules ï¿½ corner, center and pouf ï¿½ Maggie Modular Sofa has endless combination possibilities. The pouf module can be used with cushion or glass tabletop that transforms it into a practical side table.',
  {'entities': [(8, 27, 'PRODUCT'),
    (86, 90, 'PRODUCT'),
    (196, 215, 'PRODUCT')]}),
 ('Our Julian Coffee Table is a beautiful wooden table made of teak wood which is known for its high strength and beautiful glow. On our outdoor tables the wooden boards are placed with a small space between so the wood can expand and contract depending on the weather. Over time teak furniture exposed to sun and rain will turn grey. The table requires no maintenance but if you want to preserve the dark brown color, the teak must be treated with oil. Due to its natural oil content, the wood is very durable, resistant to wind and weather, and is not attacked by insects and rot. Therefore, our outdoor teak furniture can withstand all kinds of weather and be left outside all year round. At Sika-Design we only use premium teak from Indonesia for our teak furniture. Our exterior teak products are made using plantation-grown teak marked with the Indonesian Legal Woodï¿½ stamp. This teak sourcing is overseen by the Indonesian government, making sure that no tree too immature or too small, is removed from the forest. The Julian Coffee Table measures 120x120 cm and is especially great for large outdoor lounge areas.',
  {'entities': [(4, 23, 'PRODUCT'),
    (336, 341, 'PRODUCT'),
    (1023, 1042, 'PRODUCT')]}),
 ('Lyon is a small classic caf table inspired by the charming French caf style. The table is designed for restaurants, brasseries, and cafes, but is also suitable for the private garden, terrace, and balcony. The table base is made of powder coated cast iron and the tabletop of versalite ï¿½ a strong, laminated tabletop in marble look. The tabletop has a smooth, scratch free and hygienic surface. The tabletop has a diameter of 70 cm and a charming brass edge. Both the table base and tabletop are maintenance-free, easy to clean and resistant to all weather conditions which makes it a perfect choice for restaurants and hotels.',
  {'entities': [(0, 4, 'PRODUCT')]}),
 ('Nicole is a small classic caf table inspired by the charming French caf style. The table is designed for restaurants, brasseries, and cafes, but is also suitable for the private garden, terrace, and balcony. The table base is made of powder coated aluminum and the tabletop of versalite ï¿½ a durable, laminated tabletop in granite-look. The tabletop measures 80x80 cm and has a smooth, scratch free and hygienic surface. Both the table base and tabletop are maintenance-free, easy to clean and resistant to all weather conditions which makes it a perfect choice for restaurants and hotels. Choose between different colors for the table base.',
  {'entities': [(0, 6, 'PRODUCT')]}),
 ('Lyon is a small classic caf table inspired by the charming French caf style. The table is designed for restaurants, brasseries, and cafes, but is also suitable for the private garden, terrace, and balcony. The table base is made of powder coated cast iron and the tabletop of teak wood which is known for its high strength and beautiful glow. Due to the teak woods natural oil content, our teak products are very durable, resistant to wind and weather, and is not attacked by insects and rot. Our exterior teak products are made using plantation-grown teak marked with the Indonesian Legal Woodï¿½ stamp, which ensures that the furniture is part of a sustainable production. This teak sourcing is overseen by the Indonesian government, making sure that no tree too immature or too small, is removed from the forest. The table can withstand all kinds of weather and be left outside all year round. Over time teak wood exposed to sun and rain will turn into a beautiful grey color. The table requires no maintenance but if you want to preserve the dark brown color, the teak must be treated with oil.',
  {'entities': [(0, 4, 'PRODUCT')]}),
 ("The Charlottenborg Exterior Sofa is designed by Arne Jacobsen and is made in Alu-Rattan and ArtFibre materials. Alu-Rattan is weatherproof powder coated aluminum, and ArtFibre is artificial fibers made of maintenance-free polyethylene that can withstand the sun's UV radiation. Due to the weatherproof materials, the furniture can withstand being outside throughout the year without maintenance. The Charlottenborg Exterior sofa is fitted with a seat made of Quick Dry Foam. This foam is specially developed for outdoor use as the foam drains water rapidly and maximizes air circulation which enables a faster drying process. The fabric on the seat cushion can be chosen from a broad variety of high-quality fabrics and styles. The cushions are handcrafted, and made to order, at Sika-Design's own studio, located at the headquarters in Denmark.",
  {'entities': [(4, 32, 'PRODUCT'), (400, 428, 'PRODUCT')]}),
 ('The Tony Exterior Side Table has a frame of hand bend aluminium with bindings of ArtFibre handmade by skilled wicker-makers. The Aluminium frame is further hand painted with great precision to look like real rattan. On the top there is a fine round glass plate which is fixed inside the frame. The table is suitable for all-year outdoor use, is maintenance free and UV-resistant. The Tony Exterior Side Table is suitable for use in any home, summerhouse or in cafï¿½s and restaurants. We recommend placing the table next to one of our Monet Exterior Lounge Chair. The Tony Exterior Side Table is designed by Sika-Design and can also be found for indoor use in rattan.',
  {'entities': [(4, 28, 'PRODUCT'),
    (384, 408, 'PRODUCT'),
    (533, 561, 'PRODUCT'),
    (566, 590, 'PRODUCT')]}),
 ('Our Maggie Module Sofa is designed by Sika-Design and consists of three different modules; corner, center and pouf where the pouf can be used with a cushion or glass tabletop that transforms it into a practical side table. The modules offer endless possibilities and can be adapted to individual outdoor needs. This sofa consists of two corner modules, a center module and a pouf module. It includes 3 back cushions and 4 seat cushions.',
  {'entities': [(4, 25, 'PRODUCT'), (316, 320, 'PRODUCT')]}),
 ("Caroline 3-seater is designed by Sika-Design and is made in Alu-Rattan and ArtFibre materials. Alu-Rattan is weatherproof powder coated aluminum and ArtFibre is artificial fibers made of maintenance-free polyethylene that can withstand the sun's UV radiation. Due to the weatherproof materials, the furniture can withstand being outside throughout the year without maintenance. In addition to the aluminum frame, the couch is fitted with a seat with Quick Dry Foam, which comes in many different durable fabric variants manufactured in our Danish sewing room. We also offer to sew cushions in the customer's own fabric, please contact us to get enlightened about material use. Quick Dry Foam is quick-drying foam suitable for outdoor use. The foam is specially designed as an outdoor foam that can drain the water and maximize air circulation into a faster drying process. The fine bindings in the seat and back are made on a frame of aluminum of skilled wicker workers. Caroline contributes to a naturally beautiful element in any outdoor environment.",
  {'entities': [(0, 17, 'PRODUCT'), (971, 979, 'PRODUCT')]}),
 ("Caroline lounge chair is designed by Sika-Design and is made in Alu-Rattan and ArtFibre materials. Alu-Rattan is weatherproof powder coated aluminum and ArtFibre is artificial fibers made of maintenance-free polyethylene that can withstand the sun's UV radiation. Due to the weatherproof materials, the furniture can withstand being outside throughout the year without maintenance. In addition to the aluminum frame, the chair is fitted with a seat with Quick Dry Foam, which comes in many different durable fabric variants manufactured in our Danish sewing room. We also offer to sew cushions in the customer's own fabric, please contact us to get enlightened about material use. Quick Dry Foam is quick-drying foam suitable for outdoor use. The foam is specially designed as an outdoor foam that can drain the water and maximize air circulation into a faster drying process. The fine bindings in the seat and back are made on a frame of aluminum of skilled wicker workers. Caroline contributes to a naturally beautiful element in any outdoor environment. ",
  {'entities': [(0, 21, 'PRODUCT'),
    (421, 426, 'PRODUCT'),
    (975, 983, 'PRODUCT')]}),
 ("Belladonna is designed by Franco Albini and made of Alu-Rattan and ArtFibre materials. Alu-Rattan is weatherproof powder coated aluminum and ArtFibre is artificial fibers made of maintenance-free polyethylene that can withstand the sun's UV radiation. Due to the weatherproof materials, the furniture can withstand being outside throughout the year without maintenance. In addition to the frame of aluminum, the sofa is fitted with a seat with Quick Dry Foam, which comes in many different durable fabric variants manufactured on our Danish sewing room. Quick Dry Foam is quick-drying foam suitable for outdoor use. The foam is specially designed as an outdoor foam that can drain the water and maximize air circulation into a faster drying process. Franco Albini (1905-1977) was both an architect and designer and had his heyday in the 1950s, producing many acclaimed furniture designs of, among other things, the furniture design. Bookcase systems and chairs Margherita and Gala, today at MOMA in New York, as well as architectural works such as La Rinascente in Rome, properties and metro stations in Milan. ",
  {'entities': [(0, 10, 'PRODUCT')]}),
 ("Charlottenborg Exterior Lounge Chair is designed by Arne Jacobsen and is made in Alu-Rattan and ArtFibre materials. Alu-Rattan is weatherproof powder coated aluminum and ArtFibre is artificial fibers made of maintenance-free polyethylene that can withstand the sun's UV radiation. Due to the weatherproof materials, the furniture can withstand being outside throughout the year without maintenance. In addition to the aluminum frame, the chair is fitted with a seat with Quick Dry Foam, which comes in many different durable fabric variants manufactured in our Danish sewing room. We also offer to sew cushions in the customer's own fabric, please contact us to get enlightened about material use. Quick Dry Foam is quick-drying foam suitable for outdoor use. The foam is specially designed as an outdoor foam that can drain the water and maximize air circulation into a faster drying process. Due to the weatherproof materials, the chair can be used year-round. The fine bindings in the seat and back are made on a frame of aluminum of skilled wicker workers. ",
  {'entities': [(0, 36, 'PRODUCT'),
    (438, 443, 'PRODUCT'),
    (933, 938, 'PRODUCT')]}),
 ("The Hanging egg chair was designed by Nanna and Jrgen Ditzel in 1959 and is made in Alu-Rattan and ArtFibre materials. The hanging chair is made in the colour matt black, which is original. Alu-Rattan is weatherproof powder coated aluminum and ArtFibre is artificial fibers made of maintenance-free polyethylene that can withstand the sun's UV radiation. Due to the weatherproof materials, the furniture can withstand being outside throughout the year without maintenance. Hanging egg chair fits into a home where there is plenty of space but is also suitable for open outdoor areas. After being produced by an Italian producer for several years, Nanna & Jrgen Ditzel's iconic egg-shaped hanging chair is back in Danish production. In addition to the aluminium frame, a cushion can be purchased with Quick Dry Foam, which comes in many different durable fabric variants sewn in our Danish sewing room. We also offer to sew cushions in the customer's own fabric, please contact us to get enlightened about material use. Quick Dry Foam is quick-drying foam suitable for outdoor use. The foam is specially designed as an outdoor foam that can drain the water and maximize air circulation into a faster drying process. Due to the weatherproof materials, the chair can be used year-round. ",
  {'entities': [(4, 21, 'PRODUCT'),
    (123, 136, 'PRODUCT'),
    (473, 490, 'PRODUCT'),
    (677, 701, 'PRODUCT'),
    (1254, 1259, 'PRODUCT')]}),
 ("The architects behind this chair are Nanna Ditzel and her husband Jrgen Ditzel. They designed the Madame chair in 1951 and their experimental approach to materials resulted in the shape of a shell. With its soft and feminine shapes, the name became Madame. The chair is part of Sika-Design's ICONS collection, which consists exclusively of furniture designed by renowned designers. The chair has been available for indoor use for many years and is now also available as a weather-resistant outdoor chair. A seat and back cushion can be purchased for the Madame chair in any optional fabric, sewn on our Danish sewing department. We also offer to sew cushion in your own fabric, please contact us for more information about material use. ",
  {'entities': [(98, 110, 'PRODUCT'),
    (249, 256, 'PRODUCT'),
    (386, 391, 'PRODUCT'),
    (554, 566, 'PRODUCT')]}),
 ("Fox chair is designed by Viggo Boesen and made in Alu-Rattan and ArtFibre materials. Alu-Rattan is weatherproof powder coated aluminum and ArtFibre is artificial fibers made of maintenance-free polyethylene that can withstand the sun's UV radiation. Due to the weatherproof materials, the furniture can withstand being outside throughout the year without maintenance. It is possible to buy a cushion for the chair which is made in our Danish sewing room. The fine bindings in the seat and back are made on a frame of aluminum of skilled wicker workers. Fox contributes a naturally beautiful element in any outdoor environment. Viggo Boesen is a trained architect and in the 1930s he was among the architects who introduced the new villa style, Funkis, which became popular with the general population. The furniture was supposed to have an organic flow and embrace the body supple and comfortable. ",
  {'entities': [(0, 9, 'PRODUCT'),
    (408, 413, 'PRODUCT'),
    (553, 556, 'PRODUCT')]}),
 ("Luna is designed by Sika-Design and made of Alu-Rattan and ArtFibre materials. Luna is exclusively designed for our Exterior collection. Alu-Rattan is weatherproof powder coated aluminum and ArtFibre is artificial fibers made of maintenance-free polyethylene that can withstand the sun's UV radiation. Due to the weatherproof materials, the furniture can withstand being outside throughout the year without maintenance. In addition to the frame of aluminum, the chair is fitted with a seat that comes in many different durable fabric variants manufactured in our Danish sewing room. We also offer to sew cushions in the customer's own fabric, please contact us to get enlightened about material use. Due to the weatherproof materials, the chair can be used throughout the year. The fine bindings in the seat and back are made on a frame of aluminum of skilled wicker workers. The armchair contributes to a naturally beautiful element in any outdoor environment. ",
  {'entities': [(0, 4, 'PRODUCT'),
    (79, 83, 'PRODUCT'),
    (462, 467, 'PRODUCT'),
    (739, 744, 'PRODUCT'),
    (880, 888, 'PRODUCT')]}),
 ('Our new Maggie modular sofa adapts to your outdoor needs. Use it as a gorgeous lounge sofa or create a cozy corner in the sun for two. With its three different modules ï¿½ corner, center and pouf ï¿½ Maggie Modular Sofa has endless combination possibilities. The pouf module can be used with cushion or glass tabletop that transforms it into a practical side table.',
  {'entities': [(8, 27, 'PRODUCT'), (196, 215, 'PRODUCT')]}),
 ("Monet is designed by Sika-Design and made of Alu-Rattan and ArtFibre materials. The stool is made in the colour dove white which is fine and elegant. Alu-Rattan is weatherproof powder coated aluminum. ArtFibre is artificial fibers made of maintenance-free polyethylene that can withstand the sun's UV radiation. Due to the weatherproof materials, the furniture can withstand being outside throughout the year without maintenance. The Monet footstool contributes with a naturally beautiful element and of course fits our Monet armchair. You can choose to buy a cushion in optional fabric, manufactured in our Danish sewing room. We also offer to sew cushions in the customer's own fabric, please contact us to get enlightened about material use. ",
  {'entities': [(0, 5, 'PRODUCT'),
    (84, 89, 'PRODUCT'),
    (434, 449, 'PRODUCT'),
    (520, 535, 'PRODUCT')]}),
 ('Malabar ï¿½ Good EarthFine Bone China Dinnerware Malabar Good Earth, beautiful dinnerware with a delicate design that evokes the fragrance of flowers and the majesty of howdah elephants. Skillfully made in hand-decorated fine bone china, with highlights in 24 carat gold and platinum. All Good Earth items presented on our website can be viewed and purchased from our store in Singapore. They are not available online. From Indiaï¿½s leading design house Good Earth.',
  {'entities': [(0, 46, 'PRODUCT'),
    (47, 65, 'PRODUCT'),
    (77, 87, 'PRODUCT')]}),
 ('Indus ï¿½ Good EarthFine Bone China Dinnerware Indus Good Earth, a signature dinnerware collection in a new colour palette, evoking the romance of magical lake Periyar in Kerala. Motifs of majestic elephants and verdant groves of palm, banana and mango trees are an integral part of the landscape. Fine bone china, hand decorated with artwork decals and accented with 24 carat gold. Find out more about our Tropical Christmas Dinner Table decked with Indus Tableware. All Good Earth items presented on our website can be viewed and purchased in our store in Singapore. They are not available online. From Indiaï¿½s leading design house Good Earth',
  {'entities': [(0, 44, 'PRODUCT'),
    (45, 61, 'PRODUCT'),
    (449, 465, 'PRODUCT')]}),
 ('Mango Chutney ï¿½ Good EarthPorcelain Dinnerware Mango Chutney Good Earth is a lovely addition to any table setting. This dinnerware collection is decorated with motifs of raw mangoes, chillies and coriander. Made from porcelain, it is hand-decorated with artwork decals. Microwave safe. Dishwasher safe at temperature less than 150ï¿½ F in short wash cycle. From Indiaï¿½s leading design house Good Earth',
  {'entities': [(0, 13, 'PRODUCT'),
    (16, 60, 'PRODUCT'),
    (61, 71, 'PRODUCT'),
    (120, 130, 'PRODUCT')]}),
 ('Myra Centre Piece ï¿½ Good Earth A hand-blown glass centrepiece with silver plated hand-etched latticework. The Myra Centre Piece draws inspiration from the ornamental tessellations found on the architecture of centuries-old monuments of the subcontinent. Height ï¿½ 6.5ï¿½, Dia ï¿½ 7ï¿½ All Good Earth items presented on our website can be viewed and purchased from our store in Singapore. They are not available online. From Indiaï¿½s leading design house Good Earth',
  {'entities': [(0, 17, 'PRODUCT'),
    (20, 61, 'PRODUCT'),
    (110, 127, 'PRODUCT')]}),
 ('Rosabagh ï¿½ Good EarthFine Bone China Dinnerware Rosabagh Good Earth, a collection of fine bone china dinnerware with leitmotifs inspired by a vintage Persian miniature, and the enchanting garden life of the Mughals. The collection captures intricate outdoor pavilions, birds and butterflies in an oasis of verdant blossoms, illustrating a paradise garden with accents of 24 carat gold. All Good Earth items presented on our website can be viewed and purchased in our store in Singapore. They are not available online. From Indiaï¿½s leading design house Good Earth',
  {'entities': [(0, 47, 'PRODUCT'),
    (48, 67, 'PRODUCT'),
    (101, 111, 'PRODUCT')]}),
 ('Ceylon Tea SetSilver Plated Tea Service (set of 3) Refined tea service, comprised of teapot, sugar bowl and creamer. The set has a classic look with curvilinear proportions and an elegant spout and handle. Made from cast brass with silver plating, delicately cast borders and hand embossed accents.The perfect gift for a hostess who enjoys serving high tea in style. Pictured with the Mandalay Wine Tray also from Good Earth. This circular nickle plated wine tray is engraved with intricate floral artwork, encircled with bands of delicate lattice jaal and an embossed rim. Available in our store. All Good Earth items presented on our website can be viewed and purchased in our store in Singapore. They are not available online. From Indiaï¿½s leading design house Good Earth',
  {'entities': [(0, 39, 'PRODUCT'),
    (59, 70, 'PRODUCT'),
    (385, 403, 'PRODUCT')]}),
 ('Cartouche GlassesSet of 6 Set of six Cartouche glasses designed as a fresh take on the tea glasses of the Middle East.Decorated with gold-based lustre colours of the highest quality. The jewel-like glasses are a delightful way of offering cool sherbets, even just water, and makes a perfect gift combining beauty with practicality. Cartouche is a French word meaning gun cartridge. It was coined by Napoleons soldiers to describe the oblong bands around the names of Egyptian pharaohs. All Good Earth items presented on our website can be viewed and purchased in our store in Singapore. They are not available online. From Indiaï¿½s leading design house Good Earth',
  {'entities': [(0, 54, 'PRODUCT'),
    (87, 98, 'PRODUCT'),
    (198, 205, 'PRODUCT'),
    (332, 341, 'PRODUCT')]}),
 ('Mandalay Wine TrayNickle plated brass Circular nickle plated wine tray engraved with intricate floral artwork, encircled with bands of delicate lattice jaal and and an embossed rim. Bringing service with style. Nickel plated brass in a mix of polished and antique finishes with fine embossed floral accents, skillfully made by na expert craftsman. Size 14.5ï¿½. All Good Earth items presented on our website can be viewed and purchased in our store in Singapore. They are not available online. From Indiaï¿½s leading design house Good Earth.',
  {'entities': [(0, 70, 'PRODUCT'), (211, 217, 'PRODUCT')]}),
 ('Serendib ï¿½ Good EarthOval Platter Serendib oval platter featuring elephants, birds and flowers. Accented with 24-carat gold detailing. Hand decorated fine bone china. Presented in a nice gift box. All Good Earth items presented on our website can be viewed and purchased in our store in Singapore. They are not available online. From Indiaï¿½s leading design house Good Earth',
  {'entities': [(0, 8, 'PRODUCT'), (11, 55, 'PRODUCT')]}),
 ('Indus ï¿½ Good EarthTea Service (set of 3) Stylish fine bone china tea set, which includes teapot, creamer and sugar bowl. Delicately ornamented with tropical motifs, majestic elephants and 24 carat gold accents. The Indus Tea Service is made from fine bone china, hand decorated with artwork decals and accented with 24 carat gold. Pictured with the Mandalay Wine Tray also from Good Earth. This circular nickle plated wine tray is engraved with intricate floral artwork, encircled with bands of delicate lattice jaal and an embossed rim. Available in our store. All Good Earth items presented on our website can be viewed and purchased in our store in Singapore. They are not available online. From Indiaï¿½s leading design house Good Earth',
  {'entities': [(0, 29, 'PRODUCT'),
    (215, 232, 'PRODUCT'),
    (349, 367, 'PRODUCT')]}),
 ('Rosabagh ï¿½ Good EarthOval Platter Decorated with a cheetah leitmotif and edged with lush greenery, the Rosabagh oval platter brings an enchanting paradise garden to your table. Made from fine bone china and hand decorated with artwork decals and 24 carat gold detailing. From Indiaï¿½s leading design house Good Earth',
  {'entities': [(0, 33, 'PRODUCT'), (103, 124, 'PRODUCT')]}),
 ('Protect your cushions, pillows, towels, and other outdoor items with our waterproof Cushion Box. Its large size offers lots of storage while maintaining a minimal design, suitable for any outdoor space. COLLECTION CUSHION BOX MATERIALS OUTDOOR WICKER (HDPE) WIDTH 72 DEPTH 38 HEIGHT 36 WEIGHT 198.4 Our products are available in any colour that we offer. Please contact us for more information 3019 Dufferin Street Toronto, Ontario M6B 3T7, Canada 416-551-6055 [email protected] Winter Hours Monday-Saturday: 10:00-5:00 Sunday: Closed Summer Hours Sunday: 11:00-5:00 Monday-Saturday: 10:00-5:00',
  {'entities': [(73, 96, 'PRODUCT'), (214, 225, 'PRODUCT')]}),
 ('The Eclipse Tea Cart is an essential item to have in your outdoor space. Its four wheels and handles allows you to take it around your outdoor space easily and efficiently, perfect for entertaining your guests. COLLECTION ECLIPSE MATERIALS STAINLESS STEEL WIDTH 35.75 DEPTH 21 HEIGHT 37 Our products are available in any colour that we offer. Please contact us for more information 3019 Dufferin Street Toronto, Ontario M6B 3T7, Canada 416-551-6055 [email protected] Winter Hours Monday-Saturday: 10:00-5:00 Sunday: Closed Summer Hours Sunday: 11:00-5:00 Monday-Saturday: 10:00-5:00',
  {'entities': [(4, 20, 'PRODUCT'), (222, 229, 'PRODUCT')]}),
 ('When temperatures are soaring, what could be more relaxing than chilling by the pool Hereï¿½s the answer. You can now take it easy on the water as well as poolside on a Fatboy Floatzac. Welcome to the water lounge. The Floatzac water lounger gives you the best of both worlds: the familiar comfort of a Fatboy bean bag plus the refreshing sensation of being on and in the water. The Fatboy Floatzac is filled with quick-drying water-resistant pearls and features a strong mesh on the underside to allow water to drain out. It is available in a range of modern and chlorine-resistant colours. This is your new BFF (best floating friend) in the sunshine. COLLECTION Fatboy MATERIALS Acrylic and Sunbrella, filled with polystyrene WIDTH 55 DEPTH 70 3019 Dufferin Street Toronto, Ontario M6B 3T7, Canada 416-551-6055 [email protected] Winter Hours Monday-Saturday: 10:00-5:00 Sunday: Closed Summer Hours Sunday: 11:00-5:00 Monday-Saturday: 10:00-5:00',
  {'entities': [(217, 239, 'PRODUCT'), (381, 396, 'PRODUCT')]}),
 ("Rymar Synthetic Grass offers the rich look of real grass with our environmentally-friendly synthetic grass and turf products. Designed for the ever-changing Canadian climate, the use of breakthrough materials and processes in manufacturing ensures they will look and feel just as realistic and beautiful for years. The level of maintenance of a yard can make all the difference when it comes to how it looks and how clean it is. The fact is that maintaining a lawn takes a significant amount of time and effort. With our synthetic lawns be assured that the maintenance required is far less than with a conventional yard and is much safer. Our synthetic grass will always have the fresh cut look even in the winter. It is the ultimate way to enjoy a great looking yard without the regular maintenance. This grass requires no water, chemicals, fertilizers or insecticides and is great for pets. Furthermore, our synthetic products are anti-microbial so there is never a worry about harmful bacteria building up or allergic reactions caused by the grass. Size per Roll 15' x 50' Sq. Ft per Roll 750 3019 Dufferin Street Toronto, Ontario M6B 3T7, Canada 416-551-6055 [email protected] Winter Hours Monday-Saturday: 10:00-5:00 Sunday: Closed Summer Hours Sunday: 11:00-5:00 Monday-Saturday: 10:00-5:00",
  {'entities': [(0, 21, 'PRODUCT'),
    (91, 106, 'PRODUCT'),
    (643, 658, 'PRODUCT')]}),
 ('SONNENGLAS stores energy during the day and releases light at night. Producing light in this way is fun, sustainable and makes it easy to do your bit for the environment. And SONNENGLAS will light up the dark for years to come. SONNENGLAS contains four solar-powered LEDs, which are charged by solar cells in its lid. When placed in direct sunlight the batteries are charged and provide many hours of soft, yet bright lighting without glare. SONNENGLAS is ideal for garden lighting, decorating your table and other areas, barbecues, camping, alfresco dining and all other outdoor pursuits that would benefit from beautiful lighting to create a wonderful and relaxed atmosphere. For more information, click here. CLASSIC SIZE 4.5x7 MINI SIZE 3019 Dufferin Street Toronto, Ontario M6B 3T7, Canada 416-551-6055 [email protected] Winter Hours Monday-Saturday: 10:00-5:00 Sunday: Closed Summer Hours Sunday: 11:00-5:00 Monday-Saturday: 10:00-5:00',
  {'entities': [(0, 10, 'PRODUCT'),
    (175, 185, 'PRODUCT'),
    (228, 238, 'PRODUCT'),
    (442, 452, 'PRODUCT')]}),
 ('Having trouble finding an attractive towel storage solution Well look no further, the Towel Storage Tower is the perfect addition to any pool or exercise room. Constructed from high quality polyethylene with UV inhibitors so you wonï¿½t have any worries about fading or discoloring. The louvered design captures the relaxed feeling of a seaside resort making this item perfect for your spa and fitness areas. MATERIALS LLDPE WIDTH 23 DEPTH 23 HEIGHT 66 3019 Dufferin Street Toronto, Ontario M6B 3T7, Canada 416-551-6055 [email protected] Winter Hours Monday-Saturday: 10:00-5:00 Sunday: Closed Summer Hours Sunday: 11:00-5:00 Monday-Saturday: 10:00-5:00',
  {'entities': [(86, 105, 'PRODUCT')]}),
 ('Take control of your towel storage needs with the Towel Valet. This durable unit offers the perfect balance of function and attractive styling. Clean, modern lines paired with subtle cues of New England design creates a universal appeal that will compliment any space whether itï¿½s a hotel pool or busy fitness center. Thereï¿½s plenty of space for towels which means less frequent replenishment by staff and increased convenience for your guests. The hinged door provides direct access to the towel disposal bin making emptying wet towels an easy task. MATERIALS LLDPE WIDTH 47 DEPTH 20 HEIGHT 41 3019 Dufferin Street Toronto, Ontario M6B 3T7, Canada 416-551-6055 [email protected] Winter Hours Monday-Saturday: 10:00-5:00 Sunday: Closed Summer Hours Sunday: 11:00-5:00 Monday-Saturday: 10:00-5:00',
  {'entities': [(50, 62, 'PRODUCT')]}),
 ('The Adan Stool, an item with a lot of personality, and a shape which is faceted, like that of a diamond. More than objects, they are multifunctional architecture pieces since besides being used as vases, they can be used as seats, flower pots, auxiliary tables and more. They are lifestyle objects that create a stir, fitting in with any style and are made of materials that allow for both outdoor and indoor use. NOTE THIS IS A CUSTOM PRODUCT. AFTER YOU PLACE YOUR ORDER A SALES REPRESENTATIVE WILL BE IN CONTACT WITH YOU TO DISCUSS THE TIMELINE OF UP TO 6 WEEKS. MATERIALS Polyethylene Resin Width 11.75 Depth 16.25 Height 16.5 3019 Dufferin Street Toronto, Ontario M6B 3T7, Canada 416-551-6055 [email protected] Winter Hours Monday-Saturday: 10:00-5:00 Sunday: Closed Summer Hours Sunday: 11:00-5:00 Monday-Saturday: 10:00-5:00',
  {'entities': [(4, 14, 'PRODUCT')]}),
 ('This self-watering planter is made of polyethylene resin by rotational moulding, 100% recyclable. The Peacock Planter is suitable for indoor and outdoor use. It is without doubt a unique element due to itï¿½s amusing and symbolic design. NOTE THIS IS A CUSTOM PRODUCT. AFTER YOU PLACE YOUR ORDER A SALES REPRESENTATIVE WILL BE IN CONTACT WITH YOU TO DISCUSS THE TIMELINE OF UP TO 6 WEEKS. Materials Polyethylene Resin Width 29.5 Depth 45.75 Height 59 3019 Dufferin Street Toronto, Ontario M6B 3T7, Canada 416-551-6055 [email protected] Winter Hours Monday-Saturday: 10:00-5:00 Sunday: Closed Summer Hours Sunday: 11:00-5:00 Monday-Saturday: 10:00-5:00',
  {'entities': [(102, 117, 'PRODUCT')]}),
 ('The GATSBY collection, designed by Ramn Esteve for VONDOM, recalls the Art Dco lighting. This lamp is based on the idea of creating a surrounding atmosphere through a suggesting play of lights by means of the lampï¿½s surfaces, where the material reproduces the shapes and sensations of engraved glass. MATERIALS Polymethyl Methacrylate Resin DIAMETER 3.25 HEIGHT 9.75 3019 Dufferin Street Toronto, Ontario M6B 3T7, Canada 416-551-6055 [email protected] Winter Hours Monday-Saturday: 10:00-5:00 Sunday: Closed Summer Hours Sunday: 11:00-5:00 Monday-Saturday: 10:00-5:00',
  {'entities': [(94, 98, 'PRODUCT')]}),
 ("The Second Light Lamp is reminiscent of a classic light bulb with a modern flair. Clean lines and minimal design make this unique style perfect for both residential and hospitality settings. Provides soft diffused illumination. Materials Polyethylene Resin Small 7.5 dia x 9.75 Large 15 dia x 19.5 3019 Dufferin Street Toronto, Ontario M6B 3T7, Canada 416-551-6055 [email protected] Winter Hours Monday-Saturday: 10:00-5:00 Sunday: Closed Summer Hours Sunday: 11:00-5:00 Monday-Saturday: 10:00-5:00",
  {'entities': [(4, 21, 'PRODUCT')]}),
 ('Detail: The Hampton outdoor fire pit collection features durable cast aluminum, North American brand powder-coat paint, and a North American brand stainless steel burner along with a propane and natural gas conversion kit, and 4 glass options. The top has a chunky weave pattern giving it a luxury contemporary style while creating warmth and ambiance. Tested in Canada and the US as an outdoor gas appliance Listed to ULCR97-003 & CSA 4.96 U.S. The picture shows the round burner which is 60,000BTU. The fire pit is shown in the Dark Rum finish.',
  {'entities': [(12, 36, 'PRODUCT')]}),
 ("Detail: The North American brand 304 stainless steel burner comes with a propane and natural gas conversion kit, and 4 glass options. It can be used with the Monaco, Venice, & Hampton outdoor fire pits, and is also suitable for custom fire pits. The Picture Shows: 304 Stainless Steel 12'' x 24'' Burner, 60,000BTU, Clear Fire Glass (2 Bags), Flex Tube, Gas Hose With Regulator, Starter, And Ball Valve With Orifice.",
  {'entities': [(33, 59, 'PRODUCT')]}),
 ("Detail: The Hampton outdoor fire pit collection features durable cast aluminum, North American brand powder-coat paint, and a North American brand stainless steel burner along with a propane and natural gas conversion kit, and 4 glass options. The top has a chunky weave pattern giving it a luxury contemporary style while creating warmth and ambiance. Tested in Canada and the US as an outdoor gas appliance Listed to ULCR97-003 & CSA 4.96 U.S. The picture shows the 50'' x 32'' burner which is 60,000BTU. The fire pit is shown in the Dark Rum finish",
  {'entities': [(12, 36, 'PRODUCT'), (511, 519, 'PRODUCT')]}),
 ('Detail: Contemporary Outdoor Fire Pit, Cast Aluminum, Propane and Natural Gas. North American Brand Stainless Steel Burner, 60,000BTU, 4 Glass Options. Tested in Canada and the US as an outdoor gas appliance Listed to UL CR97-003 & CSA 4.96 U.S. The Picture Shows: Top Color: Dark Rum, Base Color: Dark Rum.',
  {'entities': [(8, 37, 'PRODUCT')]}),
 ('Detail: Contemporary Outdoor Fire Pit, Cast Aluminum, Propane and Natural Gas. North American Brand Stainless Steel Burner, 100,000BTU, 4 Glass Options. Tested in Canada and the US as an outdoor gas appliance Listed to UL CR97-003 & CSA 4.96 U.S. The Picture Shows: Foster.',
  {'entities': [(8, 37, 'PRODUCT')]}),
 ('Detail: The Venice outdoor fire pit collection features durable cast aluminum, North American brand powder-coat paint, and a North American brand stainless steel burner along with a propane and natural gas conversion kit, and 4 glass options. The top is solid cast with a stone like texture. It has a modern luxury look and would be a welcome addition to any patio, creating warmth and ambiance. Tested in Canada and the US as an outdoor gas appliance Listed to ULCR97-003 & CSA 4.96 U.S. The picture shows the round burner which is 60,000BTU. The fire pit is shown in the Dark Rum finish.',
  {'entities': [(12, 35, 'PRODUCT')]}),
 ("Detail: The outdoor fire pit Wind Barriers features tempered glass and aluminum brackets. They protect the fire pit flames from wind, provide extra protection for children, and helps to keeps heat on windy days. The Picture Shows: 28'' square wind barrier to be use with the square burner.",
  {'entities': [(29, 42, 'PRODUCT')]}),
 ('The Gramercy outdoor fire pit collection is an architectural statement with a high performance outdoor tile table top. Includes a North American brand stainless steel burner along with a propane and natural gas conversion kit, and multiple fire glass options. The table top has an elegant porcelain tile with a cast aluminum border that gives the outdoor fire pit a luxury and contemporary style, igniting warmth and comfort. Tested in Canada and the US as an outdoor gas appliance.',
  {'entities': [(4, 29, 'PRODUCT')]}),
 ('Contemporary Outdoor Fire Pit, Cast Aluminum, North American Brand Stainless Steel Burner, Multiple Fire Glass Options. Tested in Canada and the US as an outdoor gas appliance.',
  {'entities': [(0, 29, 'PRODUCT')]}),
 ('The TCCTF48R Fresco 48 Round Fire Table is created from Texawood, and HDPE material made from recycled plastics and comes is available in a host of natural colors. The fire table utilizes a standard LP tank but can also be converted to used Natural gas with the conversion kit that is included. The LP tank stores neatly, hidden away inside the base of the table. The table included decorative clear glass gems for the fire bowl and windscreens to help keep breezes from fanning the flame.Point to PointNo',
  {'entities': [(4, 39, 'PRODUCT')]}),
 ("Detail: The North American brand 304 stainless steel burner comes with a propane and natural gas conversion kit, and 4 glass options. It can be used with the Monaco, Venice, & Hampton outdoor fire pits, and is also suitable for custom fire pits. The Picture Shows: 304 Stainless Steel 12'' x 24'' Burner, 60,000BTU, Clear Fire Glass (2 Bags), Flex Tube, Gas Hose With Regulator, Starter, And Ball Valve With Orifice.",
  {'entities': [(33, 59, 'PRODUCT'), (265, 284, 'PRODUCT')]}),
 ('Astral-TC is designed to close over table and chairs. To turn this gigantic umbrella, just turn the crank handle only 7 times.',
  {'entities': [(0, 9, 'PRODUCT')]}),
 ('When looking for a larger umbrella, our Cantilever Collection is offered in several sizes, packed with features and function. Our multi-position provide many shade solutions by providing the opportunity to easily shade several areas.',
  {'entities': [(26, 34, 'PRODUCT')]}),
 ('When looking for a larger umbrella, our Cantilever Collection is offered in several sizes, packed with features and function. Our multi-position provide many shade solutions by providing the opportunity to easily shade several areas.',
  {'entities': [(26, 34, 'PRODUCT')]}),
 ('Collection: Umbrella Bases Dimensions: Width: 0 Depth: 0 Height: 0 Weight: 50.000 Model Number: M9650-B',
  {'entities': [(12, 26, 'PRODUCT')]}),
 ('When looking for a larger umbrella, our Cantilever Collection is offered in several sizes, packed with features and function. Our multi-position provide many shade solutions by providing the opportunity to easily shade several areas.',
  {'entities': [(26, 34, 'PRODUCT')]}),
 ('The Alon 2-Seat Loveseat is the ideal balance of casual elegance and all-day comfort. It elegant merges an airy textural weave and with highly customizable powder coated commercial grade extruded aluminum frame. The Alon 2-Seat Loveseat pairs perfectly with a sofa for a large space or stands independently for an intimate setting.',
  {'entities': [(4, 24, 'PRODUCT'), (216, 236, 'PRODUCT')]}),
 ('Detail: Cushions Shown 5401 Blue. The Deep Seating Group Picture Cushions Shown 8351 Silver, Pillows 8351 Silver. Also Available With 1 Piece Seat Cushion.',
  {'entities': [(38, 73, 'PRODUCT')]}),
 ('The Aubrey wicker patio furniture is handwoven in weather-resistant SolWeave® round-style wicker on a powder-coated aluminum frame. The SolWeave® wicker is easy to maintain. The Aubrey wicker patio furniture design is suitable for residential, commercial and patio wine bars. The Aubrey wicker patio furniture group photo is shown in Canadian-made outdoor cushions with Sunbrella® 40012-0096 Fife Smoke cushions and Sunbrella® 145339-0006 Gable Stone pillows',
  {'entities': [(4, 33, 'PRODUCT'),
    (178, 207, 'PRODUCT'),
    (280, 309, 'PRODUCT')]}),
 ("Detail: Modern Designer Cast Aluminum Patio Furniture, Teak Look Cast Aluminum, Durable Aluminum Frames, North American Brand Powder-Coat Paint. Cushions Shown 5407 Henna. The Deep Seating Group Picture Cushions Shown 5492 Flax, 56088 Catalina Tinsel, 20'' Pillows 56088 Catalina Tinsel. Also Available With 1 Piece Seat Cushion.",
  {'entities': [(8, 53, 'PRODUCT')]}),
 ('Inspired by the classic architecture of a raised structure offering a ‘fine view’ as it means in Italian, Belvedere is a collection of substantial nature. Accentuated by bold, chamfered edges and new, Mica SOL Teak™ armrests, the robust profile of the Belvedere collection brings an immediate presence to any outdoor space. Belvedere is crafted from cast aluminum, finished with Tiger-Drylac® Grey or Sherwin-Williams® Patina paint, and is offered in deep seating or sectional options. Group photo cushions shown in 15000-0003 Spotlight Ash and 18″ x 18″ pillows shown in 145339-0006 Gable Stone.',
  {'entities': [(106, 115, 'PRODUCT'),
    (252, 261, 'PRODUCT'),
    (324, 333, 'PRODUCT')]}),
 ('Detail: Elegant Cast Aluminum Patio Furniture, Durable Aluminum Frames, North American Brand Powder-Coat Paint. Cushions Shown 48031 Mushroom. The Deep Seating Group Picture Cushions Shown 48031 Mushroom, Pillows 45495-0000 Moss.',
  {'entities': [(8, 45, 'PRODUCT')]}),
 ('Inspired by the appeal of a myriad of colours in a cresting wave, new Tide twist SOL Weave™ brings a distinctive, natural charm to the new CABANACOAST® Celestine collection. The Celestine wicker patio furniture is carefully handcrafted in weather-resistant SOL Weave® Tide wicker on a powder-coated aluminum frame that are easy and simple to maintain. Absolute comfort is achieved through a slightly higher back profile and custom made cushions. The Celestine wicker patio furniture group picture shows Canadian-made outdoor cushions with Sunbrella® 48114 Cast Pumice outdoor cushions and Sunbrella® 14501-0001 Mara Classic outdoor pillows.',
  {'entities': [(178, 210, 'PRODUCT'), (450, 482, 'PRODUCT')]}),
 ('The Cove rope patio furniture collection is crafted in new SOL Rope™ paired with Sunbrella® or Tempotest® outdoor cushions. Coordinating deep seating, sectional, lounge and a variety of dining height options will transform any outdoor venue into a comfortable, restful oasis. The Cove patio furniture group photo is shown in Canadian-made outdoor cushions in Sunbrella® 40434 Cast Slate and pillows shown in Sunbrella® 145339-006 Gable Stone on Taupe SOL Rope™ with a Weathered SOL Teak™ arm.',
  {'entities': [(4, 29, 'PRODUCT'), (280, 300, 'PRODUCT')]}),
 ('The transitional design of the Savannah collection and its teak wood appearance makes this patio furniture look and feel amazingly natural. The frames are crafted in premium SolTeak™ which is as smooth and as strong as real teak, with the colour and grain throughout, and is virtually maintenance free. Classic yet modern, Savannah is available in a warm, Natural finish or Weathered grey finish, and is suitable for residential, commercial and coastal properties. The group image shows Canadian-made outdoor cushions in Sunbrella® 16001-0001 Blend Indigo and pillows shown in Sunbrella® 145339-0002 Gable Indigo on SolTeak™ Natural finish.',
  {'entities': [(91, 106, 'PRODUCT')]}),
 ('Named after the small, charming resort town in Arizona, CABANACOAST®’s Sedona collection will bring a feeling of warmth and respite to any outdoor space. Sedona is graced with a high back, arched arm, and the organic hues of new Reef SOL Weave® Wicker, which will inspire time for reading a favorite book or a comfortable chat with a friend. The Sedona wicker patio furniture design is suitable for residential, commercial and patio wine bars. Create a personal retreat using a range of deep seating, sectional, and lounge pieces. The Sedona wicker outdoor sectional furniture group photo shows Canadian-made outdoor cushions with Sunbrella® 1046-93 Charcoal Sand cushions, pillows in 18″ and 21″x13″ Sunbrella® 51684-4 Carrara Maritime and 18″ Sunbrella® 5453 Canvas.',
  {'entities': [(346, 375, 'PRODUCT'), (535, 576, 'PRODUCT')]}),
 ("The M36022 Southern Cay Loveseat is made of heavy-walled aluminum extrusions and can be finished in any of Texacraft's beautiful powder coat finishes. The Contemporary styling of the M36022 Loveseat is designed for beauty and comfort. The virgin vinyl strap cushion seat substructure provides hours or relaxing comfort for up to two people. Choose from any of Texacraft's beautiful cushion fabrics for the perfect look.",
  {'entities': [(4, 32, 'PRODUCT'), (183, 198, 'PRODUCT')]}),
 ('The Structure Stationary 2-Seat Loveseat brings bold lines and premium comfort to your outdoor commercial space. Its wide extruded aluminum frame is built for year-round use in high-use settings and is customizable in your selection of finish. The Structure Stationary 2-Seat Loveseat is available in a premium cushion offered in a vast selection of fabrics.',
  {'entities': [(4, 40, 'PRODUCT'), (248, 284, 'PRODUCT')]}),
 ('Detail: Elegant Cast Aluminum Patio Furniture, Durable Aluminum Frames, North American Brand Powder-Coat Paint. Cushions Shown 8067 Celeste, Pillows 8065 Carnegie Celeste (not included). The Deep Seating Group Picture Cushions Shown 8067 Celeste, Pillows 8065 Carnegie Celeste.',
  {'entities': [(8, 45, 'PRODUCT')]}),
 ('The York balcony furniture collection is smaller sized but big in comfort. This modern design luxury patio furniture will work on a downtown toronto condo balcony, hotel or restaurant patio. Choose from patio conversation sets and sectional combinations. Canadian made outdoor cushions shown are 1043-929 Home Sand. The deep seating group picture cushions shown are 1043-929 Home Sand, Pillows 21″*13″ 1047-24 Rigato, 18″ 1045-79 Italian Sand.',
  {'entities': [(4, 26, 'PRODUCT')]}),
 ("Vivere's Chain Hanging Kit is a safe and easy way to hang your hammock from trees or wooden posts, each item in the kit is zinc coated to withstand the elements of nature. The kit is included with all of our spreader bar hammocks and contains all the hardware necessary to hang your hammock",
  {'entities': [(0, 26, 'PRODUCT'),
    (116, 119, 'PRODUCT'),
    (176, 179, 'PRODUCT')]}),
 ("The Nexus Loveseat is created from an outdoor resin weave material in a neutral Rosewood weave color woven over a powder coated aluminum frame. The Nexus Loveseat comfort is attributed to the two piece seat and back cushions available in any of Texacraft' s hundreds of fabrics. Add some optional throw pillows for accent coloration.",
  {'entities': [(4, 18, 'PRODUCT'), (148, 162, 'PRODUCT')]}),
 ('The Apex Chaise Lounge patio furniture by CabanaCoast® is part of the Apex collection which consists of modern contemporary cast aluminum patio furniture that features deep seating. The Apex Chaise Lounge is perfect for any outdoor setting.',
  {'entities': [(4, 38, 'PRODUCT'),
    (138, 153, 'PRODUCT'),
    (186, 204, 'PRODUCT')]}),
 ('A great addition to help move Vivere’s 15ft 3-Beam hammock stand or any of our 8ft and 9ft hammock combos.  Move with ease as you follow the sun or shade throughout the day, or even when you need to mow your lawn.  The heavy duty wheel attaches in minutes with 2 bolts and 2 nuts.  Wheel measures 6 in diameter.  Comes with bolts that are 3 cm (1.18)long designed to accommodate most tubular hammock stands.',
  {'entities': [(30, 58, 'PRODUCT')]}),
 ('The Alon Arm Lounge Chair creates a unique lounging experience for guests. Its commercial grade extruded aluminum frame is extremely durable while remaining lightweight enough to easily rearrange as needed. The Alon Arm Lounge Chair’s airy Resinweave is ideal for any climate and customizable with a wide range of powder coated finishes.',
  {'entities': [(4, 25, 'PRODUCT'), (211, 232, 'PRODUCT')]}),
 ("Vivere's plate hook has been designed to conveniently install on a ceiling, veranda, pergola or gazebo. By using the 8 wood screws, securely fasten the hook into a support beam that will be able to handle a load of at least 450 lbs. Please consult an engineer if you are unclear about the load bearing potential of a given post or beam to be used.",
  {'entities': [(0, 19, 'PRODUCT')]}),
 ('The Texawood Adirondack Lounge Rocker combines classic comfort and innovative manufacturer. Guests will love its durable design and relaxing rocking motion and you’ll love that it doesn’t require any waterproofing or painting',
  {'entities': [(4, 37, 'PRODUCT')]})
  ]

## Train-test split and data preparation (DocBin creation)

A 85%-15% train-test split was used. In order to make sure the data has a proper format for Spacy, two DocBin files were created for both the training and the test data.

In [5]:
len(data)

421

In [6]:
train_data, test_data= train_test_split(data, random_state=31,test_size=0.15, shuffle=True)

In [7]:
len(train_data)

357

In [8]:
len(test_data)

64

In [9]:
nlp = spacy.blank("en") # load a new spacy model
#nlp = spacy.load("en_core_web_sm") # load other spacy model

db = DocBin() # create a DocBin object

for text, annot in tqdm(train_data): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

os.chdir(r'C:\Users\vladi\NER\NER_2') 
db.to_disk("./train_data.spacy") # save the docbin object

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 357/357 [00:00<00:00, 1341.57it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


In [10]:
db = DocBin() # create a DocBin object

for text, annot in tqdm(test_data): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("./test_data.spacy")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 64/64 [00:00<00:00, 1418.97it/s]

Skipping entity


## Running the model and evaluating the performance

Firstly, the model is initiated. It is important to save base_config.cfg in the working directory. The config.cfg file is created, which contains some of the hyperparameters (such as the number of epochs). The default settings were used.

In [11]:
import spacy.cli
spacy.cli.download("en_core_web_lg")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [9]:
!pip install spacy-transformers

In [10]:
!python -m spacy init fill-config base_config.cfg config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy 

In [11]:
!spacy evaluate ./output/model-best/ train_data.spacy  

[i] Using CPU

================================== Results ==================================

TOK     100.00
NER P   99.18 
NER R   99.59 
NER F   99.38 
SPEED   220   


=============================== NER (per type) ===============================

              P       R       F
PRODUCT   99.18   99.59   99.38

[i] Using CPU

================================== Results ==================================

TOK     100.00
NER P   81.68 
NER R   81.06 
NER F   81.37 
SPEED   186   


=============================== NER (per type) ===============================

              P       R       F
PRODUCT   81.68   81.06   81.37

[i] Using CPU

================================== Results ==================================

TOK     100.00
NER P   99.18 
NER R   99.59 
NER F   99.38 
SPEED   255   


=============================== NER (per type) ===============================

              P       R       F
PRODUCT   99.18   99.59   99.38



In [12]:
!spacy evaluate ./output/model-best/ test_data.spacy  

[i] Using CPU

================================== Results ==================================

TOK     100.00
NER P   81.68 
NER R   81.06 
NER F   81.37 
SPEED   266   


=============================== NER (per type) ===============================

              P       R       F
PRODUCT   81.68   81.06   81.37



The F1 score of the best model on the test data is around 81%.

## Using the model to make predictions

In [13]:
nlp_ner = spacy.load(r"C:\Users\vladi\NER\NER_2/output/model-best")

In [14]:
doc = nlp_ner("The Texawood Adirondack Lounge Rocker combines classic comfort and innovative manufacturer. Guests will love its durable design and relaxing rocking motion and you’ll love that it doesn’t require any waterproofing or painting") # input sample text

spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter